01 Define dataset index

In [1971]:
import pandas as pd
from typing import Union, Tuple, List, Dict
import logging
import re
import numpy as np
import string
from IPython.display import display, HTML
from datetime import datetime, date

desired_dataset_index = input("Insert number for dataset or Database.table for table")

print(desired_dataset_index)
  
try:
    desired_dataset_index.isdigit()
    desired_dataset_index = int(desired_dataset_index)
    print(f"Dataset#: {desired_dataset_index} ")
    DB_or_dataset = "Dataset"
    datasets_file_path = 'Fiftydatasets.xlsx'  
    analysed_columns_file_path = 'AnalysedColumns.xlsx'     
except ValueError:
    print(f"Database.table: {desired_dataset_index} ")
    DB_or_dataset = "DB"
    datasets_file_path = 'all_selected_databases_info.xlsx'
    analysed_columns_file_path = 'AnalysedColumnsDB.xlsx'

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Load the sheet to obtain the details of all datasets
datasets_df = pd.read_excel(datasets_file_path)
datasets_df.head()

analysed_columns_df = pd.read_excel(analysed_columns_file_path)


222
Dataset#: 222 
Last run on: 2025-04-04 19:28:20


In [1972]:
analysed_columns_df = pd.read_excel(analysed_columns_file_path)

# Display the first few rows to verify the loaded data
print(f"desired_dataset_index: {desired_dataset_index} Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
analysed_columns_df.head()


desired_dataset_index: 222 Last run on: 2025-04-04 19:28:21


,index,name,area,Original Column,ID,Column,Description,CleanedColumn,ColumnKeyword,ColumnFormat,DescriptionKeyword,DescriptionFormat,FinalFormat,SourceKeyword,FinalFormat.1,SourceKeyword.1,Unnamed: 16,Unnamed: 17
0,52,Iris,Life,1. sepal length in cm,1,sepal length in cm,NaN,sepal length in cm,length,numerical>=0,NaN,NaN,numerical>=0,length,numerical>=0,length,True,True
1,52,Iris,Life,2. sepal width in cm,2,sepal width in cm,NaN,sepal width in cm,width,numerical>=0,NaN,NaN,numerical>=0,width,numerical>=0,width,True,True
2,52,Iris,Life,3. petal length in cm,3,petal length in cm,NaN,petal length in cm,length,numerical>=0,NaN,NaN,numerical>=0,length,numerical>=0,length,True,True
3,52,Iris,Life,4. petal width in cm,4,petal width in cm,NaN,petal width in cm,width,numerical>=0,NaN,NaN,numerical>=0,width,numerical>=0,width,True,True
4,52,Iris,Life,5. class,5,class,NaN,class,class,categorical,NaN,NaN,categorical,class,categorical,class,True,True


025 Get Database Table Content

In [1973]:
import pymysql
import pandas as pd

def get_db_table_content(index: str) -> pd.DataFrame:
    """
    Connect to the database and retrieve the content of the specified table.
    
    Parameters:
    - index (str): The index in the format 'database.table'
    
    Returns:
    - pd.DataFrame: The content of the table as a DataFrame
    """
    host = 'db.relational-data.org'
    port = 3306
    user = 'guest'
    password = 'relational'

    db_name, table_name = index.split('.')
    
    try:
        connection = pymysql.connect(host=host, port=port, user=user, password=password, database=db_name)
        with connection.cursor() as cursor:
            cursor.execute(f"SELECT * FROM `{table_name}`")
            data = cursor.fetchall()
            columns = [column[0] for column in cursor.description]
        
        df = pd.DataFrame(data, columns=columns)
        print(f"Successfully retrieved data from {index}")
        return df
    except Exception as e:
        print(f"Error retrieving data from {index}: {str(e)}")
        return pd.DataFrame()
    finally:
        if 'connection' in locals():
            connection.close()

03 Get dataset file URL

In [1974]:
import pandas as pd
from typing import Union, Tuple, List
import logging

logging.basicConfig(level=logging.INFO)

def get_dataset_file_url(excel_file_path: str, dataset_index: Union[int, str], DB_or_dataset: str) -> Union[Tuple[str, str, List[str]], Exception]:
    """
    Load dataset details from an Excel file and return the dataset file URL and name for a specific dataset index.

    Parameters:
    - excel_file_path (str): The path to the Excel file containing dataset details.
    - dataset_index (int or str): The index of the dataset or the database table name.
    - DB_or_dataset (str): Indicates whether we're dealing with a dataset or a database table.

    Returns:
    - Tuple or Exception: A tuple containing the dataset file URL, name, and primary key columns (for DB only), or an exception if something goes wrong.
    """
    try:
        datasets_df = pd.read_excel(excel_file_path)
        
        if 'index' not in datasets_df.columns or 'name' not in datasets_df.columns:
            return "Required columns 'index' and 'name' do not exist in the DataFrame"

        if DB_or_dataset == "Dataset":
            # Dataset case
            dataset_index = int(dataset_index)
            dataset_details = datasets_df.loc[datasets_df['index'] == dataset_index]
            if dataset_details.empty:
                return f"No dataset found with index {dataset_index}"

            dataset_name = dataset_details['name'].values[0]
            dataset_file_url = dataset_details['dataset_file_url'].values[0]
            
            # Datasets don't have primary keys
            primary_key_columns = []

        elif DB_or_dataset == "DB":
            # Database table case
            dataset_details = datasets_df.loc[datasets_df['index'] == dataset_index]
            if dataset_details.empty:
                return f"No dataset found with index {dataset_index}"

            dataset_name = dataset_details['name'].values[0]
            dataset_file_url = dataset_index  # For database tables, the index is the URL
            
            # Extract primary key columns for DB tables
            if 'primary_key' in datasets_df.columns:
                primary_key = dataset_details['primary_key'].values[0]
                primary_key_columns = [col.strip() for col in str(primary_key).split(',')] if pd.notna(primary_key) else []
            else:
                primary_key_columns = []
                print("No primary key column found in the DataFrame")
        
        else:
            return f"Invalid DB_or_dataset value: {DB_or_dataset}"

        return (dataset_file_url, dataset_name, primary_key_columns)

    except Exception as e:
        logging.error(f"An error occurred: {e}")
        return e

# Example usage
if __name__ == "__main__":
    excel_file_path = datasets_file_path
    dataset_details = get_dataset_file_url(excel_file_path, desired_dataset_index, DB_or_dataset)
    
    if isinstance(dataset_details, tuple):
        dataset_file_url, dataset_name, primary_key_columns = dataset_details
        print(f"The dataset file URL or database index for dataset {dataset_name} (index {desired_dataset_index}) is: {dataset_file_url}")
        if DB_or_dataset == "DB":
            print(f"Primary key columns: {primary_key_columns}")
        print(DB_or_dataset)
    else:
        print(f"An error occurred or the dataset was not found: {dataset_details}")

The dataset file URL or database index for dataset Bank Marketing (index 222) is: https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
Dataset


04 Load dataset

In [1975]:
import pandas as pd
import requests
import io
from io import BytesIO
import zipfile
import tarfile
import gzip
import logging
import csv
from typing import Optional, Union
import os
import tempfile
import shutil
import sys

logging.basicConfig(level=logging.INFO)

def is_header_for_csv(line, delimiter=' '):
    """
    Determine if a line is likely a header by checking if there are no numeric values.
    If at least one numeric value is found, the line is considered not a header (i.e., a data line).
    """
    # Regex to match quoted strings or non-whitespace sequences
    pattern = re.compile(r'\".*?\"|\S+')

    elements = pattern.findall(line.replace(delimiter, ' '))  # Replace delimiter with space for easier parsing

    # Check if any element is a number
    is_numeric_present = any(element.replace('.', '', 1).lstrip('-').isdigit() for element in elements)

    # Check for replicated elements
    unique_elements = set(elements)
    replicated_elements = len(elements) - len(unique_elements)

    # If no numeric value is found and there are replicated elements, consider this a header line
    is_header = not is_numeric_present and replicated_elements == 0

    return is_header


def load_csv(file_content: Union[str, bytes], header: Optional[int], na_values: Optional[Union[str, list]]) -> pd.DataFrame:
    # Adjust to decode bytes if necessary
    if isinstance(file_content, bytes):
        try:
            file_content_decoded = file_content.decode('utf-8')
        except UnicodeDecodeError:
            file_content_decoded = file_content.decode('ISO-8859-1')
    else:
        file_content_decoded = file_content
    
    # Preprocess to replace multiple tabs with a single tab
    #file_content_processed = re.sub('\t+', '\t', file_content_decoded)
    file_content_processed = re.sub('[ \t]+', ' ', file_content_decoded)

    file_stream = io.StringIO(file_content_processed)
        
    try:
        dialect = csv.Sniffer().sniff(file_stream.readline())
        delimiter = dialect.delimiter
        file_stream.seek(0)
    except csv.Error:
        delimiter = ','
        logging.info("Falling back to default delimiter ',' due to detection failure.")
  
    # Use the heuristic to decide if the first line is likely a header
    first_line = file_stream.readline()
    # Print the first row for inspection
    print("First row for inspection:", first_line)
    is_header_row = is_header_for_csv(first_line, delimiter)
    print('Is first row header ?',is_header_row )
    file_stream.seek(0)  # Reset to start of file after reading the first line
    
    header_decision = 0 if is_header_for_csv(first_line, delimiter) else None
    #header_decision = None if is_data_row else 0
    # Regex pattern to match quoted strings or non-whitespace sequences
    pattern = re.compile(r'\".*?\"|\S+')

    df = pd.read_csv(file_stream, header=header_decision, delimiter=delimiter, na_values=na_values, keep_default_na=False)
   
  # Check if the DataFrame needs re-parsing with regex pattern
    if len(df.columns) < 2:
        file_stream.seek(0)
        lines = file_stream.readlines()
        parsed_data = [pattern.findall(line) for line in lines]

        if header is None and header_decision == 0:
            header_row = parsed_data.pop(0)  # Use the first row as header
        else:
            header_row = None  # Let pandas create default headers or use provided header index

        df = pd.DataFrame(parsed_data, columns=header_row)
        df = df.apply(pd.to_numeric, errors='ignore')  # Attempt to correct data types
    return df

def is_header_for_excel(first_row: pd.DataFrame) -> bool:
    """
    Determines if the first row of a DataFrame is likely to be a header by checking the entire row's content.

    Args:
        first_row (pd.DataFrame): DataFrame containing at least one row of data to inspect.

    Returns:
        bool: True if the first row is likely a header (i.e., contains mostly non-numeric data), False if it's likely data.
    """
    # Check if the majority of the items in the first row are non-numeric
    non_numeric_count = first_row.iloc[0].apply(lambda x: not isinstance(x, (int, float))).sum()
    
    # Determine if the first row is likely a header based on the proportion of non-numeric items
    is_header = non_numeric_count > len(first_row.columns) / 2
    return is_header


def load_excel(file_content: Union[str, bytes, io.BytesIO], na_values: Optional[Union[str, list]] = None, skip_rows: Union[int, list] = 0, parse_dates: bool = False) -> pd.DataFrame:
    # Ajuste para o file_content ser um objeto BytesIO, se for bytes
    if isinstance(file_content, bytes):
        file_content = BytesIO(file_content)  # Envolve bytes em BytesIO
    try:
        # Load the first row to check if it's likely to be a header
        first_row = pd.read_excel(file_content, nrows=1, header=None)
        
        # Print the first row for inspection
        print("First row for inspection:", first_row.iloc[0].values)

        # Execute the heuristic
        likely_header = is_header_for_excel(first_row)

        # Decide on using the first row as header based on heuristic
        #header_decision = None if likely_header else 0  
        header_decision = 0 if likely_header else None

        # Reset file_content to read from the beginning if it's a stream
        if isinstance(file_content, io.BytesIO):
            print("Resetting stream position")  # Diagnostic print
            file_content.seek(0)

        # Load the full Excel file with determined header option
        df = pd.read_excel(file_content, na_values=na_values, skiprows=skip_rows, header=header_decision, parse_dates=parse_dates)
        return df
    except ValueError as e:
        logging.error(f"Excel loading error: {e}")
        return pd.DataFrame()
        
def download_and_extract(url: str) -> str:
    """
    Downloads an archive from the given URL and extracts it into a temporary directory.
    Handles .zip, .tar.gz, and .gz files. Returns the path to the directory or file.
    """
    temp_dir = tempfile.mkdtemp()
    response = requests.get(url, stream=True)
    file_name = os.path.basename(url)
    temp_file_path = os.path.join(temp_dir, file_name)
    
    with open(temp_file_path, 'wb') as file:
        shutil.copyfileobj(response.raw, file)
    
    if file_name.endswith('.zip'):
        with zipfile.ZipFile(temp_file_path, 'r') as archive:
            archive.extractall(temp_dir)
    elif file_name.endswith(('.tar.gz', '.tgz')):
        with tarfile.open(temp_file_path, 'r:gz') as archive:
            archive.extractall(temp_dir)
    elif file_name.endswith('.gz'):
        # Handle single .gz files by extracting to the same directory
        extracted_file_path = temp_file_path[:-3]  # Remove .gz extension
        with gzip.open(temp_file_path, 'rb') as f_in, open(extracted_file_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
        os.remove(temp_file_path)  # Remove the original .gz file
        return extracted_file_path  # Direct path to the extracted file for .gz
    
    os.remove(temp_file_path)  # Clean up archive file after extraction
    return temp_dir  # Path to directory with extracted files

def select_file_from_extracted(directory: str) -> Optional[str]:
    """
    Recursively lists all files in the extracted directory and prompts the user to select one.
    """
    files_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            files_list.append(os.path.join(root, file))

    if not files_list:
        logging.error("No files found in the directory.")
        return None

    for index, file in enumerate(files_list, start=1):
        relative_path = os.path.relpath(file, directory)  # Show relative path for clarity
        print(f"{index}: {relative_path}", flush=True)

    try:
        file_index = int(input("Enter the number of the file you want to load: ")) - 1
        if 0 <= file_index < len(files_list):
            selected_relative_path = os.path.relpath(files_list[file_index], directory)  # Get relative path
            print(f"Selected File: {selected_relative_path}")  # Print relative path
            return files_list[file_index]  # Return full path for further processing
        else:
            logging.error("Invalid selection.")
    except ValueError:
        logging.error("Please enter a valid number.")
    
    return None

def fetch_file_content(url: str) -> str:
    response = requests.get(url, stream=True)
    try:
        content = response.content.decode('utf-8')
    except UnicodeDecodeError:
        content = response.content.decode('ISO-8859-1')
    return content

   
def load_dataset(file_path_or_url: str, DB_or_dataset: str, na_values: Optional[Union[str, list]] = None, skip_rows: Union[int, list] = 0, parse_dates: bool = False) -> Union[pd.DataFrame, str]:

    if DB_or_dataset == "DB":
        try:
            df = get_db_table_content(file_path_or_url)
            if len(primary_key_columns) > 1:
                composite_pk_name = '|'.join(primary_key_columns)
                df[composite_pk_name] = df[primary_key_columns].astype(str).agg('|'.join, axis=1)
                # Move the composite key column to the front
                cols = df.columns.tolist()
                cols = [composite_pk_name] + [col for col in cols if col != composite_pk_name]
                df = df[cols]
            return df
        except Exception as e:
            logging.error(f"Database load error: {e}")
            return None
        
    # Handle files or URLs       
    elif file_path_or_url.startswith(('http://', 'https://')):
        if file_path_or_url.endswith(('.csv', '.txt', '.data')):
            response = requests.get(file_path_or_url)
            file_content = response.content
            return load_csv(file_content, header=None, na_values=na_values)
        elif file_path_or_url.endswith(('.xlsx', '.xls')):
            response = requests.get(file_path_or_url)
            return load_excel(response.content, na_values=na_values, skip_rows=skip_rows, parse_dates=parse_dates)
    else:
        # Handle local file path
        if os.path.exists(file_path_or_url):
            if file_path_or_url.endswith(('.csv', '.txt', '.data')):
                with open(file_path_or_url, 'r', encoding='utf-8') as f:
                    file_content = f.read()
                return load_csv(file_content, header=None, na_values=na_values)
            elif file_path_or_url.endswith(('.xlsx', '.xls')):
                with open(file_path_or_url, 'rb') as f:
                    file_content = f.read()
                return load_excel(file_content, na_values=na_values, skip_rows=skip_rows, parse_dates=parse_dates)
        else:
            logging.error(f"File does not exist: {file_path_or_url}")
            return None
    # Handling archives
    extracted_path = download_and_extract(file_path_or_url)
    if os.path.isdir(extracted_path):
        selected_file = select_file_from_extracted(extracted_path)
        if selected_file and selected_file.endswith(('.csv', '.txt', '.data')):
            with open(selected_file, 'rb') as f:
                file_content = f.read()
            return load_csv(file_content, header=None, na_values=na_values)
        elif selected_file and selected_file.endswith(('.xlsx', '.xls')):
            with open(selected_file, 'rb') as f:
                file_content = f.read()
            return load_excel(file_content, na_values=na_values, skip_rows=skip_rows, parse_dates=parse_dates)
    elif os.path.isfile(extracted_path):
        if extracted_path.endswith(('.csv', '.txt', '.data')):
            with open(extracted_path, 'rb') as f:
                file_content = f.read()
            return load_csv(file_content, header=None, na_values=na_values)
        elif extracted_path.endswith(('.xlsx', '.xls')):
            with open(extracted_path, 'rb') as f:
                file_content = f.read()
            return load_excel(file_content, na_values=na_values, skip_rows=skip_rows, parse_dates=parse_dates)
        else:
            # Attempt to load the file without assuming an extension, particularly useful for .gz extracted files
            with open(extracted_path, 'rb') as f:
                file_content = f.read()
            try:
                # Attempt to load as CSV first; this part assumes CSV if no extension is found
                return load_csv(file_content, header=None, na_values=na_values)
            except Exception as e:
                logging.error(f"Failed to automatically determine file type for {extracted_path}: {e}")
                return None
    return None

# Example usage
if __name__ == "__main__":
    excel_file_path = datasets_file_path
    dataset_details = get_dataset_file_url(excel_file_path, desired_dataset_index, DB_or_dataset)

    if isinstance(dataset_details, tuple):
        dataset_file_url, dataset_name, primary_key_columns = dataset_details
        print(f"The dataset file URL or database index for dataset {dataset_name} (index {desired_dataset_index}) is: {dataset_file_url}")
        print(f"Primary key columns: {primary_key_columns}")
        print(DB_or_dataset)
        dataset_df = load_dataset(dataset_file_url, DB_or_dataset, primary_key_columns)
        if dataset_df is not None:
            print("Dataset loaded successfully.")
            if len(primary_key_columns) > 1:
                composite_pk_name = '|'.join(primary_key_columns)
                print(f"\nComposite Primary Key ({composite_pk_name}):")
                print(dataset_df[[composite_pk_name] + [col for col in dataset_df.columns if col != composite_pk_name]].head())
            else:
                print(dataset_df.head())
        else:
            print("Failed to load dataset.")
    else:
        print(f"An error occurred or the dataset was not found: {dataset_details}")
       
from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        

The dataset file URL or database index for dataset Bank Marketing (index 222) is: https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
Primary key columns: []
Dataset
1: bank-additional\.DS_Store
2: bank-additional\.Rhistory
3: bank-additional\bank-additional-full.csv
4: bank-additional\bank-additional-names.txt
5: bank-additional\bank-additional.csv
6: __MACOSX\._bank-additional
7: __MACOSX\bank-additional\._.DS_Store
Selected File: bank-additional\bank-additional-full.csv
First row for inspection: "age";"job";"marital";"education";"default";"housing";"loan";"contact";"month";"day_of_week";"duration";"campaign";"pdays";"previous";"poutcome";"emp.var.rate";"cons.price.idx";"cons.conf.idx";"euribor3m";"nr.employed";"y"

Is first row header ? True
Dataset loaded successfully.
   age        job  marital    education  default housing loan    contact  \
0   56  housemaid  married     basic.4y       no      no   no  telephone   
1   57   services  married  high

05 Assign Column names

In [1976]:
def assign_column_names(analysed_columns_df: pd.DataFrame, dataset_index: Union[int, str], dataset_df: pd.DataFrame, dataset_name: str = "Unknown") -> Union[pd.DataFrame, Exception]:
    """
    Assign column names to a DataFrame based on a given dataset index from an "AnalysedColumns" DataFrame.

    Parameters:
    - analysed_columns_df (pd.DataFrame): The DataFrame containing analysed columns information.
    - dataset_index (Union[int, str]): The index of the dataset for which to get the column names.
    - dataset_df (pd.DataFrame): The DataFrame to which the column names will be assigned.
    - dataset_name (str): The name of the dataset. Default is "Unknown".

    Returns:
    - pd.DataFrame or Exception: The DataFrame with assigned column names or an exception if something goes wrong.
    """
    print(f"Assigning column names for dataset index: {dataset_index}")
    print(f"Type of dataset_index: {type(dataset_index)}")
    print(f"Shape of dataset_df: {dataset_df.shape}")
    print(f"Current columns of dataset_df: {dataset_df.columns.tolist()}")
    
    try:
        # Check if 'index' and 'Column' columns exist in the DataFrame
        if 'index' not in analysed_columns_df.columns or 'Column' not in analysed_columns_df.columns:
            return f"Required columns 'index' or 'Column' do not exist in the DataFrame"

        # Extract the column names for the given dataset index
        column_names = analysed_columns_df.loc[analysed_columns_df['index'] == dataset_index, 'Column'].tolist()
        print(f"Extracted column names: {column_names}")
        
        if not column_names:
            print(f"No column names found. Unique values in 'index' column: {analysed_columns_df['index'].unique()}")
            return f"No column names found for dataset index {dataset_index}"

        # Check if there's a mismatch in the number of columns
        if len(column_names) != len(dataset_df.columns):
            print(f"Warning: Mismatch in number of columns. DataFrame has {len(dataset_df.columns)} columns, but {len(column_names)} names were provided.")
            print(f"Current DataFrame columns: {dataset_df.columns.tolist()}")
            print(f"Provided column names: {column_names}")

            # If there's an extra column (likely the composite primary key), keep it
            if len(dataset_df.columns) == len(column_names) + 1 and '|' in dataset_df.columns[0]:
                composite_pk_name = dataset_df.columns[0]
                new_column_names = [composite_pk_name] + column_names
                dataset_df.columns = new_column_names
                print(f"Kept composite primary key column '{composite_pk_name}' and assigned provided names to other columns.")
                
                # Remove the individual columns that make up the composite key
                individual_pk_columns = composite_pk_name.split('|')
                dataset_df = dataset_df.drop(columns=individual_pk_columns)
                print(f"Removed individual primary key columns: {individual_pk_columns}")
            else:
                return f"Cannot assign column names due to mismatch. Please check the column definitions."
        else:
            # Assign the extracted column names to the target DataFrame
            dataset_df.columns = column_names

        logging.info(f"Successfully assigned column names to the dataset '{dataset_name}' for index {dataset_index}")
        print(f"Final columns of dataset_df: {dataset_df.columns.tolist()}")
        return dataset_df
    except Exception as e:
        logging.error(f"An error occurred while assigning column names to the dataset '{dataset_name}': {e}")
        print(f"Error details: {str(e)}")
        return e
    
# Assume analysed_columns_df contains the analysed columns information
# Assume target_df is the DataFrame loaded previously
# Assume dataset_name contains the name of the dataset

# Call the function to assign the column names
dataset_df = assign_column_names(analysed_columns_df, desired_dataset_index, dataset_df, dataset_name)

# Display the first few rows to verify the column names have been correctly assigned


from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

dataset_df


INFO:root:Successfully assigned column names to the dataset 'Bank Marketing' for index 222


Assigning column names for dataset index: 222
Type of dataset_index: <class 'int'>
Shape of dataset_df: (41188, 21)
Current columns of dataset_df: ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y']
Extracted column names: ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y - has the client subscribed a term deposit?']
Final columns of dataset_df: ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y - has the client subscr

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y - has the client subscribed a term deposit?
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


List of functions in the DataQualityIssues class along with their associated DQI (Data Quality Issue) numbers:

is_blank - Not directly associated with a DQI number

handle_blank_empty_null_nan - DQI #1 (Missing Data - Completeness)

handle_predefined_unacceptable_values - DQI #4 (Ambiguous Data - Accuracy, Consistency)

handle_extraneous_data - DQI #5 (Extraneous Data - Consistency, Uniqueness)

handle_street_extraneous_data - DQI #5 (Extraneous Data - Consistency, Uniqueness)

standardize_date - Not directly associated with a DQI number

standardize_date_time - Not directly associated with a DQI number

is_valid_time - Not directly associated with a DQI number

handle_outdated_temporal_data - DQI #6 (Outdated Temporal Data - Timeliness)

handle_outdated_temporal_data_datetime - DQI #6 (Outdated Temporal Data - Timeliness)

handle_duplicates - DQI #9 (Duplicates - Uniqueness)

handle_excessive_distinct_values - DQI #10 (Structural Conflicts - Consistency, Uniqueness)

handle_dates_format - DQI #14 (Different units/representations - Consistency)

handle_datetimes_format - DQI #14 (Different units/representations - Consistency)

handle_negative_values - DQI #15 (Domain Violation - Accuracy)

handle_values_outside_range - DQI #15 (Domain Violation - Accuracy)

handle_floating_point_values - DQI #15 (Domain Violation - Accuracy)

handle_capitalization_format - DQI #15 (Domain Violation - Accuracy)

handle_short_length_values - DQI #15 (Domain Violation - Accuracy)

handle_invalid_months - DQI #15 (Domain Violation - Accuracy)

handle_invalid_weekdays - DQI #15 (Domain Violation - Accuracy)

handle_street_format - DQI #15 (Domain Violation - Accuracy)

is_date_valid - Not directly associated with a DQI number

handle_invalid_dates - DQI #15 (Domain Violation - Accuracy)

is_structurally_valid_date - Not directly associated with a DQI number

handle_invalid_datetimes - DQI #15 (Domain Violation - Accuracy)

standardize_phone_number - Not directly associated with a DQI number

handle_phone_number_format - DQI #15 (Domain Violation - Accuracy)

handle_ip_format - DQI #15 (Domain Violation - Accuracy)

handle_url_format - DQI #15 (Domain Violation - Accuracy)

is_valid_email - Not directly associated with a DQI number

handle_email_format - DQI #15 (Domain Violation - Accuracy)

handle_binary_values - DQI #15 (Domain Violation - Accuracy)

handle_non_numeric_values - DQI #17 (Wrong Data Type - Consistency)

handle_non_alphanumeric_values - DQI #17 (Wrong Data Type - Consistency)

handle_non_string_values - DQI #17 (Non-String Data Type - Consistency)

handle_alphanumeric_consistency - DQI #17 (Wrong Data Type - Consistency)

handle_uniqueness_violation - DQI #19 (Uniqueness Violation - Uniqueness)

handle_special_characters - DQI #21 (Use of Special Characters - Consistency)

06 Data Quality Issues

In [2004]:
import numpy as np
import pandas as pd
import re
import string
from datetime import datetime, time  

min_valid_year = 1800
max_valid_year = 2100

class DataQualityIssues:

    # Single-character symbols considered as errors
    error_symbols = set(string.punctuation)

    @staticmethod
    def categorize_value(val):
        if pd.isna(val):
            return 'NULL'
        
        val_str = str(val).lower().strip()
        
        if val_str == '':
            return 'Empty'
        elif val_str == 'null':
            return 'NULL'
        elif val_str == 'n/a':
            return 'N/A'
        elif val_str == 'none':
            return 'None'
        elif val_str in ['""', "''"]:
            return 'Empty'
        elif re.match(r'^\s+$', val_str): 
            return 'Blank'
        elif val_str in ['" "', "' '"]:
            return 'Blank'
        elif len(val_str) == 1 and val_str in DataQualityIssues.error_symbols:
            return val_str
        else:
            return 'Valid'


    @staticmethod
    def is_blank(x):
        category = DataQualityIssues.categorize_value(x)
        return category != 'Valid'
    
    @staticmethod
    def represent_value(val):
        category = DataQualityIssues.categorize_value(val)
        if category == 'Valid':
            return str(val)
        else:
            return category

    @staticmethod
    def handle_blank_empty_null_nan(df, column):
        blank_indices = df[df[column].apply(DataQualityIssues.is_blank)].index
        blank_values = [(idx, DataQualityIssues.represent_value(df.loc[idx, column])) for idx in blank_indices]

        total_issues = len(blank_values)

        if total_issues > 0:
            null_values = [v for v in blank_values if v[1] == 'NULL']
            non_null_values = [v for v in blank_values if v[1] != 'NULL']

            message = f"{total_issues} Blank/Empty/Null/Missing value(s) at index(es): "
            
            if len(null_values) == total_issues:
                message += "All NULL values:\n"
                if total_issues > 20:
                    null_display = null_values[:10] + [('...', '...')] + null_values[-10:]
                else:
                    null_display = null_values
                message += f"{null_display}"
            else:
                if null_values:
                    if len(null_values) > 20:
                        null_display = null_values[:10] + [('...', '...')] + null_values[-10:]
                    else:
                        null_display = null_values
                    message += f"\n{len(null_values)} NULL values: {null_display}"
                
                if non_null_values:
                    if len(non_null_values) > 20:
                        non_null_display = non_null_values[:10] + [('...', '...')] + non_null_values[-10:]
                    else:
                        non_null_display = non_null_values
                    message += f"\n{len(non_null_values)} Non-NULL values: {non_null_display}"

            # Prepare data_issues for Summary Sheet
            all_unique_values = set(v[1] for v in blank_values)
            data_issues = ', '.join(f'"{v}"' for v in all_unique_values if v != '...')

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #1 (Missing Data - Completeness)",
                "blank_indices": blank_indices,
                "null_values": null_values,
                "non_null_values": non_null_values,
                "data_issues": data_issues
            }
        return {"issue": False, "blank_indices": []}
       
    @staticmethod
    def handle_predefined_unacceptable_values(df, column):
        predefined_unacceptable_values = ['N/A', 'Invalid', 'unknown','nonexistent']
        unacceptable_indices_and_values = []

        for idx, val in df[column].items():
            val_str = str(val).strip()
            if val_str in predefined_unacceptable_values:
                unacceptable_indices_and_values.append((idx, val))

        total_issues = len(unacceptable_indices_and_values)
        if unacceptable_indices_and_values:
            if total_issues > 20:
                first_10 = unacceptable_indices_and_values[:10]
                last_10 = unacceptable_indices_and_values[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                message = f"{total_issues} Unacceptable value(s) at index(es): {display_list} (displaying only the first and last 10 items)"
            else:
                message = f"{total_issues} Unacceptable value(s) at index(es): {unacceptable_indices_and_values}"

            unacceptable_indices = [idx for idx, _ in unacceptable_indices_and_values]
            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #4 (Ambiguous Data - Accuracy, Consistency)",
                "unacceptable_indices": unacceptable_indices
            }
        return {"issue": False, "unacceptable_indices": []}


    @staticmethod
    def handle_extraneous_data(df, column):
        # Define the function to check for extraneous data
        def has_extraneous_data(x):
            return any(char.isdigit() or char in ['!', '?'] for char in str(x))

        # Get indices and values for extraneous data
        extraneous_data_indices = df[df[column].apply(has_extraneous_data)].index
        extraneous_data_values = df.loc[extraneous_data_indices, column].tolist()
        issue_data = list(zip(extraneous_data_indices, extraneous_data_values))
        total_issues = len(issue_data)

        if issue_data:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = issue_data[:10]
                last_10 = issue_data[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                message = f"{total_issues} Extraneous data value(s) at index(es): {display_list} (displaying only the first and last 10 items)"
            else:
                message = f"{total_issues} Extraneous data value(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "indices": extraneous_data_indices,  # Include all indices directly
                "dq_issue": "DQI #5 (Extraneous Data - Consistency, Uniqueness)"
            }

        return {"issue": False}


    @staticmethod
    def handle_street_extraneous_data(df, column):
        """
        Check for extraneous data in street names, allowing common patterns including numbers,
        hyphens, periods, slashes, and commas which are typical in street addresses, but flagging street names
        composed solely of numbers as errors.
        """
        def has_extraneous_street_data(x):
            # Allow numbers, letters, spaces, hyphens, periods, slashes, and commas
            allowed_chars = string.ascii_letters + string.digits + "' -./,äæúéñßíÅ#"
            # Flag if the string is solely numeric
            if x.isdigit():
                return True
            return any(char not in allowed_chars for char in x)

        extraneous_data_indices = df[df[column].apply(lambda x: isinstance(x, str) and has_extraneous_street_data(x))].index
        extraneous_data_values = df.loc[extraneous_data_indices, column].tolist()
        issue_data = list(zip(extraneous_data_indices, extraneous_data_values))
        total_issues = len(issue_data)

        if issue_data:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = issue_data[:10]
                last_10 = issue_data[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                message = f"{total_issues} Extraneous street data value(s) at index(es): {display_list} (displaying only the first and last 10 items)"
            else:
                message = f"{total_issues} Extraneous street data value(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "indices": extraneous_data_indices,  # Include all indices directly
                "dq_issue": "DQI #5 (Extraneous Data - Consistency, Uniqueness)"
            }

        return {"issue": False}

    @staticmethod
    def convert_to_strftime_format(deduced_format):
        format_mappings = {
            "DDMMYYYY": "%d/%m/%Y",
            "MMDDYYYY": "%m/%d/%Y",
            "YYYYMMDD": "%Y/%m/%d"
        }
        return format_mappings.get(deduced_format, "%Y-%m-%d")  # Default format

        
    @staticmethod
    def standardize_date(date_str, deduced_format):
        month_mapping = {
            'January': '01', 'February': '02', 'March': '03', 'April': '04', 'May': '05', 'June': '06',
            'July': '07', 'August': '08', 'September': '09', 'October': '10', 'November': '11', 'December': '12',
            'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
            'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
        }
        # Remove ordinal suffixes and commas
        date_str = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_str.replace(',', ''))

        for month, num in month_mapping.items():
            date_str = re.sub(r'\b' + month + r'\b', num, date_str, flags=re.IGNORECASE)

        # Split the date string into components
        date_parts = re.split(r'[-/. ]', date_str)

        if len(date_parts) == 3:
            if deduced_format == 'YYYYMMDD':
                standardized_date = date_parts[0][:4] + date_parts[1].zfill(2) + date_parts[2].zfill(2)
            elif deduced_format == 'DDMMYYYY':
                standardized_date = date_parts[0].zfill(2) + date_parts[1].zfill(2) + date_parts[2][:4]
            elif deduced_format == 'MMDDYYYY':
                standardized_date = date_parts[0].zfill(2) + date_parts[1].zfill(2) + date_parts[2][:4]
            else:
                return None  # Format mismatch
        else:
            return None  # Format not recognized    
        
        return standardized_date

    @staticmethod
    def standardize_date_time(date_str, deduced_format):
        
       # Convert to string in case the input is not a string (e.g., float, int)
        date_str = str(date_str)
        
        # Handle dates with ordinal suffixes and commas
        date_str = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_str.replace(',', ''))

        parts = date_str.split(' ')
        
        if len(parts) > 1 and parts[-1] in ["AM", "PM"]:
            time_part = ' '.join(parts[-2:])
            date_part = ' '.join(parts[:-2])
        elif len(parts) > 1 and ':' in parts[-1]:
            time_part = parts[-1]
            date_part = ' '.join(parts[:-1])
        else:
            date_part = date_str
            time_part = '00:00:00'  # Default time for date-only entries

        # Convert textual months to numbers
        month_mapping = {
            'January': '01', 'February': '02', 'March': '03', 'April': '04', 'May': '05', 'June': '06',
            'July': '07', 'August': '08', 'September': '09', 'October': '10', 'November': '11', 'December': '12',
            'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
            'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
        }

        # Only apply month conversion if there's a textual month
        needs_conversion = any(month in date_part for month in month_mapping)

        if needs_conversion:
            for month, num in month_mapping.items():
                date_part = re.sub(r'\b' + month + r'\b', num, date_part, flags=re.IGNORECASE)

        # Process the date part
        date_parts = re.split(r'[-/. ]', date_part)

        if len(date_parts) == 3:
            if deduced_format == 'YYYYMMDD':
                standardized_date = date_parts[0][:4] + date_parts[1].zfill(2) + date_parts[2].zfill(2)
            elif deduced_format == 'DDMMYYYY':
                standardized_date = date_parts[0].zfill(2) + date_parts[1].zfill(2) + date_parts[2][:4]
            elif deduced_format == 'MMDDYYYY':
                standardized_date = date_parts[0].zfill(2) + date_parts[1].zfill(2) + date_parts[2][:4]
            else:
                return None  # Format mismatch
        else:
            return None  # Format not recognized

        return standardized_date + ' ' + time_part if time_part and standardized_date else None

        
    @staticmethod
    def is_valid_time(time_str):
        if isinstance(time_str, time):  
            return True

        if not time_str or pd.isna(time_str):
            return False

        time_str = str(time_str)

        # Check for timedelta-like format
        if re.match(r'\d+ days \d{2}:\d{2}:\d{2}', time_str):
            return True

        time_formats = ['%H:%M', '%I:%M %p', '%H:%M:%S', '%I:%M:%S %p']

        for fmt in time_formats:
            try:
                datetime.strptime(time_str, fmt)
                return True
            except ValueError:
                continue

        return False

    @staticmethod
    def handle_outdated_temporal_data(df, column, min_year, max_year):
        outdated_entries = []
        # Assume a default format for date standardization
        default_format = 'DDMMYYYY'  

        for idx, val in df[column].items():
            standardized_date = DataQualityIssues.standardize_date(str(val), default_format)
            year = None

            # Skip the processing if standardized_date is None
            if standardized_date is None:
                continue

            try:
                if len(standardized_date) == 8 and standardized_date.isdigit():
                    # Check for YYYYMMDD format
                    if int(standardized_date[4:6]) <= 12 and int(standardized_date[6:]) <= 31:
                        year = int(standardized_date[:4])
                        date_obj = datetime(year, int(standardized_date[4:6]), int(standardized_date[6:]))
                    # Check for DDMMYYYY format
                    elif int(standardized_date[:2]) <= 31 and int(standardized_date[2:4]) <= 12:
                        year = int(standardized_date[4:])
                        date_obj = datetime(year, int(standardized_date[2:4]), int(standardized_date[:2]))

                # Check if the year is within the valid range
                if year and not (min_year <= year <= max_year):
                    # Append the date in a readable 'YYYY-MM-DD' format
                    outdated_entries.append((idx, date_obj.strftime('%Y-%m-%d')))

            except ValueError as e:
                #print(f"Error processing date at index {idx}: {e}")
                continue

        total_issues = len(outdated_entries)
        if outdated_entries:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = outdated_entries[:10]
                last_10 = outdated_entries[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                message = f"{total_issues} Date value(s) not in [{min_year}-{max_year}] period at index(es): {display_list} (displaying only the first and last 10 items)"
            else:
                message = f"{total_issues} Date value(s) not in [{min_year}-{max_year}] period at index(es): {outdated_entries}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #6 (Outdated Temporal Data - Timeliness)",
                "outdated_entries": outdated_entries
            }

        return {"issue": False, "outdated_entries": []}
    
    @staticmethod
    def handle_outdated_temporal_data_datetime(df, column, min_year, max_year):
        outdated_entries = []
        # Assume a default format for date standardization
        default_format = 'DDMMYYYY'  

        for idx, val in df[column].items():
            # Split the datetime into date and time parts using standardize_date_time
            standardized_datetime = DataQualityIssues.standardize_date_time(str(val), default_format)
            if standardized_datetime is None:
                continue

            parts = standardized_datetime.split(' ')
            date_part = parts[0]  # The date part is always the first part

            year = None

            # Extract the year from the standardized date
            if len(date_part) == 8 and date_part.isdigit():
                # Check for YYYYMMDD format
                if int(date_part[4:6]) <= 12 and int(date_part[6:]) <= 31:
                    year = int(date_part[:4])
                # Check for DDMMYYYY format
                elif int(date_part[:2]) <= 31 and int(date_part[2:4]) <= 12:
                    year = int(date_part[4:])

            # Check if the year is within the valid range
            if year and not (min_year <= year <= max_year):
                outdated_entries.append((idx, val))

        total_issues = len(outdated_entries)
        if outdated_entries:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = outdated_entries[:10]
                last_10 = outdated_entries[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                message = f"{total_issues} Datetime value(s) not in [{min_year}-{max_year}] period at index(es): {display_list} (displaying only the first and last 10 items)"
            else:
                message = f"{total_issues} Datetime value(s) not in [{min_year}-{max_year}] period at index(es): {outdated_entries}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #6 (Outdated Temporal Data - Timeliness)"
            }

        return {"issue": False}
    
    @staticmethod
    def handle_duplicates(df, column):
        duplicate_values = df[column].duplicated(keep=False)  # Mark all duplicates
        issue_data = df[duplicate_values][column].reset_index().values.tolist()
        total_issues = len(issue_data)

        if issue_data:
            if total_issues > 20:
                first_10 = issue_data[:10]
                last_10 = issue_data[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                message = f"{total_issues} Duplicate value(s) at index(es): {display_list} (displaying only the first and last 10 items)"
            else:
                message = f"{total_issues} Duplicate value(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #9 (Duplicates - Uniqueness)"
            }
        return {"issue": False}
    
    @staticmethod
    def handle_excessive_distinct_values(df, column, threshold=100):
        unique_values_count = df[column].nunique(dropna=False)

        if unique_values_count > threshold:
            sample_values = df[column].dropna().unique()[:10]  # Sample of up to 10 unique values
            return {
                "issue": True,
                "error_message": f"Data seems not categorical or has too many categories (> {threshold}).",
                "dq_issue": "DQI #10 (Structural Conflicts - Consistency, Uniqueness)"
            }
        return {"issue": False}

                
    @staticmethod
    def is_date_valid(date_str, fmt):
        try:
            # Parse the date string based on the provided format
            if fmt == 'YYYYMMDD':
                year, month, day = int(date_str[:4]), int(date_str[4:6]), int(date_str[6:8])
            elif fmt == 'DDMMYYYY':
                day, month, year = int(date_str[:2]), int(date_str[2:4]), int(date_str[4:8])
            elif fmt == 'MMDDYYYY':
                month, day, year = int(date_str[:2]), int(date_str[2:4]), int(date_str[4:8])
            else:
                return False

            # Construct datetime object to validate the date
            date(year, month, day)

            return min_valid_year <= year <= max_valid_year
        except ValueError as e:
            # print(f"Failed date parsing for {date_str} in format {fmt}: {e}")
            # print(f"Date parsing resulted in an error: {e}")
            return False

    @staticmethod
    def is_valid_in_any_format(date_str):
        formats = ['%Y%m%d', '%d%m%Y', '%m%d%Y']  # Add other potential formats
        for fmt in formats:
            try:
                datetime.strptime(date_str, fmt)
                return True
            except ValueError:
                continue
        return False
        
    @staticmethod
    def handle_invalid_dates(df, column):
        invalid_entries = []
        # Assume a default format for date standardization
        default_format = 'DDMMYYYY'  

        for idx, val in df[column].items():
            if isinstance(val, (date, datetime)):
                continue  

            standardized_date = DataQualityIssues.standardize_date(str(val), default_format)
            #print(f"Index: {idx}, Original Value: {val}, Standardized Date: {standardized_date}")
           
            # Skip the validation if standardized_date is None
            if standardized_date is None:
                continue
            
            # Check if date is structurally valid in any format
            if not (DataQualityIssues.is_date_valid(standardized_date, 'DDMMYYYY') or
                    DataQualityIssues.is_date_valid(standardized_date, 'MMDDYYYY') or
                    DataQualityIssues.is_date_valid(standardized_date, 'YYYYMMDD')):
                invalid_entries.append((idx, val))
        
        total_issues = len(invalid_entries)
        if invalid_entries:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = invalid_entries[:10]
                last_10 = invalid_entries[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                # Convert the list to a string while keeping the square brackets
                issue_data = str(display_list)
                message = f"{total_issues} Invalid date value(s) at index(es): {issue_data} (displaying only the first and last 10 items)"
            else:
                # Convert the list to a string while keeping the square brackets
                issue_data = str(invalid_entries)
                message = f"{total_issues} Invalid date value(s) at index(es): {issue_data}"
            
            return {
                "issue": True,
                "error_message": message,
                "dq_issue":  "DQI #13 (Temporal mismatch - Accuracy, Timeliness)"
            }
        
        return {"issue": False}

    @staticmethod
    def is_structurally_valid_date(date_str, expected_format):
        # Check the date in any format for structural validity
        return DataQualityIssues.is_date_valid(date_str, expected_format)

    @staticmethod
    def handle_invalid_datetimes(df, column, deduced_format):
        #print(f"Handling invalid datetimes. Deduced format: {deduced_format}")
        invalid_datetime_errors = []

        for idx, val in df[column].items():
            #print(f"Processing index {idx}, value: {val}")
            standardized_datetime = DataQualityIssues.standardize_date_time(val, deduced_format)

            if not standardized_datetime:
                #print(f"Standardization failed for: {val}")
                invalid_datetime_errors.append((idx, val))
                continue

            parts = standardized_datetime.split(' ')
            date_str = parts[0]
            time_str = parts[1] if len(parts) > 1 else '00:00:00'

            #print(f"Standardized datetime: {standardized_datetime}, Date: {date_str}, Time: {time_str}")

            if not DataQualityIssues.is_valid_time(time_str):
                #print(f"Invalid time for: {val}")
                invalid_datetime_errors.append((idx, val))
            elif not DataQualityIssues.is_structurally_valid_date(date_str, deduced_format):
                #print(f"Date structure invalid for: {val}")
                invalid_datetime_errors.append((idx, val))

        total_issues = len(invalid_datetime_errors)
        if invalid_datetime_errors:
            #print(f"Total invalid datetime values: {total_issues}")
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = invalid_datetime_errors[:10]
                last_10 = invalid_datetime_errors[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                issue_data = str(display_list)
                message = f"{total_issues} Invalid datetime value(s) at index(es): {issue_data} (displaying only the first and last 10 items)"
            else:
                issue_data = str(invalid_datetime_errors)
                message = f"{total_issues} Invalid datetime value(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue":  "DQI #13 (Temporal mismatch - Accuracy, Timeliness)"
            }

        #print("No invalid datetimes found.")
        return {"issue": False}


    
    @staticmethod
    def handle_invalid_times(df, column):
        invalid_time_errors = []
        for idx, time_str in df[column].items():
            if not DataQualityIssues.is_valid_time(time_str):
                invalid_time_errors.append((idx, time_str))

        total_issues = len(invalid_time_errors)
        if invalid_time_errors:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = invalid_time_errors[:10]
                last_10 = invalid_time_errors[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                # Convert the list to a string while keeping the square brackets
                issue_data = str(display_list)
                message = f"{total_issues} Invalid time value(s) at index(es): {issue_data} (displaying only the first and last 10 items)"
            else:
                # Convert the list to a string while keeping the square brackets
                issue_data = str(invalid_time_errors)
                message = f"{total_issues} Invalid time value(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #13 (Temporal mismatch - Accuracy, Timeliness)"
            }

        return {"issue": False}

    @staticmethod
    def deduce_regional_format(date_samples):
        month_mapping = {
            'January': '01', 'February': '02', 'March': '03', 'April': '04', 'May': '05', 'June': '06',
            'July': '07', 'August': '08', 'September': '09', 'October': '10', 'November': '11', 'December': '12',
            'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
            'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
        }
        standardized_dates = []
        format_counts = {'DDMMYYYY': 0, 'MMDDYYYY': 0, 'YYYYMMDD': 0}

        for date in date_samples:
            # Convert textual months to numbers
            for month, num in month_mapping.items():
                date = re.sub(r'\b' + month + r'\b', num, date, flags=re.IGNORECASE)

            # Handle continuous string dates (without separators)
            if len(date) == 8 and date.isdigit():
                # Check if the first four characters represent a plausible year
                if 1800 <= int(date[:4]) <= 2100:
                    format_counts['YYYYMMDD'] += 1
                # Check if the first two characters represent a plausible day and next two a month
                elif 1 <= int(date[:2]) <= 31 and 1 <= int(date[2:4]) <= 12:
                    format_counts['DDMMYYYY'] += 1
                # Check if the first two characters represent a plausible month and next two a day
                elif 1 <= int(date[:2]) <= 12 and 1 <= int(date[2:4]) <= 31:
                    format_counts['MMDDYYYY'] += 1
            else:
                date_parts = re.split(r'[-/. ]', date)
                if len(date_parts) == 3:
                    # Check if the year is the first or last component
                    if len(date_parts[0]) == 4:
                        format_counts['YYYYMMDD'] += 1
                    elif len(date_parts[2]) == 4:
                        format_counts['DDMMYYYY'] += 1
                    else:
                        # Default to MMDDYYYY if year is not first or last
                        format_counts['MMDDYYYY'] += 1

                    standardized_date = '-'.join(date_parts)  # Rejoin the date parts for standardized dates
                    standardized_dates.append(standardized_date)

        preferred_format = max(format_counts, key=format_counts.get)
        print()
        print(f"Deduced format: {preferred_format}")  # Diagnostic print
        print(f"Processed date sample: {date}, detected format counts: {format_counts}")
        print()

        return preferred_format, standardized_dates
    
    @staticmethod
    def handle_dates_format(df, column, expected_format):
        format_errors = []
        for idx, val in df[column].items():
            if isinstance(val, (date, datetime)):
                # If it's already a date or datetime object, it's considered valid
                continue

            standardized_date = DataQualityIssues.standardize_date(str(val), expected_format)
            #print(f"Index: {idx}, Original Value: {val}, Standardized Date: {standardized_date}")

            # Check if standardized_date is None
            if standardized_date is None:
                # If the date can't be standardized, skip it (it will be caught in other checks)
                continue
            
            # Check if the date is valid according to the expected format
            if not DataQualityIssues.is_date_valid(standardized_date, expected_format):
                format_errors.append((idx, val))
        total_issues = len(format_errors)

        if format_errors:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = format_errors[:10]
                last_10 = format_errors[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                message = f"{total_issues} Date value(s) without format '{expected_format}' in [{min_valid_year}-{max_valid_year}] period at index(es): {display_list} (displaying only the first and last 10 items)"
            else:
                message = f"{total_issues} Date value(s) without format '{expected_format}' in [{min_valid_year}-{max_valid_year}] period at index(es): {format_errors}"
           
            return{
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #14 (Different units/representations - Consistency)"
            }
        return {"issue": False}

    @staticmethod
    def handle_datetimes_format(df, column, expected_format):
        format_errors = []
 
        for idx, val in df[column].items():
            standardized_datetime = DataQualityIssues.standardize_date_time(str(val), expected_format)

            if not standardized_datetime:
                print(f"Index {idx}: Standardization failed.")
                continue

            date_str = standardized_datetime.split(' ')[0]
            
             # Check if the date is valid according to the expected format
            if DataQualityIssues.is_valid_in_any_format(date_str) and not DataQualityIssues.is_structurally_valid_date(date_str, expected_format):
                format_errors.append((idx, val))
       
        total_issues = len(format_errors)
        if format_errors:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = format_errors[:10]
                last_10 = format_errors[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                message = f"{total_issues} Datetime value(s) without format '{expected_format}' in [{min_valid_year}-{max_valid_year}] period at index(es): {display_list} (displaying only the first and last 10 items)"
            else:
                message = f"{total_issues} Datetime value(s) without format '{expected_format}' in [{min_valid_year}-{max_valid_year}] period at index(es): {format_errors}"
           
            return{
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #14 (Different units/representations - Consistency)"
            }

        return {"issue": False}

    @staticmethod
    def handle_negative_values(df, column):
        # Function to check if a value is negative
        def is_negative(x):
            try:
                num_val = float(x)
                return num_val < 0  # Check if the value is negative
            except (ValueError, TypeError):
                return False  # Non-numeric values are not considered here

        # Get indices and values for negative values
        negative_indices = df[df[column].apply(is_negative)].index
        negative_values = df.loc[negative_indices, column].tolist()
        issue_data = list(zip(negative_indices, negative_values))
        total_issues = len(issue_data)

        if issue_data:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = issue_data[:10]
                last_10 = issue_data[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                message = f"{total_issues} Negative value(s) at index(es): {display_list} (displaying only the first and last 10 items)"
            else:
                message = f"{total_issues} Negative value(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #15 (Domain Violation - Accuracy)"
            }

        return {"issue": False}

    
    @staticmethod
    def handle_values_outside_range(df, column, min_value, max_value):
        # Function to check if a value is within the specified range
        def is_outside_range(x):
            try:
                num_val = float(x)
                return num_val < min_value or num_val > max_value
            except (ValueError, TypeError):
                return False  # Non-numeric values are not considered here

        # Get indices and values for values outside the specified range
        outside_range_indices = df[df[column].apply(is_outside_range)].index
        outside_range_values = df.loc[outside_range_indices, column].tolist()
        issue_data = list(zip(outside_range_indices, outside_range_values))
        total_issues = len(issue_data)

        if issue_data:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = issue_data[:10]
                last_10 = issue_data[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                message = f"{total_issues} Value(s) outside range [{min_value}, {max_value}] at index(es): {display_list} (displaying only the first and last 10 items)"
            else:
                message = f"{total_issues} Value(s) outside range [{min_value}, {max_value}] at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #15 (Domain Violation - Accuracy)"
            }

        return {"issue": False}
       
           
    @staticmethod
    def handle_floating_point_values(df, column):
        """
        Flags floating-point numbers in a column.

        Parameters:
        - df (pd.DataFrame): The pandas DataFrame to analyze.
        - column (str): The name of the column to check.

        Returns:
        - dict: A dictionary indicating if there are floating-point values, with their indices and values, and the associated DQI.
        """
        floating_point_indices = []
        for idx, value in df[column].items():
            try:
                # If it's a floating point and not an integer
                if float(value) and not float(value).is_integer():
                    floating_point_indices.append((idx, value))
            except ValueError:
                # Ignore non-numeric values, handle them in other checks
                continue

        total_issues = len(floating_point_indices)
        if floating_point_indices:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = floating_point_indices[:10]
                last_10 = floating_point_indices[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                message = f"{total_issues} Floating-point number(s) at index(es): {display_list} (displaying only the first and last 10 items)"
            else:
                message = f"{total_issues} Floating-point number(s) at index(es): {floating_point_indices}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #15 (Domain Violation - Accuracy)"
            }

        return {"issue": False}


    @staticmethod
    def handle_capitalization_format(df, column, linking_words):
        """
        Check if the values in the column adhere to capitalization and format standards.
        """
        def is_capitalization_issue(word):
            # Allow all-uppercase words, ordinal numbers, linking words, and suffixes
            if (word.isupper() or 
                re.match(r"^\d+(st|nd|rd|th)$", word.lower()) or 
                word.lower() in linking_words or
                word.lower() in ['ii', 'iii', 'iv', 'v', 'jr', 'sr']):
                return False
            
            # Special handling for 'PhD' and 'MD'
            if word.lower() in ['phd', 'md', 'c/']:
                return word != word.upper()
            
            # Handle special cases like 'McDonald', 'O'Riley', etc.
            if re.match(r"^(Mc|Mac|O'|D')[A-Z]", word):
                return False

            # Handle possessive forms
            if word.endswith("'s"):
                return is_capitalization_issue(word[:-2])
        
            # For all other words, check if it's title case
            return word != word.title()

        format_issues = []
        for idx, val in df[column].items():
            # Split the string into words, preserving important punctuation
            words = re.findall(r"\b[\w'.,-]+\b|#\S+", str(val))
            if any(is_capitalization_issue(word) for word in words):
                format_issues.append((idx, val))

        total_issues = len(format_issues)
        if format_issues:
            if total_issues > 20:
                first_10 = format_issues[:10]
                last_10 = format_issues[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                issue_data = str(display_list)
                message = f"{total_issues} Capitalization/Format issue(s) at index(es): {issue_data} (displaying only the first and last 10 items)"
            else:
                issue_data = str(format_issues)
                message = f"{total_issues} Capitalization/Format issue(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #15 (Domain Violation - Accuracy)",
                "issue_indices": [idx for idx, _ in format_issues]
            }
        return {"issue": False, "issue_indices": []}

    @staticmethod
    def handle_capitalization_format_country(df, column, linking_words):
        """
        Check if the values in the column adhere to capitalization and format standards,
        with exceptions for certain patterns typical in country names.
        """
        def is_exceptional_case(word, prev_word=None):
            # Directly return False (no issue) for all-uppercase words or ordinal numbers
            if word.isupper() or re.match(r"^\d+(st|nd|rd|th)$", word.lower()):
                return False
            # Check if the word is a linking word or part of an exception pattern (e.g., within parentheses or after a hyphen)
            if word.lower() in linking_words or (prev_word and prev_word.endswith('-')):
                return True
            return False
        
        def is_capitalization_issue(words):
            # Check for capitalization issue while considering exceptions
            for i, word in enumerate(words):
                prev_word = words[i-1] if i > 0 else None
                # Skip words that are part of an exceptional case
                if is_exceptional_case(word, prev_word):
                    continue
                # Identify words that fail the capitalization check (excluding exceptions)
                if not word.istitle() and not word.isupper():
                    return True
            return False

        format_issues = []
        for idx, val in df[column].items():
            original_val = str(val)
            # Split the value considering spaces and treating content in parentheses as a single block
            words = re.findall(r'\b\w+\b', original_val)
            if is_capitalization_issue(words):
                format_issues.append((idx, val))

        total_issues = len(format_issues)
        if format_issues:
            # Prepare the message with a simplified display if there are many issues
            display_list = format_issues[:10] + [('...', '...')] if total_issues > 20 else format_issues
            issue_data = str(display_list)
            message = f"{total_issues} Capitalization/Format issue(s) at index(es): " + issue_data
            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #15 (Domain Violation - Accuracy)"
            }

        return {"issue": False}


    @staticmethod
    def handle_short_length_values(df, column, min_length):
        """
        Flags values in a column that are shorter than the specified minimum length, excluding non-alphanumeric characters.

        Parameters:
        - df (pd.DataFrame): The pandas DataFrame to analyze.
        - column (str): The name of the column to check.
        - min_length (int): The minimum acceptable length for values.

        Returns:
        - dict: A dictionary indicating if there are short values, with their indices and values, and the associated DQI.
        """
        def is_short_and_alphanumeric(x):
            return isinstance(x, str) and len(x) < min_length and x.isalnum()

        short_values = df[df[column].apply(is_short_and_alphanumeric)]
        issue_data = list(zip(short_values.index, short_values[column]))
        total_issues = len(issue_data)

        if issue_data:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = issue_data[:10]
                last_10 = issue_data[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                message = f"{total_issues} Short length alphanumeric value(s) at index(es): {display_list} (displaying only the first and last 10 items)"
            else:
                message = f"{total_issues} Short length alphanumeric value(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #15 (Domain Violation - Accuracy)"
            }

        return {"issue": False}
    
    @staticmethod
    def normalize_month(month_str):
        """
        Normalize the given month string to its full month name if valid.

        Parameters:
        - month_str (str or float): The month string or number to normalize.

        Returns:
        - str/None: Normalized month name if valid, None otherwise.
        """
        try:
            # Convert to string if it's a float
            if isinstance(month_str, float):
                month_str = str(int(month_str))
            else:
                month_str = str(month_str).strip()

            if month_str.replace('.', '').isdigit():
                month_val = int(float(month_str))
                if 1 <= month_val <= 12:
                    return datetime(2000, month_val, 1).strftime('%B')
            elif len(month_str) == 3:
                return datetime.strptime(month_str.title(), '%b').strftime('%B')
            else:
                datetime.strptime(month_str.title(), '%B')
                return month_str.title()
        except ValueError:
            return None
    
    @staticmethod
    def handle_invalid_months(df, column):
        """
        Check if all values in the specified column are valid representations of months.

        Parameters:
        - df (pd.DataFrame): The pandas DataFrame to analyze.
        - column (str): The name of the column to analyze.

        Returns:
        - dict: A dictionary indicating if there are invalid month values, with their indices and values.
        """
        invalid_entries = []
        for idx, val in df[column].items():
            if not DataQualityIssues.normalize_month(str(val)):
                invalid_entries.append((idx, val))

        total_issues = len(invalid_entries)
        if invalid_entries:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = invalid_entries[:10]
                last_10 = invalid_entries[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                message = f"{total_issues} Invalid month value(s) at index(es): {display_list} (displaying only the first and last 10 items)"
            else:
                message = f"{total_issues} Invalid month value(s) at index(es): {invalid_entries}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #15 (Domain Violation - Accuracy)"
            }

        return {"issue": False}

    
    @staticmethod
    def handle_invalid_weekdays(df, column):
        """
        Check if all values in the specified column are valid representations of weekdays.

        Parameters:
        - df (pd.DataFrame): The pandas DataFrame to analyze.
        - column (str): The name of the column to analyze.

        Returns:
        - dict: A dictionary indicating if there are invalid weekday values, with their indices and values.
        """
        invalid_entries = []
        weekdays = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
        weekday_abbr = ["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]
        weekday_abbr_short = ["su", "mo", "tu", "we", "th", "fr", "sa"]

        for idx, val in df[column].items():
            val_str = str(val).strip().lower()
            # Normalize full names and abbreviations to full weekday name
            if val_str.title() in weekdays or val_str.title() in weekday_abbr or val_str in weekday_abbr_short:
                continue
            elif val_str.isdigit() and 0 <= int(val_str) <= 7:
                continue
            else:
                invalid_entries.append((idx, val))

        total_issues = len(invalid_entries)
        if invalid_entries:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = invalid_entries[:10]
                last_10 = invalid_entries[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                message = f"{total_issues} Invalid weekday value(s) at index(es): {display_list} (displaying only the first and last 10 items)"
            else:
                message = f"{total_issues} Invalid weekday value(s) at index(es): {invalid_entries}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #15 (Domain Violation - Accuracy)"
            }

        return {"issue": False}


    @staticmethod
    def handle_street_format(df, column):
        """
        Check if the street names conform to a typical street format.
        """
        invalid_format_indices = []
        street_format_regex = re.compile(r'^[\dA-Za-zÀ-ÖØ-öø-ÿ .,\-\'#]+(?:\s[A-Za-zÀ-ÖØ-öø-ÿ]+)*$')

        for idx, val in df[column].items():
            if not isinstance(val, str) or not street_format_regex.match(val):
                invalid_format_indices.append((idx, val))

        total_issues = len(invalid_format_indices)
        if invalid_format_indices:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = invalid_format_indices[:10]
                last_10 = invalid_format_indices[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                issue_data = ", ".join([f"({idx}, '{value}')" for idx, value in display_list])
                message = f"{total_issues} Incorrect street format issue(s) at index(es): {issue_data} (displaying only the first and last 10 items)"
            else:
                issue_data = ", ".join([f"({idx}, '{value}')" for idx, value in invalid_format_indices])
                message = f"{total_issues} Incorrect street format issue(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #15 (Domain Violation - Accuracy)"
            }

        return {"issue": False}

    @staticmethod
    def standardize_phone_number(phone_number):
        """
        Standardize the phone number by removing common separators.
        """
        return re.sub(r'[()\-+ ]', '', str(phone_number))

    @staticmethod
    def handle_phone_number_format(df, column):
        """
        Check if phone numbers in the specified column conform to expected formats.
        """
        incorrect_format = []

        for idx, phone_number in df[column].items():
            if phone_number is None or isinstance(phone_number, str) and phone_number.strip() == '':
                incorrect_format.append((idx, phone_number))
                continue

            cleaned_number = DataQualityIssues.standardize_phone_number(phone_number)
            
            if not (cleaned_number.isdigit() and 3 <= len(cleaned_number) <= 15):
                incorrect_format.append((idx, phone_number))

        total_issues = len(incorrect_format)
        if incorrect_format:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = incorrect_format[:10]
                last_10 = incorrect_format[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                # Convert the list to a string while keeping the square brackets
                issue_data = str(display_list)
                message = f"{total_issues} Incorrect telephone number format issue(s) at index(es): {issue_data} (displaying only the first and last 10 items)"
            else:
                # Convert the list to a string while keeping the square brackets
                issue_data = str(incorrect_format)
                message = f"{total_issues} Incorrect telephone number format issue(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #15 (Domain Violation - Accuracy)"
            }

        return {"issue": False}



    @staticmethod
    def handle_ip_format(df, column):
        # Regular expression patterns for IPv4 and IPv6
        ipv4_pattern = re.compile(r'^(?:[0-9]{1,3}\.){3}[0-9]{1,3}$')
        ipv6_pattern = re.compile(r'^([0-9a-fA-F]{1,4}:){7}[0-9a-fA-F]{1,4}$')

        incorrect_indices_and_values = []

        for idx, ip in df[column].items():
            ip_str = str(ip).strip()  # Convert to string and strip whitespace
            if not (ipv4_pattern.match(ip_str) or ipv6_pattern.match(ip_str)):
                incorrect_indices_and_values.append((idx, ip))

        total_issues = len(incorrect_indices_and_values)
        if incorrect_indices_and_values:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = incorrect_indices_and_values[:10]
                last_10 = incorrect_indices_and_values[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                # Convert the list to a string while keeping the square brackets
                issue_data = str(display_list)
                message = f"{total_issues} Invalid IP format issue(s) at index(es): {issue_data} (displaying only the first and last 10 items)"
            else:
                # Convert the list to a string while keeping the square brackets
                issue_data = str(incorrect_indices_and_values)
                message = f"{total_issues} Invalid IP format issue(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #15 (Domain Violation - Accuracy)"
            }

        return {"issue": False}

    
    @staticmethod
    def handle_url_format(df, column):
        url_pattern = re.compile(
            r'^(https?:\/\/)?'  # protocol
            r'((([a-zA-Z0-9-]+\.)+[a-zA-Z]{2,})|'  # domain name
            r'localhost|'  # localhost
            r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})'  # or ip
            r'(?::\d+)?(\/[-a-zA-Z0-9@:%_\+.~#?&//=]*)?$'  # port and path
        )
        incorrect_indices_and_values = []

        for idx, url in df[column].items():
            url_str = str(url)  # Convert to string
            if not url_pattern.match(url_str):
                incorrect_indices_and_values.append((idx, url))

        total_issues = len(incorrect_indices_and_values)
        if incorrect_indices_and_values:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = incorrect_indices_and_values[:10]
                last_10 = incorrect_indices_and_values[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                # Convert the list to a string while keeping the square brackets
                issue_data = str(display_list)
                message = f"{total_issues} Invalid URL format issue(s) at index(es): {issue_data} (displaying only the first and last 10 items)"
            else:
                # Convert the list to a string while keeping the square brackets
                issue_data = str(incorrect_indices_and_values)
                message = f"{total_issues} Invalid URL format issue(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #15 (Domain Violation - Accuracy)"
            }

        return {"issue": False}


    
    @staticmethod
    def is_valid_email(email):
        email_str = str(email)  # Convert to string

        # Check the length of the email
        if len(email_str) > 254:
            return False

        # Check if '@' is present
        if '@' not in email_str:
            return False

        # Enhanced email regex pattern
        email_pattern = re.compile(
            r'^[a-zA-Z0-9.!#$%&\'*+/=?^_`{|}~-]+@'  # local part
            r'(?:[a-zA-Z0-9-]+)'  # domain name part (subdomains allowed)
            r'(?:\.[a-zA-Z0-9-]+)*'  # additional subdomains
            r'\.[a-zA-Z]{2,}$'  # TLD part
        )

        # Check for consecutive dots in local part
        if '..' in email.split('@')[0]:
            return False

        # Extract and check the domain part
        domain_part = email.split('@')[1]
        if '--' in domain_part or domain_part.startswith('-') or domain_part.endswith('-'):
            return False

        return email_pattern.match(email) is not None

    @staticmethod
    def handle_email_format(df, column):
        incorrect_indices_and_values = []
        
        for idx, email in df[column].items():
            if not DataQualityIssues.is_valid_email(email):
                incorrect_indices_and_values.append((idx, email))

        total_issues = len(incorrect_indices_and_values)
        if incorrect_indices_and_values:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = incorrect_indices_and_values[:10]
                last_10 = incorrect_indices_and_values[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                # Convert the list to a string while keeping the square brackets
                issue_data = str(display_list)
                message = f"{total_issues} Invalid email format issue(s) at index(es): {issue_data} (displaying only the first and last 10 items)"
            else:
                # Convert the list to a string while keeping the square brackets
                issue_data = str(incorrect_indices_and_values)
                message = f"{total_issues} Invalid email format issue(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #15 (Domain Violation - Accuracy)"
            }

        return {"issue": False}


    @staticmethod
    def handle_binary_values(df, column):
        """
        Check if the values in the specified column are valid binary values or if the column has exactly two unique values.

        Parameters:
        - df (pd.DataFrame): The pandas DataFrame to analyze.
        - column (str): The name of the column to check.

        Returns:
        - dict: A dictionary indicating if there are non-binary values, with their indices and values, and the associated DQI.
        """
        # Calculate unique non-null values
        unique_values = df[column].dropna().unique()
        
        # If there are exactly two unique values, consider it binary and return no issue
        if len(unique_values) == 2:
            return {"issue": False}
        
        # Define acceptable binary values (including lowercase)
        true_values = ['1', 'True', 'Yes', 'T', 'Y']
        false_values = ['0', 'False', 'No', 'F','N']

        non_binary_indices = []

        for idx, val in df[column].items():
            str_val = str(val).strip().capitalize()  # Capitalize to match 'True', 'False', etc.
            if str_val not in true_values + false_values:
                non_binary_indices.append((idx, val))

        total_issues = len(non_binary_indices)
        if non_binary_indices:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = non_binary_indices[:10]
                last_10 = non_binary_indices[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                # Convert the list to a string while keeping the square brackets
                issue_data = str(display_list)
                message = f"{total_issues} Non-binary value(s) at index(es): {issue_data} (displaying only the first and last 10 items)"
            else:
                # Convert the list to a string while keeping the square brackets
                issue_data = str(non_binary_indices)
                message = f"{total_issues} Non-binary value(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #15 (Domain Violation - Accuracy)"
            }

        return {"issue": False}


    
    @staticmethod
    def handle_non_numeric_values(df, column):
        # Define a function to check if a value is non-numeric and not a special placeholder
        def is_non_numeric_and_not_placeholder(x):
            special_placeholders = ["''", '""', "``"]  # List of special placeholder strings
            x_str = str(x).strip()
            if x_str in special_placeholders:
                return False  # Do not flag special placeholders as non-numeric

            try:
                float(x_str)  # Attempt to convert to float
                return False  # Conversion successful, value is numeric
            except (ValueError, TypeError):
                return True  # Conversion failed, value is non-numeric

        # Get indices and values for non-numeric values
        non_numeric_indices = df[df[column].apply(is_non_numeric_and_not_placeholder)].index
        non_numeric_values = df.loc[non_numeric_indices, column].tolist()
        issue_data = list(zip(non_numeric_indices, non_numeric_values))
        total_issues = len(issue_data)

        if issue_data:
            if total_issues > 20:
                first_10 = issue_data[:10]
                last_10 = issue_data[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                message = f"{total_issues} Non-numeric value(s) at index(es): {display_list} (displaying only the first and last 10 items)"
            else:
                message = f"{total_issues} Non-numeric value(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #17 (Wrong Data Type - Consistency)"
            }
        return {"issue": False}


    @staticmethod
    def handle_non_alphanumeric_values(df, column):
        # Define a function to check for strictly non-alphanumeric characters (excluding spaces and hyphens)
        def is_strictly_non_alphanumeric(x):
            return any(char not in string.ascii_letters + string.digits + ' -' for char in x)

        # Get indices and values that are strictly non-alphanumeric
        non_alphanumeric_indices = df[df[column].apply(lambda x: isinstance(x, str) and is_strictly_non_alphanumeric(x))].index
        non_alphanumeric_values = df.loc[non_alphanumeric_indices, column].tolist()
        issue_data = list(zip(non_alphanumeric_indices, non_alphanumeric_values))

        total_issues = len(issue_data)
        if issue_data:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = issue_data[:10]
                last_10 = issue_data[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                # Convert the list to a string while keeping the square brackets
                issue_data = str(display_list)
                message = f"{total_issues} Non-alphanumeric value(s) at index(es): {issue_data} (displaying only the first and last 10 items)"
            else:
                # Convert the list to a string while keeping the square brackets
                issue_data = str(issue_data)
                message = f"{total_issues} Non-alphanumeric value(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #17 (Wrong Data Type - Consistency)"
            }

        return {"issue": False}



    @staticmethod
    def handle_non_string_values(df, column):
        """
        Flags values in a column that are not of string type.

        Parameters:
        - df (pd.DataFrame): The pandas DataFrame to analyze.
        - column (str): The name of the column to check.

        Returns:
        - dict: A dictionary indicating if there are non-string values, with their indices and values, and the associated DQI.
        """
        non_string_indices = df[df[column].apply(lambda x: not isinstance(x, str))].index
        non_string_values = df.loc[non_string_indices, column].tolist()
        issue_data = list(zip(non_string_indices, non_string_values))

        total_issues = len(issue_data)
        if issue_data:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = issue_data[:10]
                last_10 = issue_data[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                # Convert the list to a string while keeping the square brackets
                issue_data = str(display_list)
                message = f"{total_issues} Non-string value(s) at index(es): {issue_data} (displaying only the first and last 10 items)"
            else:
                # Convert the list to a string while keeping the square brackets
                issue_data = str(issue_data)
                message = f"{total_issues} Non-string value(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #17 (Non-String Data Type - Consistency)"
            }

        return {"issue": False}

    @staticmethod
    def handle_alphanumeric_consistency(df, column):
        """
        Check if the alphanumeric values in the column have consistent total character length,
        excluding numeric values from this check.
        """
        def is_alphanumeric(val):
            val_str = str(val).strip()
            return val_str.isalnum() and not val_str.isdigit()

        def get_total_length(val):
            return len(str(val).strip())
        
        # Filter out negative and floating-point numbers
        filtered_df = df[df[column].apply(lambda x: not isinstance(x, (float, int)) or x >= 0 and float(x).is_integer())]

        # Sample the first 10 alphanumeric values
        id_samples = filtered_df[filtered_df[column].apply(is_alphanumeric)][column].head(10)

        length_set = {len(val) for val in id_samples if val.isalnum()}
        alphanumeric_consistent = len(length_set) == 1

        if not alphanumeric_consistent:
            return {"issue": False}  # Return no issue if first 10 samples are not consistent
        
        # Determine the consistent length from the samples
        length_set = {get_total_length(val) for val in id_samples}

        consistent_length = length_set.pop()

        # Check all alphanumeric values against the consistent length
        inconsistent_indices = []
        for idx, val in filtered_df[filtered_df[column].apply(is_alphanumeric)][column].items():
            val_str = str(val).strip()
            if get_total_length(val_str) != consistent_length:
                inconsistent_indices.append((idx, val))

        total_issues = len(inconsistent_indices)
        if total_issues > 0:
            if total_issues > 20:
                first_10 = inconsistent_indices[:10]
                last_10 = inconsistent_indices[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                issue_data = str(display_list)
                message = f"{total_issues} Inconsistent length in alphanumeric value(s) at index(es): {issue_data} (displaying only the first and last 10 items)"
            else:
                issue_data = str(inconsistent_indices)
                message = f"{total_issues} Inconsistent length in alphanumeric value(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #17 (Wrong Data Type - Consistency)"
            }
        return {"issue": False}


    @staticmethod
    def handle_uniqueness_violation(df, column):
        unique_violation_indices = df[df[column].duplicated()].index
        unique_violation_values = df.loc[unique_violation_indices, column].tolist()
        issue_data = list(zip(unique_violation_indices, unique_violation_values))
        total_issues = len(issue_data)

        if issue_data:
            if total_issues > 20:
                first_10 = issue_data[:10]
                last_10 = issue_data[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                message = f"{total_issues} Uniqueness violation(s) at index(es): {display_list} (displaying only the first and last 10 items)"
            else:
                message = f"{total_issues} Uniqueness violation(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #19 (Uniqueness Violation - Uniqueness)"
            }
        return {"issue": False}

    @staticmethod
    def handle_special_characters(df, column):
        """
        Check for special characters in the specified column.
        Allow periods in names for titles (e.g., Mr., Mrs.).
        """
        def has_special_chars(x):
            return any(char in x for char in ['!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '+', '=', '{', '}', '[', ']', '|', '\\', ':', ';', '"', "'", '<', '>', ',', '/', '?']) and '.' not in x

        # Get indices and values with special characters
        special_chars_indices = df[df[column].apply(lambda x: isinstance(x, str) and has_special_chars(x))].index
        special_chars_values = df.loc[special_chars_indices, column].tolist()
        issue_data = list(zip(special_chars_indices, special_chars_values))

        total_issues = len(issue_data)
        if issue_data:
            if total_issues > 20:
                # Keep only the first 10 and last 10 items
                first_10 = issue_data[:10]
                last_10 = issue_data[-10:]
                display_list = first_10 + [('...', '...')] + last_10
                issue_data = ", ".join([f"({idx}, '{value}')" for idx, value in display_list])
                message = f"{total_issues} Special character(s) at index(es): {issue_data} (displaying only the first and last 10 items)"
            else:
                issue_data = ", ".join([f"({idx}, '{value}')" for idx, value in issue_data])
                message = f"{total_issues} Special character(s) at index(es): {issue_data}"

            return {
                "issue": True,
                "error_message": message,
                "dq_issue": "DQI #21 (Use of Special Characters - Consistency)"
            }

        return {"issue": False}

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Last run on: 2025-04-04 19:33:46


07 Check Numerical ge zero

In [1978]:
def check_numerical_ge_zero(df: pd.DataFrame, column: str) -> dict:
    """
    Check if all the values in the specified numerical column are greater than or equal to zero,
    and also flag any non-numeric values and Blank/Empty/Null/Missing values.
    Additionally, report the range of numeric values.
    """
    # First, handle Blank/Empty/Null/Missing values
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)
    
    # Filtering df based on blank checks
    blank_indices = list(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)]

    # Only proceed with other checks if the value is not Blank/Empty/Null/Missing
    #df_filtered = df[~df[column].apply(lambda x: pd.isnull(x) or str(x).strip() == '' or str(x).lower() == 'null')]
    
    result_negative = DataQualityIssues.handle_negative_values(df_filtered, column)
    result_non_numeric = DataQualityIssues.handle_non_numeric_values(df_filtered, column)

    error_summary_parts = []

    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message']+ '\n')

    if result_negative['issue']:
        error_summary_parts.append(result_negative['dq_issue'] + ':\n ' + result_negative['error_message']+ '\n')

    if result_non_numeric['issue']:
        error_summary_parts.append(result_non_numeric['dq_issue'] + ':\n ' + result_non_numeric['error_message']+ '\n')

   # Convert the column to numeric, ignoring errors to leave non-numeric as NaN
    numeric_values = pd.to_numeric(df_filtered[column], errors='coerce')

    # Filter out non-numeric values to avoid them in min/max calculations
    numeric_values_filtered = numeric_values.dropna()

    # Calculate the range (min and max) of the numeric values
    min_value = numeric_values_filtered.min()
    max_value = numeric_values_filtered.max()
    
    # Count correct values
    correct_values_count = df_filtered[df_filtered[column].apply(lambda x: pd.to_numeric(x, errors='coerce') >= 0)].shape[0]

    if not error_summary_parts:
        range_summary = f"in the range ({min_value}:{max_value})" if not numeric_values_filtered.empty else "No valid numerical values found."
        return f"All {correct_values_count} values are numerical and greater or equal to 0 {range_summary}."

    error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts)
    range_summary = f"Range of values: ({min_value}:{max_value})." if not numeric_values_filtered.empty else "No valid numerical values found."


    return f"{error_summary}\n{range_summary}"

# Test function
df_test = pd.DataFrame({'hours-per-week': [1, 2, 3, 'a3', 5, '?', None, -1, '', " ", 'null', 5.67, np.nan, '    ',  "''", ';', '*', '-']})
#df_test = pd.DataFrame({'hours-per-week': [1, 2, 3]})
result = check_numerical_ge_zero(df_test, 'hours-per-week')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Error(s) found: 
DQI #1 (Missing Data - Completeness):
 11 Blank/Empty/Null/Missing value(s) at index(es): 
3 NULL values: [(6, 'NULL'), (10, 'NULL'), (12, 'NULL')]
8 Non-NULL values: [(5, '?'), (8, 'Empty'), (9, 'Empty'), (13, 'Empty'), (14, 'Empty'), (15, ';'), (16, '*'), (17, '-')]

DQI #15 (Domain Violation - Accuracy):
 1 Negative value(s) at index(es): [(7, -1)]

DQI #17 (Wrong Data Type - Consistency):
 1 Non-numeric value(s) at index(es): [(3, 'a3')]

Range of values: (-1.0:5.67).
Last run on: 2025-04-04 19:28:37


08 Check Numerical

In [1979]:
def check_numerical(df: pd.DataFrame, column: str) -> str:
    """
    Check if all the values in the specified numerical column are numerical.
    Additionally, report the range of numeric values.
    """
    # First, handle Blank/Empty/Null/Missing values
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)

    # Filtering df based on blank checks
    blank_indices = list(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)]
    
    result_non_numeric = DataQualityIssues.handle_non_numeric_values(df_filtered, column)

    error_summary_parts = []

    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message']+ '\n')

    if result_non_numeric['issue']:
        error_summary_parts.append(result_non_numeric['dq_issue'] + ':\n ' + result_non_numeric['error_message'])

    # Convert the column to numeric, ignoring errors to leave non-numeric as NaN
    numeric_values = pd.to_numeric(df_filtered[column], errors='coerce')

    # Filter out non-numeric values to avoid them in min/max calculations
    numeric_values_filtered = numeric_values.dropna()

    # Calculate the range (min and max) of the numeric values
    min_value = numeric_values_filtered.min()
    max_value = numeric_values_filtered.max()

    # Count correct values
    correct_values_count = numeric_values_filtered.notna().sum()

    if not error_summary_parts:
        range_summary = f"in the range ({min_value}:{max_value})" if not numeric_values_filtered.empty else "No valid numeric values found."
        return f"All {correct_values_count} values are numerical {range_summary}."

    error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts)
    range_summary = f"Range of values: ({min_value}:{max_value})." if not numeric_values_filtered.empty else "No valid numeric values found."

    return f"{error_summary}\n{range_summary}"

# Test function
#df_test = pd.DataFrame({'hours': [1, 2, 3, -2, 4.777]})
df_test = pd.DataFrame({'hours': [1, 2, 3, 'a3', 5, '?', None, -1, '', " ", 'null', 5.67, '  ']})
result = check_numerical(df_test, 'hours')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Error(s) found: 
DQI #1 (Missing Data - Completeness):
 6 Blank/Empty/Null/Missing value(s) at index(es): 
2 NULL values: [(6, 'NULL'), (10, 'NULL')]
4 Non-NULL values: [(5, '?'), (8, 'Empty'), (9, 'Empty'), (12, 'Empty')]

DQI #17 (Wrong Data Type - Consistency):
 1 Non-numeric value(s) at index(es): [(3, 'a3')]
Range of values: (-1.0:5.67).
Last run on: 2025-04-04 19:28:37


09 Check Numerical between

In [1980]:
def check_numerical_between(df: pd.DataFrame, column: str, min_value: float, max_value: float) -> str:
    """
    Check if all values in the specified column are numerical and fall within the given range.
    Additionally, report the actual range of numeric values.
    """
    # First, handle Blank/Empty/Null/Missing values
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)

    # Only proceed with other checks if the value is not Blank/Empty/Null/Missing
    # Filtering df based on blank checks
    blank_indices = list(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)]

    # Checks for non-numeric and out-of-range values
    result_non_numeric = DataQualityIssues.handle_non_numeric_values(df_filtered, column)
    result_outside_range = DataQualityIssues.handle_values_outside_range(df_filtered, column, min_value, max_value)

    error_summary_parts = []

    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message'] + '\n')

    if result_outside_range['issue']:
        error_summary_parts.append(result_outside_range['dq_issue'] + ':\n ' + result_outside_range['error_message'] + '\n')

    if result_non_numeric['issue']:
        error_summary_parts.append(result_non_numeric['dq_issue'] + ':\n ' + result_non_numeric['error_message'] + '\n')

    # Convert the column to numeric, ignoring errors to leave non-numeric as NaN
    numeric_values = pd.to_numeric(df_filtered[column], errors='coerce')

    # Filter out non-numeric values to avoid them in min/max calculations
    numeric_values_filtered = numeric_values.dropna()

    # Calculate the range (min and max) of the numeric values
    actual_min_value = numeric_values_filtered.min()
    actual_max_value = numeric_values_filtered.max()

    range_summary = f"Actual range of values: ({actual_min_value} : {actual_max_value})" if not numeric_values_filtered.empty else "No valid numeric values found."

    if not error_summary_parts:
        return f"All {len(numeric_values_filtered)} values are numerical and valid in the range [{min_value}, {max_value}].\n{range_summary}"

    error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts)

    return f"{error_summary}\n{range_summary}"

# Test function
df_test = pd.DataFrame({'age': [1, 2, 131, 'a3', 5, '?', None, -1, '', "", 'NULL', '  ', 140.0, 129.0]})
result = check_numerical_between(df_test, 'age', 0, 130)
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Error(s) found: 
DQI #1 (Missing Data - Completeness):
 6 Blank/Empty/Null/Missing value(s) at index(es): 
2 NULL values: [(6, 'NULL'), (10, 'NULL')]
4 Non-NULL values: [(5, '?'), (8, 'Empty'), (9, 'Empty'), (11, 'Empty')]

DQI #15 (Domain Violation - Accuracy):
 3 Value(s) outside range [0, 130] at index(es): [(2, 131), (7, -1), (12, 140.0)]

DQI #17 (Wrong Data Type - Consistency):
 1 Non-numeric value(s) at index(es): [(3, 'a3')]

Actual range of values: (-1.0 : 140.0)
Last run on: 2025-04-04 19:28:37


10 Check ID column

In [1981]:
def check_id_attributes(df: pd.DataFrame, column: str, is_foreign_key: bool = False) -> str:
    print(f"Checking {'composite ' if '|' in column else ''}{'foreign' if is_foreign_key else 'primary'} key: {column}")
    """
    Check if the values in the specified column are suitable for use as a Primary Key (PK) or Foreign Key (FK).
    This function checks for blank, non-numeric, negative values, and uniqueness violations.
    For PKs, it also checks for duplicates.
    Additionally, report the range of numeric or alphanumeric ID values.
    """
    print(f"Checking column '{column}' (Foreign Key: {is_foreign_key})")
    
    # Calculate the total number of values in the column
    total_values_count = df[column].size

    # Handle Blank/Empty/Null/Missing values
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)

    # Filtering df based on blank checks
    blank_indices = list(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)]
 
    # Handle negative values in the filtered DataFrame
    result_negative = DataQualityIssues.handle_negative_values(df_filtered, column)

    # Check for floating-point numbers in the filtered DataFrame
    result_floating_point = DataQualityIssues.handle_floating_point_values(df_filtered, column)

    # Check if values are alphanumeric and consistent in length in the filtered DataFrame
    result_alphanumeric_consistency = DataQualityIssues.handle_alphanumeric_consistency(df_filtered, column)
  
    # Handle duplicates and uniqueness violations only if it's not a foreign key
    if not is_foreign_key:
        result_duplicates = DataQualityIssues.handle_duplicates(df_filtered, column)
        result_uniqueness = DataQualityIssues.handle_uniqueness_violation(df_filtered, column)
        print(f"Column {column} is not FK")
    else:
        result_duplicates = {"issue": False}
        result_uniqueness = {"issue": False}
        print(f"Skipping duplicate and uniqueness checks for FK column '{column}'")

    error_summary_parts = []

    # Append results to error summary
    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message'] + '\n')

    if result_duplicates['issue']:
        error_summary_parts.append(result_duplicates['dq_issue'] + ':\n ' + result_duplicates['error_message'] + '\n')

    if result_negative['issue']:
        error_summary_parts.append(result_negative['dq_issue'] + ':\n ' + result_negative['error_message'] + '\n')

    if result_floating_point['issue']:
        error_summary_parts.append(result_floating_point['dq_issue'] + ':\n ' + result_floating_point['error_message'] + '\n')

    if result_alphanumeric_consistency['issue']:
        error_summary_parts.append(result_alphanumeric_consistency['dq_issue'] + ':\n ' + result_alphanumeric_consistency['error_message'] + '\n')

    if not is_foreign_key and result_uniqueness['issue']:
        error_summary_parts.append(result_uniqueness['dq_issue'] + ':\n ' + result_uniqueness['error_message'] + '\n')

    # Calculate the range (min and max) of the values
    filtered_values = df_filtered[column].dropna()
    
    if filtered_values.empty:
        range_summary = "No valid values found."
    elif filtered_values.dtype.kind in 'biufc':  # Check if all values are numeric
        min_value = filtered_values.min()
        max_value = filtered_values.max()
        range_summary = f"Numeric range of values: ({min_value} : {max_value})"
    else:
        alphanumeric_values = filtered_values.astype(str)
        alphanumeric_min_value = alphanumeric_values.min()
        alphanumeric_max_value = alphanumeric_values.max()
        range_summary = f"Alphanumeric range of values: ({alphanumeric_min_value} : {alphanumeric_max_value})"

    if not error_summary_parts:
        if is_foreign_key:
            return f"All {total_values_count} ID values are valid for use as a Foreign Key, and duplicate values are allowed.\n{range_summary}"
        else:
            return f"All {total_values_count} ID values are unique and valid, and thus suitable for use as a Primary Key.\n{range_summary}"

    error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts)
    if is_foreign_key:
        return f"{error_summary}\nNote: This is a Foreign Key, so duplicate values are allowed.\n{range_summary}"
    else:
        return f"{error_summary}\n{range_summary}"

# Example usage
'''df_test = pd.DataFrame({'id_column': [1, 2, 3, 5, None, -1, 1, '', " ", 'null', 5.67,'&']})'''

'''df_test = pd.DataFrame({'id_column': [1, 2, 3, 5, 'NULL']})'''

df_test = pd.DataFrame({
    'id_column': ['AB123CD456', 'AB123CD457', 'AB123CD458', 'AB123CD459', 'AB123CD460', 
                  'AB123CD461', 'AB123CD462', 'AB123CD463', 'AB123CD464', 'AB123CD465', 
                  'AB123CD466', 'AB123CD467', 'AB123CD468', 'AB123CD469', 'AB123CD470', 
                  'AB123CD471', 'AB123CD472', 'AB123CD473', 'AB123CD474', 'AB123CD475', 
                  'Duplicate', 'Duplicate', 'WrongLength1', 'AB123CD456', 'AB123CD479', 
                  'AB123CD480', 'AB123CD481', 'AB123CD4-2', 'AB123CD*83', 'AB123C_484', 
                  'AB123CD48', 'AB123CD456', '?','--', 'Aaa', None, -1, 1, '', " ", 'NULL', '""', 'None']})

'''df_test = pd.DataFrame({
    'id_column': ['AB123CD456', 'AB123CD457', 'AB123CD458', 'AB123CD459', 'AB123CD460', 
                  'AB123CD461', 'AB123CD462', 'AB123CD463', 'AB123CD464', 'AB123CD465', 
                  'AB123CD466', 'AB123CD467', 'AB123CD468', 'AB123CD469', 'AB123CD470', 
                  'AB123CD471', 'AB123CD472', 'AB123CD473']})'''

result = check_id_attributes(df_test, 'id_column')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Checking primary key: id_column
Checking column 'id_column' (Foreign Key: False)
Column id_column is not FK
Error(s) found: 
DQI #1 (Missing Data - Completeness):
 7 Blank/Empty/Null/Missing value(s) at index(es): 
2 NULL values: [(35, 'NULL'), (40, 'NULL')]
5 Non-NULL values: [(32, '?'), (38, 'Empty'), (39, 'Empty'), (41, 'Empty'), (42, 'None')]

DQI #9 (Duplicates - Uniqueness):
 5 Duplicate value(s) at index(es): [[0, 'AB123CD456'], [20, 'Duplicate'], [21, 'Duplicate'], [23, 'AB123CD456'], [31, 'AB123CD456']]

DQI #15 (Domain Violation - Accuracy):
 1 Negative value(s) at index(es): [(36, -1)]

DQI #17 (Wrong Data Type - Consistency):
 5 Inconsistent length in alphanumeric value(s) at index(es): [(20, 'Duplicate'), (21, 'Duplicate'), (22, 'WrongLength1'), (30, 'AB123CD48'), (34, 'Aaa')]

DQI #19 (Uniqueness Violation - Uniqueness):
 3 Uniqueness violation(s) at index(es): [(21, 'Duplicate'), (23, 'AB123CD456'), (31, 'AB123CD456')]

Alphanumeric range of values: (-- : WrongLength1)
L

11 Check String

In [1982]:
import pandas as pd
import numpy as np

def check_string(df: pd.DataFrame, column: str) -> str:
    """
    Check if all the values in the specified column are non-empty, non-null strings.
    Additionally, report the range of string values based on lexicographical order.
    """
    # Calculate the total number of values in the column
    total_values_count = df[column].size

    if column not in df.columns:
        return f"Column {column} does not exist in the DataFrame"
    
    try:
        # First, handle Blank/Empty/Null/Missing values
        result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)

        # Filtering df based on blank checks
        blank_indices = list(result_blank["blank_indices"])
        df_filtered = df[~df.index.isin(blank_indices)]
        result_non_string = DataQualityIssues.handle_non_string_values(df_filtered, column)

        error_summary_parts = []
        
        if result_blank['issue']:
            error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message'] + '\n')
        
        if result_non_string['issue']:
            error_summary_parts.append(result_non_string['dq_issue'] + ':\n ' + result_non_string['error_message'] + '\n')

        # Calculate the lexicographical range of the string values
        string_values = df_filtered[column].dropna().astype(str)
        lex_min_value = string_values.min()
        lex_max_value = string_values.max()

        range_summary = f"String range (lexicographical): ({lex_min_value} : {lex_max_value})" if not string_values.empty else "No valid string values found."

        value_counts = df[column].apply(DataQualityIssues.represent_value).value_counts().to_dict()
        frequency_table = pd.DataFrame(value_counts.items(), columns=[column, 'Frequency'])
        frequency_table = frequency_table.sort_values(by=['Frequency', column], ascending=[False, True]).reset_index(drop=True)

        if len(frequency_table) > 20:
            top_rows = frequency_table.head(10)
            bottom_rows = frequency_table.tail(10)
            ellipsis_row = pd.DataFrame([("...", "...")], columns=[column, 'Frequency'])
            display_table = pd.concat([top_rows, ellipsis_row, bottom_rows], ignore_index=True)
        else:
            display_table = frequency_table

        frequency_distribution = f"\nFrequency Distribution:\n{display_table.to_string(index=False)}"

        if error_summary_parts:
            return "Error(s) found: \n" + "\n".join(error_summary_parts) + range_summary + frequency_distribution
        else:
            return f"All {total_values_count} string values are valid. {range_summary}\n{frequency_distribution}"
    except Exception as e:
        return str(e)


# Example usage
df_bad_string_data = pd.DataFrame({
    'text_column': [
        'Hello',          # Valid string
        None,             # Null value
        np.nan,           # NaN value (also treated as null)
        '',               # Empty string
        ' ', "   ",       # String with only spaces
        123,              # Integer (non-string data type)
        5.67,             # Float (non-string data type)
        True,             # Boolean (non-string data type)
        {'key': 'value'}, # Dictionary (non-string data type)
        '  Goodbye ',     # Whitespace-padded string
        '*'               # Missing value
    ]
}, columns=['text_column'])

'''df_bad_string_data = pd.DataFrame({
    'text_column': [
        'Hello',          # Valid string
        '  Goodbye  '     # Whitespace-padded string
    ]
}, columns=['text_column'])'''


result = check_string(df_bad_string_data, 'text_column')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Error(s) found: 
DQI #1 (Missing Data - Completeness):
 6 Blank/Empty/Null/Missing value(s) at index(es): 
2 NULL values: [(1, 'NULL'), (2, 'NULL')]
4 Non-NULL values: [(3, 'Empty'), (4, 'Empty'), (5, 'Empty'), (11, '*')]

DQI #17 (Non-String Data Type - Consistency):
 4 Non-string value(s) at index(es): [(6, 123), (7, 5.67), (8, True), (9, {'key': 'value'})]
String range (lexicographical): (  Goodbye  : {'key': 'value'})
Frequency Distribution:
     text_column  Frequency
           Empty          3
            NULL          2
        Goodbye           1
               *          1
             123          1
            5.67          1
           Hello          1
            True          1
{'key': 'value'}          1
Last run on: 2025-04-04 19:28:37


12 Check Categorical

In [2006]:
import pandas as pd
import numpy as np

def check_categorical(df, column, threshold=100):
    # Calculate the total number of values in the column
    total_values_count = df[column].size
    error_summary_parts = []
        
    # Handle Blank/Empty/Null/Missing values
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)
    blank_indices = list(result_blank["blank_indices"])

    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message'] + '\n')

    df_filtered_for_dqi4 = df[~df.index.isin(blank_indices)]

    # Handle predefined unacceptable values
    result_unacceptable_values = DataQualityIssues.handle_predefined_unacceptable_values(df_filtered_for_dqi4, column)
    unacceptable_indices = list(result_unacceptable_values["unacceptable_indices"])

    if result_unacceptable_values['issue']:
        error_summary_parts.append(result_unacceptable_values['dq_issue'] + ':\n ' + result_unacceptable_values['error_message'] + '\n')

    # Combine all excluded indices
    all_excluded_indices = list(set(blank_indices + unacceptable_indices))

    # Check for excessive distinct values after filtering out unacceptable values
    df_filtered_for_dqi10 = df[~df.index.isin(all_excluded_indices)]
    result_excessive_values = DataQualityIssues.handle_excessive_distinct_values(df_filtered_for_dqi10, column, threshold)

    if result_excessive_values['issue']:
        error_summary_parts.append(result_excessive_values['dq_issue'] + ':\n ' + result_excessive_values['error_message'] + '\n')

    # Display frequency distribution with a maximum of 20 entries (10 from the beginning and 10 from the end)
    frequency_table = df[column].apply(DataQualityIssues.represent_value).value_counts().reset_index()
    frequency_table.columns = ['Category', 'Frequency']
    
    # Sort first by Frequency (descending), then by Category (alphabetically)
    frequency_table = frequency_table.sort_values(by=['Frequency', 'Category'], ascending=[False, True]).reset_index(drop=True)

    # Select the first 10 and last 10 rows
    top_rows = frequency_table.head(10)
    bottom_rows = frequency_table.tail(10)
    if len(frequency_table) > 20:
        ellipsis_row = pd.DataFrame([("...", "...")], columns=['Category', 'Frequency'])
        display_table = pd.concat([top_rows, ellipsis_row, bottom_rows], ignore_index=True)
    else:
        display_table = frequency_table

    frequency_distribution_str = f"\nCategorical format with {df[column].nunique(dropna=False)} unique value(s):\n{display_table.to_string(index=False)}"

    # If no errors are found, return a message stating that all values are correct along with frequency distribution
    if not error_summary_parts:
        return f"All {total_values_count} values are correctly categorical.\n{frequency_distribution_str}"

    # Combine frequency distribution with error summary (if any)
    error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts) if error_summary_parts else ""
    return error_summary + frequency_distribution_str

# Example usage
df_test = pd.DataFrame({'categorical_column': ['cat', 'dog', 'bird', '  ', 'Null', 100, 200, 'cat', 'dog', 'fish', 'fish', 'bird', '?', None, '', "", "''",'NA','N/A', " ", " ? ",'nonexistent']})
result = check_categorical(df_test, 'categorical_column')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Error(s) found: 
DQI #1 (Missing Data - Completeness):
 10 Blank/Empty/Null/Missing value(s) at index(es): 
2 NULL values: [(4, 'NULL'), (13, 'NULL')]
8 Non-NULL values: [(3, 'Empty'), (12, '?'), (14, 'Empty'), (15, 'Empty'), (16, 'Empty'), (18, 'N/A'), (19, 'Empty'), (20, '?')]

DQI #4 (Ambiguous Data - Accuracy, Consistency):
 1 Unacceptable value(s) at index(es): [(21, 'nonexistent')]

Categorical format with 17 unique value(s):
   Category  Frequency
      Empty          5
          ?          2
       NULL          2
       bird          2
        cat          2
        dog          2
       fish          2
        100          1
        200          1
        N/A          1
         NA          1
nonexistent          1
Last run on: 2025-04-04 19:34:23


13 Check Month

In [1984]:
import pandas as pd
from datetime import datetime

def check_month(df, column):
    error_summary_parts = []
    month_counts = {}

    # Calculate the total number of values in the column
    total_values_count = df[column].size
    
    # First, handle Blank/Empty/Null/Missing values
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)

    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message'] + '\n')

    # Handling invalid months
    # Filtering df based on blank checks
    blank_indices = list(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)]

    result_invalid_months = DataQualityIssues.handle_invalid_months(df_filtered, column)

    if result_invalid_months['issue']:
        error_summary_parts.append(result_invalid_months['dq_issue'] + ':\n ' + result_invalid_months['error_message'] + '\n')

    # Create frequency distribution for valid months
    for val in df_filtered[column].dropna():
        normalized_month = DataQualityIssues.normalize_month(str(val))
        if normalized_month:
            month_counts[normalized_month] = month_counts.get(normalized_month, 0) + 1

    # Compile frequency distribution
    frequency_table = pd.DataFrame([
        (month, month_to_number(month), count) 
        for month, count in month_counts.items()
    ], columns=['Month', 'MonthNum', 'Frequency'])
    frequency_table = frequency_table.sort_values(by=['Frequency', 'MonthNum'], ascending=[False, True]).reset_index(drop=True)
    frequency_distribution = f"\nFrequency Distribution:\n{frequency_table[['Month', 'Frequency']].to_string(index=False)}"

    # If no errors are found, return a message stating that all values are valid along with frequency distribution
    if not error_summary_parts:
        return f"All {total_values_count} month values are valid.\n{frequency_distribution}"

    # Compile the final result message
    error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts) if error_summary_parts else ""
    return error_summary + frequency_distribution

def month_to_number(month_name):
    """
    Convert a month name to its corresponding numeric value.

    Parameters:
    - month_name (str): The full name of the month.

    Returns:
    - int: Numeric representation of the month.
    """
    try:
        return datetime.strptime(month_name, '%B').month
    except ValueError:
        return None

# Example usage
#df_test = pd.DataFrame({'month_column': [1, '3', 12, 'Jan', 'January', 'feb', 'NOV', 'Dec','FEBRUARY']})
df_test = pd.DataFrame({'month_column': [1, '3', 12, '0', '13', 'Jan', 'January', 'feb', 'NOV', 'Dec', 'not a month', None, 'mn', 'FEBRUARY',"%", 1.0]})
result = check_month(df_test, 'month_column')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Error(s) found: 
DQI #1 (Missing Data - Completeness):
 2 Blank/Empty/Null/Missing value(s) at index(es): 
1 NULL values: [(11, 'NULL')]
1 Non-NULL values: [(14, '%')]

DQI #15 (Domain Violation - Accuracy):
 4 Invalid month value(s) at index(es): [(3, '0'), (4, '13'), (10, 'not a month'), (12, 'mn')]

Frequency Distribution:
   Month  Frequency
 January          4
February          2
December          2
   March          1
November          1
Last run on: 2025-04-04 19:28:37


14 Check Weekday

In [1985]:
from datetime import datetime

def normalize_weekday(weekday_str):
    """
    Normalize the given weekday string to its full weekday name if valid.

    Parameters:
    - weekday_str (str): The weekday string to normalize.

    Returns:
    - str/None: Normalized weekday name if valid, None otherwise.
    """
    weekdays = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    weekday_abbr = ["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]
    weekday_abbr_short = ["su", "mo", "tu", "we", "th", "fr", "sa"]

    weekday_str_norm = weekday_str.lower()

    # Normalize full names and abbreviations to full weekday name
    if weekday_str_norm.title() in weekdays:
        return weekday_str_norm.title()
    elif weekday_str_norm.title() in weekday_abbr:
        return weekdays[weekday_abbr.index(weekday_str_norm.title())]
    elif weekday_str_norm in weekday_abbr_short:
        return weekdays[weekday_abbr_short.index(weekday_str_norm)]
    elif weekday_str_norm.isdigit():
        # Convert numeric representation to weekday name
        weekday_num = int(weekday_str_norm)
        if 1 <= weekday_num <= 7:
            return weekdays[weekday_num - 1]  # Adjusting for 1-indexed weekdays
    return None

def check_weekday(df, column):
    error_summary_parts = []
    weekday_counts = {}

    # Calculate the total number of values in the column
    total_values_count = df[column].size

    # First, handle Blank/Empty/Null/Missing values
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)

    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message'] + '\n')

    # Handling invalid weekdays
    # Filtering df based on blank checks
    blank_indices = list(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)]

    result_invalid_weekdays = DataQualityIssues.handle_invalid_weekdays(df_filtered, column)

    if result_invalid_weekdays['issue']:
        error_summary_parts.append(result_invalid_weekdays['dq_issue'] + ':\n ' + result_invalid_weekdays['error_message'] + '\n')

    # Create frequency distribution for valid weekdays
    for val in df_filtered[column].dropna():
        normalized_weekday = normalize_weekday(str(val))
        if normalized_weekday:
            weekday_counts[normalized_weekday] = weekday_counts.get(normalized_weekday, 0) + 1

    # Compile frequency distribution
    frequency_table = pd.DataFrame(weekday_counts.items(), columns=['Weekday', 'Frequency'])
    frequency_table = frequency_table.sort_values(by='Frequency', ascending=False).reset_index(drop=True)
    frequency_distribution = f"\nFrequency Distribution:\n{frequency_table.to_string(index=False)}"

    # If no errors are found, return a message stating that all values are valid along with frequency distribution
    if not error_summary_parts:
        return f"All {total_values_count} weekday values are valid.\n{frequency_distribution}"

    # Compile the final result message
    error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts) if error_summary_parts else ""
    return error_summary + frequency_distribution

# Example usage
df_test = pd.DataFrame({'weekday_column': ['Monday', 'Tue', 2, 'wed', 'Sunday', 'sun', 'not a weekday', None, 'Mo', 'WED', 7, -1, 'Mn',"$"]})
#df_test = pd.DataFrame({'weekday_column': ['Monday', 'Tue', 2, 'wed', 'Sunday', 'sun', 'Mo', 'WED', 7, 0]})

result = check_weekday(df_test, 'weekday_column')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Error(s) found: 
DQI #1 (Missing Data - Completeness):
 2 Blank/Empty/Null/Missing value(s) at index(es): 
1 NULL values: [(7, 'NULL')]
1 Non-NULL values: [(13, '$')]

DQI #15 (Domain Violation - Accuracy):
 3 Invalid weekday value(s) at index(es): [(6, 'not a weekday'), (11, -1), (12, 'Mn')]

Frequency Distribution:
  Weekday  Frequency
   Monday          3
Wednesday          2
   Sunday          2
  Tuesday          1
 Saturday          1
Last run on: 2025-04-04 19:28:37


15 Check Date 

In [1986]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

min_valid_year = 1800
max_valid_year = 2100

def extract_indices_from_error_message(error_message):
    # Regular expression to find tuples in the format (index, 'date')
    return [int(m.group(1)) for m in re.finditer(r"\((\d+), '.*?'\)", error_message)]

def check_date(df, column, sample_size=10):
    print(f"Checking dates in column: {column}")
    total_values_count = df[column].size
    error_summary_parts = []
    problematic_indices = set()
    valid_standardized_dates = []
    outdated_entries = []  
    earliest_date_str = "N/A"
    latest_date_str = "N/A"

    # Handle Blank/Empty/Null/Missing values
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)
    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message'] + '\n') 
        problematic_indices.update(result_blank["blank_indices"])

    df_filtered = df[~df.index.isin(problematic_indices)]

    # Deduce regional format
    date_samples = df_filtered[column].astype(str).str.strip()
    date_samples = date_samples[:sample_size]
    deduced_date_format, _ = DataQualityIssues.deduce_regional_format(date_samples)
    print(f"Deduced date format: {deduced_date_format}")

    deduced_strftime_format = DataQualityIssues.convert_to_strftime_format(deduced_date_format)
    #print(f"deduced_strftime_format: {deduced_strftime_format}")

    # Handling outdated temporal data
    result_outdated = DataQualityIssues.handle_outdated_temporal_data(df_filtered, column, min_valid_year, max_valid_year)
    if result_outdated['issue']:
        error_summary_parts.append(result_outdated['dq_issue'] + ':\n ' + result_outdated['error_message'] + '\n')
        outdated_indices = extract_indices_from_error_message(result_outdated['error_message'])
        problematic_indices.update(outdated_indices)
        # Capture outdated entries to be included in valid date range
        outdated_entries = result_outdated.get('outdated_entries', [])

    df_filtered = df[~df.index.isin(problematic_indices)]
    #print(f"Remaining rows after outdated check: {len(df_filtered)}")
  
    # Handling invalid dates
    result_invalid_dates = DataQualityIssues.handle_invalid_dates(df_filtered, column)
    if result_invalid_dates['issue']:
        error_summary_parts.append(result_invalid_dates['dq_issue'] + ':\n ' + result_invalid_dates['error_message'] + '\n')
        invalid_indices = extract_indices_from_error_message(result_invalid_dates['error_message'])
        problematic_indices.update(invalid_indices)

    df_filtered = df[~df.index.isin(problematic_indices)]
    #print(f"Remaining rows after invalid dates check: {len(df_filtered)}")

    # Handling format issues for valid dates
    result_format_issues = DataQualityIssues.handle_dates_format(df_filtered, column, deduced_date_format)
    if result_format_issues['issue']:
        error_summary_parts.append(result_format_issues['dq_issue'] + ':\n ' + result_format_issues['error_message'] + '\n')

    df_filtered = df[~df.index.isin(problematic_indices)]
    #print(f"Remaining rows after format check: {len(df_filtered)}")
 
    # Iterate through the dataset to extract standardized dates
    for idx, val in df[column].items():
        standardized_date = DataQualityIssues.standardize_date(str(val), 'DDMMYYYY')
        
        if standardized_date and len(standardized_date) == 8 and standardized_date.isdigit():
            try:
                # Ensure year is parsed first for DDMMYYYY format and validate within a larger year range
                if standardized_date[:4].isdigit() and 0000 <= int(standardized_date[:4]) <= 3000:
                    year, month, day = int(standardized_date[:4]), int(standardized_date[4:6]), int(standardized_date[6:])
                else:
                    day, month, year = int(standardized_date[:2]), int(standardized_date[2:4]), int(standardized_date[4:])
                
                date_obj = datetime(year, month, day)
                valid_standardized_dates.append(date_obj)
                #print(f"Index: {idx}, Original Value: {val}, Standardized Date: {standardized_date}, Date Object: {date_obj}")

            except ValueError as e:
                #print(f"Failed to convert standardized date: {standardized_date} at Index: {idx} - Error: {e}")
                continue

    # Now, also include outdated dates in the valid date list
    if outdated_entries:
        for idx, val in outdated_entries:
            standardized_date = DataQualityIssues.standardize_date(str(val), 'DDMMYYYY')
            if standardized_date and len(standardized_date) == 8 and standardized_date.isdigit():
                try:
                    if 0000 <= int(standardized_date[:4]) <= 3000:
                        year, month, day = int(standardized_date[:4]), int(standardized_date[4:6]), int(standardized_date[6:])
                    else:
                        day, month, year = int(standardized_date[:2]), int(standardized_date[2:4]), int(standardized_date[4:])
                    date_obj = datetime(year, month, day)
                    valid_standardized_dates.append(date_obj)
                    print(f"Outdated Date: Index {idx}, Date Object: {date_obj}")
                except ValueError:
                    continue

    # Handling the earliest and latest dates
    try:
        if valid_standardized_dates:
            # Ensure all dates are considered, including outdated dates
            all_dates = valid_standardized_dates + [date_obj for _, date_obj in outdated_entries]

            # Convert all standardized dates to `datetime` objects, if they're not already
            all_dates = [
                datetime.strptime(str(date), "%Y-%m-%d") if isinstance(date, str) else date
                for date in all_dates
            ]
            
            # Now calculate the minimum and maximum dates
            earliest_date = min(all_dates)
            latest_date = max(all_dates)

            # Display the earliest and latest dates
            earliest_date_str = earliest_date.strftime('%Y-%m-%d')
            latest_date_str = latest_date.strftime('%Y-%m-%d')
            date_range_summary = f"Date range: {earliest_date_str} to {latest_date_str}"
        else:
            date_range_summary = "No valid date values found."

    except Exception as e:
        #print(f"Error converting dates: {e}")
        date_range_summary = "Error processing date range."

    # Create the date range summary
    if earliest_date_str != "N/A" and latest_date_str != "N/A":
        date_range_summary = f"Date range: {earliest_date_str} to {latest_date_str}"
    else:
        date_range_summary = "No valid date range could be determined."

    if error_summary_parts:
        error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts) + '\n' + date_range_summary
    else:
        if earliest_date_str != "N/A" and latest_date_str != "N/A":
            error_summary = f"All {total_values_count} date values are valid in the {deduced_date_format} format in the range {earliest_date_str} to {latest_date_str}."
        else:
            error_summary = f"All {total_values_count} date values are in the correct format, but no valid date range could be determined."

    return error_summary

# Example usage
dates = [
    '3/4/2021', '14/5/2021', '01/01/2021', '12/31/2021', '2021/12/25', # Valid dates
    '14 January 2021', '1 Feb 2021', '28 Mar 2021', '4 Apr 2021', '15 Oct 2021', '23 Nov 2021', # Textual months
    '2021/04/7', '2021/08/15', '2021/11/03', '12/30/2021', '07/04/2021', '11/11/2021', # Various formats
    '32/01/2021', '29/02/2021', '31/11/2021', '00/01/2021', '01/00/2021', '2021/13/01', # Invalid dates
    'not a date', '', ' ', '2021-02-30', np.nan, None, 'Null', "#", # Non-date and empty strings
    '1 Feb 2021','1 February 2021',
    '3/4/2121', '14/5/2222', '01/01/1500', '31/12/2321', '2121/12/25', '0301-01-01', '1 feb 1719'  # Dates before 1800 or greater than 2100
]

'''dates = [
'2010-12-01','2010-12-02','2010-12-03','2010-12-14','2010-12-05',
'2010-12-06','2010-12-07','2010-12-08','2010-12-09','2010-12-10',
'2010-12-11','2010-12-12','2010-12-13','2010-12-14']'''

'''dates = [
    '01/01/2021', '02/01/2021', '03/01/2021', 
    '04/01/2021', '05/01/2021', '06/01/2021',
    '07/01/2021', '08/01/2021', '09/01/2021', 
    '10/01/2021', '11/01/2021', '13/01/2021']'''
   
df_test_dates = pd.DataFrame({'date_column': dates})
result = check_date(df_test_dates, 'date_column')
print(result)

print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Checking dates in column: date_column

Deduced format: DDMMYYYY
Processed date sample: 15 10 2021, detected format counts: {'DDMMYYYY': 9, 'MMDDYYYY': 0, 'YYYYMMDD': 1}

Deduced date format: DDMMYYYY
Outdated Date: Index 33, Date Object: 2121-04-03 00:00:00
Outdated Date: Index 34, Date Object: 2222-05-14 00:00:00
Outdated Date: Index 35, Date Object: 1500-01-01 00:00:00
Outdated Date: Index 36, Date Object: 2321-12-31 00:00:00
Outdated Date: Index 37, Date Object: 2121-12-25 00:00:00
Outdated Date: Index 38, Date Object: 0301-01-01 00:00:00
Outdated Date: Index 39, Date Object: 1719-02-01 00:00:00
Error(s) found: 
DQI #1 (Missing Data - Completeness):
 6 Blank/Empty/Null/Missing value(s) at index(es): 
3 NULL values: [(27, 'NULL'), (28, 'NULL'), (29, 'NULL')]
3 Non-NULL values: [(24, 'Empty'), (25, 'Empty'), (30, '#')]

DQI #6 (Outdated Temporal Data - Timeliness):
 7 Date value(s) not in [1800-2100] period at index(es): [(33, '2121-04-03'), (34, '2222-05-14'), (35, '1500-01-01'), (36

16 Check DateTime

In [1987]:
def extract_indices_from_error_message(error_message):
    # Regular expression to find tuples in the format (index, 'value')
    return [int(m.group(1)) for m in re.finditer(r"\((\d+), '.*?'\)", error_message)]

def update_format_counts(standardized_date, format_counts):
    if len(standardized_date) == 8:
        day, month, year = int(standardized_date[:2]), int(standardized_date[2:4]), int(standardized_date[4:])
        if 1 <= day <= 31 and 1 <= month <= 12:
            format_counts['DDMMYYYY'] += 1
        if 1 <= month <= 12 and 1 <= day <= 31:
            format_counts['MMDDYYYY'] += 1
        year, month, day = int(standardized_date[:4]), int(standardized_date[4:6]), int(standardized_date[6:])
        if min_valid_year <= year <= max_valid_year and 1 <= month <= 12 and 1 <= day <= 31:
            format_counts['YYYYMMDD'] += 1
    return format_counts

def check_datetime(df, column, sample_size=10):
    # Calculate the total number of values in the column
    total_values_count = df[column].size
    error_summary_parts = []

    # Handle Blank/Empty/Null/Missing values
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)
    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message'] + '\n')
    
    # Filtering df based on blank checks
    blank_indices = list(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)]

    # Deduce regional format
    datetime_samples = df_filtered[column].astype(str).str.strip()
    date_samples = [sample.split(' ')[0] for sample in datetime_samples][:sample_size]
    deduced_date_format, _ = DataQualityIssues.deduce_regional_format(date_samples)
    deduced_strftime_format = DataQualityIssues.convert_to_strftime_format(deduced_date_format) + " %H:%M"
    
    # Handling outdated temporal datetimes
    result_outdated = DataQualityIssues.handle_outdated_temporal_data_datetime(df_filtered, column, min_valid_year, max_valid_year)
    if result_outdated['issue']:
        error_summary_parts.append(result_outdated['dq_issue'] + ':\n ' + result_outdated['error_message'] + '\n')
        outdated_indices = extract_indices_from_error_message(result_outdated['error_message'])
        df_filtered = df_filtered[~df_filtered.index.isin(outdated_indices)]

    # Handling format issues for valid dates
    result_format_issues = DataQualityIssues.handle_datetimes_format(df_filtered, column, deduced_date_format)
    if result_format_issues['issue']:
        error_summary_parts.append(result_format_issues['dq_issue'] + ':\n ' + result_format_issues['error_message'] + '\n')
        invalid_indices = extract_indices_from_error_message(result_format_issues['error_message'])
        df_filtered = df_filtered[~df_filtered.index.isin(invalid_indices )]
  
    # Handle invalid datetime formats
    result_invalid_formats = DataQualityIssues.handle_invalid_datetimes(df_filtered, column, deduced_date_format)
    if result_invalid_formats['issue']:
        error_summary_parts.append(result_invalid_formats['dq_issue'] + ':\n ' + result_invalid_formats['error_message'] + '\n')

    # Find the earliest and latest datetime values
    valid_datetimes = pd.to_datetime(df_filtered[column], errors='coerce', format=deduced_strftime_format).dropna()

    earliest_datetime = valid_datetimes.min()
    latest_datetime = valid_datetimes.max()

    # Convert earliest and latest datetime to the deduced date format
    datetime_range_summary = ""
    if not valid_datetimes.empty:
        earliest_datetime_str = earliest_datetime.strftime(deduced_strftime_format)
        latest_datetime_str = latest_datetime.strftime(deduced_strftime_format)
        datetime_range_summary = f"Date range: {earliest_datetime_str} to {latest_datetime_str}\n"
    else:
        datetime_range_summary = "No valid datetime values found.\n"

   # Compile final result message
    error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts) if error_summary_parts else f"All {total_values_count} datetime values are valid in the {deduced_date_format} format in the range {earliest_datetime_str} to {latest_datetime_str}."
    return error_summary

# Example usage
timestamps = [
    '01/01/2021 13:00', '02/01/2021 14:30', '03/01/2021 15:45', 
    '04/01/2021 16:00', '05/01/2021 17:15', '06/01/2021 18:30',
    '07/01/2021 19:45', '08/01/2021 20:00', '09/01/2021 21:15', 
    '10/01/2021 22:30', '2021-01-11 23:45', '11-01-2021 10:00 PM', 
    '2021/01/12 23:00', '13/01/2021 12:60', 'January 14, 2021 12:00', 
    '15/01/2021', '2021/16/01 14:00', '17-01-2021', '18/01/2021 25:00', 
    '2021-01-19T15:30', '20th Jan 2021 16:00', '20 Jan 2021 16:00:10', '21/01/2021 16:00:60', 
    'not a datetime', '2021/01/23', '24/01/2021 26:30', '14 January 2021 12:00',
    np.nan, None, 'Null', '', '  ', '29/02/2021 15:20','2021-02-30 15:20:05', '@',
    '3/4/2121 13:00', '14/5/2222 13:05', '01/01/1500 13:00:10', '31/12/2321 13:20', 
    '2121/12/25 13:00:12' # Dates before 1800 or greater than 2100
]

'''timestamps = [
    '01/01/2021 13:00', '02/01/2021 14:30', '03/01/2021 15:45', 
    '04/01/2021 16:00', '05/01/2021 17:15', '06/01/2021 18:30',
    '07/01/2021 19:45', '08/01/2021 20:00', '09/01/2021 21:15', 
    '10/01/2021 22:30']'''

'''timestamps = ['2010-12-01 08:26:00','2010-12-02 08:26:00','2010-12-03 08:28:00','2010-12-01 08:26:00','2010-12-01 08:26:45','2010-12-01 08:26:50',
              '2010-12-01 08:27:00','2010-12-01 08:27:00','2010-12-01 08:26:15','2010-12-01 08:26:00','2010-12-01 08:26:05','2010-12-01 08:26:10',
              '2010-12-01 08:26:20','2010-12-01 08:26:06']'''

df_test_timestamps = pd.DataFrame({'timestamp_column': timestamps})
result = check_datetime(df_test_timestamps, 'timestamp_column')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Deduced format: DDMMYYYY
Processed date sample: 10/01/2021, detected format counts: {'DDMMYYYY': 10, 'MMDDYYYY': 0, 'YYYYMMDD': 0}

Error(s) found: 
DQI #1 (Missing Data - Completeness):
 6 Blank/Empty/Null/Missing value(s) at index(es): 
3 NULL values: [(27, 'NULL'), (28, 'NULL'), (29, 'NULL')]
3 Non-NULL values: [(30, 'Empty'), (31, 'Empty'), (34, '@')]

DQI #6 (Outdated Temporal Data - Timeliness):
 5 Datetime value(s) not in [1800-2100] period at index(es): [(35, '3/4/2121 13:00'), (36, '14/5/2222 13:05'), (37, '01/01/1500 13:00:10'), (38, '31/12/2321 13:20'), (39, '2121/12/25 13:00:12')]

DQI #14 (Different units/representations - Consistency):
 4 Datetime value(s) without format 'DDMMYYYY' in [1800-2100] period at index(es): [(10, '2021-01-11 23:45'), (12, '2021/01/12 23:00'), (14, 'January 14, 2021 12:00'), (24, '2021/01/23')]

DQI #13 (Temporal mismatch - Accuracy, Timeliness):
 9 Invalid datetime value(s) at index(es): [(13, '13/01/2021 12:60'), (16, '2021/16/01 14:00'), (18,

17 Check Time

In [1988]:
from datetime import datetime

def parse_timedelta(time_str):
    match = re.match(r'(\d+) days (\d{2}):(\d{2}):(\d{2})', time_str)
    if match:
        days, hours, minutes, seconds = map(int, match.groups())
        total_seconds = days * 86400 + hours * 3600 + minutes * 60 + seconds
        return time(hour=total_seconds // 3600 % 24, 
                    minute=(total_seconds % 3600) // 60, 
                    second=total_seconds % 60)
    return None

def parse_time(time_str):
    # If the input is already a time object, return it directly
    if isinstance(time_str, time):
        return time_str
    
    if pd.isna(time_str) or time_str == '':
        return None
    
    time_str = str(time_str)

    # Check for timedelta-like format
    timedelta_time = parse_timedelta(time_str)
    if timedelta_time:
        return timedelta_time
    
    time_formats = ['%H:%M', '%I:%M %p', '%H:%M:%S', '%I:%M:%S %p']
    for fmt in time_formats:
        try:
            return datetime.strptime(time_str, fmt).time()
        except ValueError:
            continue
    return None  # Return None for invalid time formats

        
def check_time(df, column):
    error_summary_parts = []
    
    # Calculate the total number of values in the column
    total_values_count = df[column].size
    
    # Handle Blank/Empty/Null/Missing values
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)
    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message'] + '\n')
    
    # Filtering df based on blank checks
    blank_indices = list(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)].copy()

    # Handle invalid time formats
    result_invalid_times = DataQualityIssues.handle_invalid_times(df_filtered, column)
    if result_invalid_times['issue']:
        error_summary_parts.append(result_invalid_times['dq_issue'] + ':\n ' + result_invalid_times['error_message'] + '\n')

    # Convert time strings to datetime.time objects
    #df_filtered['time_converted'] = df_filtered[column].apply(parse_time)
    df_filtered.loc[:, 'time_converted'] = df_filtered[column].apply(parse_time)

    valid_times = df_filtered['time_converted'].dropna()
    earliest_time = min(valid_times, default=None)
    latest_time = max(valid_times, default=None)

    time_range_summary = ""
    if earliest_time and latest_time:
        time_range_summary = f"\nTime range: ({earliest_time} to {latest_time})\n"
    else:
        time_range_summary = "No valid time values found.\n"

    final_summary = ""
    if error_summary_parts:
        final_summary = "Error(s) found: \n" + "\n".join(error_summary_parts) + time_range_summary
    else:
        final_summary = f"All {total_values_count} time values are valid in the range {earliest_time} to {latest_time}."

    return final_summary

# Example usage
#df_test = pd.DataFrame({'time_column': ['12:30', '02:30 PM', '14:30:15', '03:05 AM']})
df_test = pd.DataFrame({'time_column': ['12:30', '13:61', '02:30 PM', '14:30:15', '03:05 AM', 'invalid', '', None, '02:30 PN', '25:03',"*",'19:41:00']})

result = check_time(df_test, 'time_column')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Error(s) found: 
DQI #1 (Missing Data - Completeness):
 3 Blank/Empty/Null/Missing value(s) at index(es): 
1 NULL values: [(7, 'NULL')]
2 Non-NULL values: [(6, 'Empty'), (10, '*')]

DQI #13 (Temporal mismatch - Accuracy, Timeliness):
 4 Invalid time value(s) at index(es): [(1, '13:61'), (5, 'invalid'), (8, '02:30 PN'), (9, '25:03')]

Time range: (03:05:00 to 19:41:00)

Last run on: 2025-04-04 19:28:37


18 Check Model Name

In [1989]:
def check_model_name(df, column):
    error_summary_parts = []
    
    total_values_count = df[column].size
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)
    
    # Filtering df based on blank checks
    blank_indices = list(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)]

    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message']+ '\n')

    error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts) if error_summary_parts else ""

    # Frequency distribution calculation
    def represent_value(val):
        return 'NULL' if pd.isna(val) else str(val)

    model_counts = df[column].apply(represent_value).value_counts().to_dict()
    frequency_table = pd.DataFrame(model_counts.items(), columns=[column, 'Frequency'])
    frequency_table = frequency_table.sort_values(by=['Frequency', column], ascending=[False, True]).reset_index(drop=True)

    if len(frequency_table) > 20:
        top_rows = frequency_table.head(10)
        bottom_rows = frequency_table.tail(10)
        ellipsis_row = pd.DataFrame([("...", "...")], columns=[column, 'Frequency'])
        display_table = pd.concat([top_rows, ellipsis_row, bottom_rows], ignore_index=True)
    else:
        display_table = frequency_table

    frequency_distribution = f"\nFrequency Distribution:\n{display_table.to_string(index=False)}\n"

    # Get the range of values (smallest and largest)
    sorted_df = df.sort_values(by=column)
    smallest_model = sorted_df.iloc[0][column]
    biggest_model = sorted_df.iloc[-1][column]
    range_of_values = f"\nRange of Values: ({smallest_model} to {biggest_model})"

    return error_summary + frequency_distribution + range_of_values if error_summary_parts else f"All {total_values_count} {column} values are valid.\n{frequency_distribution}{range_of_values}"

# Test the function with your dataframe
df_test = pd.DataFrame({
    'model_name_column': [
        '32/60', '470v/7', 'vs-100', '', '  ', '?', None, 
        '90/80-model-3', '11',  '50-850-ii'
        'dn420', '580-5840'
    ]
})
result = check_model_name(df_test, 'model_name_column')
print(result)

print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")



Error(s) found: 
DQI #1 (Missing Data - Completeness):
 4 Blank/Empty/Null/Missing value(s) at index(es): 
1 NULL values: [(6, 'NULL')]
3 Non-NULL values: [(3, 'Empty'), (4, 'Empty'), (5, '?')]

Frequency Distribution:
model_name_column  Frequency
                           1
                           1
               11          1
            32/60          1
           470v/7          1
   50-850-iidn420          1
         580-5840          1
    90/80-model-3          1
                ?          1
             NULL          1
           vs-100          1

Range of Values: ( to None)
Last run on: 2025-04-04 19:28:37


18.5 Check Name

In [1990]:
import pandas as pd

def check_name(df, column):
    error_summary_parts = []
    linking_words = {'the', 'and', 'of', 'do', 'da', 'de', 'del', 'dos', 'e', 'md', 'ii', 'iii', 'iv', 'v', 'jr', 'sr', 'phd','von', 'van','y'}  # Set of lowercase linking words and suffixes
        
    # Calculate the total number of values in the column
    total_values_count = df[column].size

    # First, handle Blank/Empty/Null/Missing values
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)

    # Check if first 10 values are all lowercase
    first_10_lowercase = df[column].head(10).str.islower().all()

    # Filtering df based on blank checks
    blank_indices = list(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)]
    
    # Handling special characters and extraneous data
    result_extraneous = DataQualityIssues.handle_extraneous_data(df_filtered, column)

    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message'] + '\n')

    indices_to_exclude = set()

    if result_extraneous['issue']:
        # Convert indices to integers (if necessary)
        indices_to_exclude.update(result_extraneous.get('indices', []))
        #print("indices_to_exclude",indices_to_exclude)
        error_summary_parts.append(result_extraneous['dq_issue'] + ':\n ' + result_extraneous['error_message'] + '\n')

    if indices_to_exclude:
        #print(f"Indices to be excluded: {sorted(indices_to_exclude)}")    
        # Filter out rows with extraneous data
        df_filtered = df_filtered.loc[~df_filtered.index.isin(indices_to_exclude)]    
 
    # Perform capitalization format check only if not all first 10 names are lowercase
    if not first_10_lowercase:
        result_capitalization_format = DataQualityIssues.handle_capitalization_format(df_filtered, column, linking_words)
        if result_capitalization_format['issue']:
            error_summary_parts.append(result_capitalization_format['dq_issue'] + ':\n ' + result_capitalization_format['error_message']+ '\n')
    
    # Compile the final result message
    error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts) if error_summary_parts else ""

    # Frequency distribution calculation
    def represent_value(val):
        return 'NULL' if pd.isna(val) else str(val)

    name_counts = df[column].apply(represent_value).value_counts().to_dict()
    frequency_table = pd.DataFrame(name_counts.items(), columns=[column, 'Frequency'])
    frequency_table = frequency_table.sort_values(by=['Frequency', column], ascending=[False, True]).reset_index(drop=True)

    if len(frequency_table) > 20:
        top_rows = frequency_table.head(10)
        bottom_rows = frequency_table.tail(10)
        ellipsis_row = pd.DataFrame([("...", "...")], columns=[column, 'Frequency'])
        display_table = pd.concat([top_rows, ellipsis_row, bottom_rows], ignore_index=True)
    else:
        display_table = frequency_table

    error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts) if error_summary_parts else ""
    frequency_distribution = f"\nFrequency Distribution:\n{display_table.to_string(index=False)}\n"

    # Get the range of values (smallest and largest)
    '''sorted_df = df.sort_values(by=column)
    smallest_name = sorted_df.iloc[0][column]
    biggest_name = sorted_df.iloc[-1][column]
    range_of_values = f"\nRange of Values: ({smallest_name} to {biggest_name})"

    return error_summary + frequency_distribution + range_of_values if error_summary_parts else f"All {total_values_count} {column} values are valid.\n{frequency_distribution}{range_of_values}"'''

    non_empty_values = df[column].dropna().replace('', np.nan).dropna()
    if not non_empty_values.empty:
        smallest_name = non_empty_values.min()
        biggest_name = non_empty_values.max()
        range_of_values = f"\nRange of Values: ({smallest_name} to {biggest_name})"
    else:
        range_of_values = "\nNo valid values found for range calculation."

    return error_summary + frequency_distribution + range_of_values if error_summary_parts else f"All {total_values_count} {column} values are valid.\n{frequency_distribution}{range_of_values}"

# Test the function
df_test = pd.DataFrame({
    'name_column': [
        'John Doe', 'jane doe', 'Mr. Smith', 'Anne-Marie', '', '  ', '?',
        'John3 Doe', 'Emily!', '11', 'Mary Joe MD', 'John Williams II', 'Madonna',
        'Jean Paul Gautier, Jr', 'João Paulo Pereira e Souza Filho', 'José Augusto Napoleão Ferreira dos Santos', 
        'John F. Kennedy, Phd', 'John Newman, PhD', 'ORiley','McDonald', 'van Unen', 'von Brown', None, None, "D'arcy O'Connor"
    ]
})

'''df_test = pd.DataFrame({
    'name_column': [
        'John Doe','Mr. Smith', 'Anne-Marie', 'Madonna',
        'Jean Paul Gautier, Jr', 'João Paulo Pereira e Souza Filho', 'José Augusto Napoleão Ferreira dos Santos', 
        'John F. Kennedy, Phd', 'John Newman, PhD'
    ]
})'''
result = check_name(df_test, 'name_column')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Error(s) found: 
DQI #1 (Missing Data - Completeness):
 5 Blank/Empty/Null/Missing value(s) at index(es): 
2 NULL values: [(22, 'NULL'), (23, 'NULL')]
3 Non-NULL values: [(4, 'Empty'), (5, 'Empty'), (6, '?')]

DQI #5 (Extraneous Data - Consistency, Uniqueness):
 3 Extraneous data value(s) at index(es): [(7, 'John3 Doe'), (8, 'Emily!'), (9, '11')]

DQI #15 (Domain Violation - Accuracy):
 3 Capitalization/Format issue(s) at index(es): [(1, 'jane doe'), (18, 'ORiley'), (24, "D'arcy O'Connor")]

Frequency Distribution:
                              name_column Frequency
                                     NULL         2
                                                  1
                                                  1
                                       11         1
                                        ?         1
                               Anne-Marie         1
                          D'arcy O'Connor         1
                                   Emily!         1
           

19 Check Street

In [1991]:
def check_street(df, column):
    """
    Check if street names in the specified column conform to expected standards and provide a frequency distribution.
    
    Parameters:
    - df (pd.DataFrame): The pandas DataFrame to analyze.
    - column (str): The name of the column with street names.
    
    Returns:
    - str: A message indicating the result of the street name checks, including a frequency distribution.
    """
    
    # Calculate the total number of values in the column
    total_values_count = df[column].size
    
    error_summary_parts = []
    linking_words = {'the', 'and', 'of', 'do', 'da', 'de', 'del', 'e', 'th','rd', 'rue', 'du', 'das', 'des'}  # Set of lowercase linking words
    
    # First, handle Blank/Empty/Null/Missing values
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)
    
    # Filtering df based on blank checks
    blank_indices = set(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)]
    
    # Handling special characters and extraneous data
    result_street_extraneous = DataQualityIssues.handle_street_extraneous_data(df_filtered, column)
    
    extraneous_indices = set(result_street_extraneous.get("indices", []))
    df_filtered = df_filtered[~df_filtered.index.isin(extraneous_indices)]
    result_capitalization_format = DataQualityIssues.handle_capitalization_format(df_filtered, column, linking_words)
    
    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message'] + '\n')
    
    if result_street_extraneous['issue']:
        error_summary_parts.append(result_street_extraneous['dq_issue'] + ':\n ' + result_street_extraneous['error_message'] + '\n')
    
    if result_capitalization_format['issue']:
        error_summary_parts.append(result_capitalization_format['dq_issue'] + ':\n ' + result_capitalization_format['error_message']+ '\n')
    
    # Frequency distribution calculation
    street_counts = {}
    valid_streets = []
    for street in df[column]:
        category = DataQualityIssues.represent_value(street)
        if category == street:  # It's a valid street name
            street_counts[street] = street_counts.get(street, 0) + 1
            valid_streets.append(street)
        else:  # It's a special category
            street_counts[category] = street_counts.get(category, 0) + 1
            
    # Creating a frequency table sorted first by frequency and then alphabetically
    if street_counts:
        frequency_table = pd.DataFrame(street_counts.items(), columns=['Street', 'Frequency'])
        frequency_table = frequency_table.sort_values(by=['Frequency', 'Street'], ascending=[False, True]).reset_index(drop=True)

        # Select the first 10 and last 10 rows if more than 20 distinct items
        if len(frequency_table) > 20:
            top_rows = frequency_table.head(10)
            bottom_rows = frequency_table.tail(10)
            ellipsis_row = pd.DataFrame([("...", "...")], columns=['Street', 'Frequency'])
            display_table = pd.concat([top_rows, ellipsis_row, bottom_rows], ignore_index=True)
            freq_dist_str = f"Frequency Distribution (showing top and bottom 10 of {len(frequency_table)} streets):\n{display_table.to_string(index=False)}\n"
        else:
            freq_dist_str = f"Frequency Distribution:\n{frequency_table.to_string(index=False)}\n"
    else:
        freq_dist_str = "\nNo valid street values found for frequency distribution."

    # Compile the final result message
    error_summary = ''
    if error_summary_parts:
        error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts) + "\n"

    # Get the range of values
    sorted_df = df.sort_values(by=column)
    smallest_name = sorted_df.iloc[0][column]
    biggest_name = sorted_df.iloc[-1][column]
    range_str = f"\nRange of Values: ({smallest_name} to {biggest_name})"

    if error_summary_parts:
        final_message = error_summary + freq_dist_str + range_str
    else:
        final_message = f"All {total_values_count} street values are valid.\n" + freq_dist_str + range_str

    return final_message

# Sample data for street checks
'''streets = [
    '123 Main St', '45 Oxford Road', 'Broadway Ave', '5th Avenue', 
    'Mt. Everest Street', 'InvalidStreet', '12, Elm Street', '77 Sunset Strip', 
    '221B Baker Street', 'Elm St.', 'Ocean Drive', 'Park Ave', 'Sesame St', 
    'Main Street 123', 'Pennsylvania Avenue NW', 'Sunset boulevard', 
    'Abbey Road', 'Fleet Street', 'Diagon Alley', '15/250 Beaufort St',
    None, '', '  ', '?', 'John3 Doe', 'Emily!', '11', 'R. Prof Paulo Roberto Martins, 2', 'Null',
    "27-30 Merchant's Quay","27 30 Merchant's Quay","27-30 Merchants Quay", "59 rue de l'Abbaye", "59 rue de Abbaye",
    '12, rue des Bouchers','Berguvsvägen  8', 'Jardim das rosas n. 32', 'Jardim das Rosas 32',
    'c/ Gobelas, 19-1 Urb. La Florida', 'Gobelas, 19-1 Urb. La Florida','NatWest Center #13-03','Natwest Center 13-03'
]'''

'''streets = [
    '123 Main St', '45 Oxford Road', 'Broadway Ave', '5th Avenue', 
    'Mt. Everest Street', '12, Elm Street', '77 Sunset Strip', 
    '221B Baker Street', 'Elm St.', 'Ocean Drive', 'Park Ave', 'Sesame St', 
    'Main Street 123', 'Pennsylvania Avenue NW', 
    'Abbey Road', 'Fleet Street', 'Diagon Alley', '15/250 Beaufort St',
    'R. Prof Paulo Roberto Martins, 2'
]'''
streets = [None, '', '  ', '?', 'Berguvsvägen  8']
df_streets = pd.DataFrame({'street': streets})
result = check_street(df_streets, 'street')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Error(s) found: 
DQI #1 (Missing Data - Completeness):
 4 Blank/Empty/Null/Missing value(s) at index(es): 
1 NULL values: [(0, 'NULL')]
3 Non-NULL values: [(1, 'Empty'), (2, 'Empty'), (3, '?')]

Frequency Distribution:
         Street  Frequency
          Empty          2
              ?          1
Berguvsvägen  8          1
           NULL          1

Range of Values: ( to None)
Last run on: 2025-04-04 19:28:37


20 Check City

In [1992]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

def check_city(df, column):
    """
    Check if city names in the specified column conform to expected standards and provide a comprehensive frequency distribution.
    """
    total_values_count = df[column].size
    city_counts = {}
    linking_words = {'the', 'and', 'of', 'do', 'da', 'de', 'del', 'e', 'dos'}
    error_summary_parts = []

    # Frequency distribution calculation (including all values)
    for city in df[column]:
        if pd.isna(city):
            if city is None:
                city_str = 'NULL'
            elif isinstance(city, float) and np.isnan(city):
                city_str = '[NaN]'
            else:
                city_str = '[OTHER_NA]'
        else:
            city_str = str(city)  # Keep everything as is, including empty strings and spaces
        city_counts[city_str] = city_counts.get(city_str, 0) + 1

    # Error checking
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)
    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message'] + '\n')

    df_filtered = df[~df.index.isin(result_blank["blank_indices"])]

    result_extraneous = DataQualityIssues.handle_extraneous_data(df_filtered, column)
    if result_extraneous['issue']:
        error_summary_parts.append(result_extraneous['dq_issue'] + ':\n ' + result_extraneous['error_message'] + '\n')

    result_capitalization_format = DataQualityIssues.handle_capitalization_format(df_filtered, column, linking_words)
    if result_capitalization_format['issue']:
        error_summary_parts.append(result_capitalization_format['dq_issue'] + ':\n ' + result_capitalization_format['error_message'] + '\n')

    # Creating a frequency table sorted by frequency (descending) and then alphabetically
    frequency_table = pd.DataFrame(city_counts.items(), columns=['City', 'Frequency'])
    frequency_table = frequency_table.sort_values(by=['Frequency', 'City'], ascending=[False, True]).reset_index(drop=True)

    # Prepare the display table
    if len(frequency_table) > 20:
        top_rows = frequency_table.head(10)
        bottom_rows = frequency_table.tail(10)
        ellipsis_row = pd.DataFrame([("...", "...")], columns=['City', 'Frequency'])
        display_table = pd.concat([top_rows, ellipsis_row, bottom_rows], ignore_index=True)
    else:
        display_table = frequency_table

    frequency_distribution = f"\nFrequency Distribution (showing top and bottom 10 of {len(frequency_table)} cities):\n{display_table.to_string(index=False)}\n"

    if not error_summary_parts:
        return f"All {total_values_count} city values are valid.\n{frequency_distribution}"

    error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts)
    return error_summary + frequency_distribution

# Sample data for city checks
cities = [
    'New York', 'London', 'Paris', 'Tokyo', 'Tokyo',
    'los angeles', 'Sydney', 'Beijing', 'Cairo', 'new delhi', 'San Francisco', 
    'San francisco', 'Chicago', 'Boston', 'Berlin', 'Amsterdam', 
    'Hong Kong', 'Singapore', 'Dubai', 'Moscow', 'São Paulo','São José dos Campos',
    None, '', '  ', '?', 'Dubai!',  11, None
]

'''cities = [
    'New York', 'London', 'Paris', 'Tokyo', 'Tokyo',
    'Sydney', 'Beijing', 'Cairo', 'San Francisco', 
    'Chicago', 'Boston', 'Berlin', 'Amsterdam', 
    'Hong Kong', 'Singapore', 'Dubai', 'Moscow', 'São Paulo','São José dos Campos',
    'NY', 'Los Angeles', 'Cairo', 'Brasilia'
]'''
df_cities = pd.DataFrame({'city': cities})
result = check_city(df_cities, 'city')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Error(s) found: 
DQI #1 (Missing Data - Completeness):
 5 Blank/Empty/Null/Missing value(s) at index(es): 
2 NULL values: [(22, 'NULL'), (28, 'NULL')]
3 Non-NULL values: [(23, 'Empty'), (24, 'Empty'), (25, '?')]

DQI #5 (Extraneous Data - Consistency, Uniqueness):
 2 Extraneous data value(s) at index(es): [(26, 'Dubai!'), (27, 11)]

DQI #15 (Domain Violation - Accuracy):
 3 Capitalization/Format issue(s) at index(es): [(5, 'los angeles'), (9, 'new delhi'), (11, 'San francisco')]

Frequency Distribution (showing top and bottom 10 of 27 cities):
               City Frequency
               NULL         2
              Tokyo         2
                            1
                            1
                 11         1
                  ?         1
          Amsterdam         1
            Beijing         1
             Berlin         1
             Boston         1
                ...       ...
           New York         1
              Paris         1
      San Francisco         1


21 Check State

In [1993]:
def check_state(df, column):
    """
    Check if state names in the specified column conform to expected standards of capitalization
    and provide a frequency distribution, allowing certain lowercase words and abbreviations.

    Parameters:
    - df (pd.DataFrame): The pandas DataFrame to analyze.
    - column (str): The name of the column with state names.

    Returns:
    - str: A message indicating the result of the state name checks and their frequency distribution.
    """
    incorrect_indices_and_values = []
    state_counts = {}
    lowercase_exceptions = {"e", "do", "dos", "da", "das", "de","dos", "of", "y"}  # Lowercase exceptions
    
    # Calculate the total number of values in the column
    total_values_count = df[column].size

    # First, handle Blank/Empty/Null/Missing values
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)

    # Filtering df based on blank checks
    blank_indices = list(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)]

    # Handling special characters and extraneous data
    result_extraneous = DataQualityIssues.handle_extraneous_data(df_filtered, column)
    result_capitalization_format = DataQualityIssues.handle_capitalization_format(df_filtered, column, lowercase_exceptions)

    error_summary_parts = []

    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message']+ '\n')

    if result_extraneous['issue']:
        error_summary_parts.append(result_extraneous['dq_issue'] + ':\n ' + result_extraneous['error_message']+ '\n')

    if result_capitalization_format['issue']:
        error_summary_parts.append(result_capitalization_format['dq_issue'] + ':\n ' + result_capitalization_format['error_message']+ '\n')

    # Frequency distribution calculation
    for idx, state in df.iterrows():
        state_value = state[column]
        if pd.isna(state_value):
            state_str = 'NULL'
        else:
            state_str = str(state_value).strip()
        
        # Counting occurrences of each state
        state_counts[state_str] = state_counts.get(state_str, 0) + 1

    # Creating a frequency table sorted first by frequency and then alphabetically
    if state_counts:
        frequency_table = pd.DataFrame(state_counts.items(), columns=['State', 'Frequency'])
        frequency_table = frequency_table.sort_values(by=['Frequency', 'State'], ascending=[False, True]).reset_index(drop=True)

        # Select the first 10 and last 10 rows if more than 20 distinct items
        top_rows = frequency_table.head(10)
        bottom_rows = frequency_table.tail(10)
        if len(frequency_table) > 20:
            ellipsis_row = pd.DataFrame([("...", "...")], columns=['State', 'Frequency'])
            display_table = pd.concat([top_rows, ellipsis_row, bottom_rows], ignore_index=True)
        else:
            display_table = frequency_table

        result_str = f"\nFrequency Distribution (showing top and bottom 10 of {len(frequency_table)} states/provinces):\n{display_table.to_string(index=False)}\n"   

        # Compile the final result message
        error_summary = ''
        if error_summary_parts:
            error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts) + "\n"

        final_message = error_summary + result_str

        # If no errors are found, return a message stating that all values are valid along with frequency distribution
        if not error_summary_parts:
            final_message = f"All {total_values_count} state values are valid.\n{result_str}"

        return final_message

# Sample data for state checks
states = [
     'CA2', 'New York', 'Texas', 'FL', 'Texas',
    'Nevada', 'WA', 'Queensland', 'Bavaria', 'Delhi', 'New york',
    'Illinois', 'Victoria', 'Ontario', 'Colorado', 'Arizona', 
    'NSW', 'Gauteng', 'Hawaii', 'Alaska', 'Punjab', 'new south wales',
    None, '', '  ', '?', 'California!', '11', 'Rio Grande do Sul', 'new Jersey', 'n york', 'N. Dakota', 'São Paulo','Isle of Wight', None
]

'''states = [
     'CA', 'New York', 'Texas', 'FL', 'Texas',
    'Nevada', 'WA', 'Queensland', 'Bavaria', 'Delhi', 'New York',
    'Illinois', 'Victoria', 'Ontario', 'Colorado', 'Arizona', 
    'NSW', 'Gauteng', 'Hawaii', 'Alaska', 'Punjab', 
    'Rio Grande do Sul', 'N. Dakota', 'São Paulo'
]'''

df_states = pd.DataFrame({'state': states})
result = check_state(df_states, 'state')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Error(s) found: 
DQI #1 (Missing Data - Completeness):
 5 Blank/Empty/Null/Missing value(s) at index(es): 
2 NULL values: [(22, 'NULL'), (34, 'NULL')]
3 Non-NULL values: [(23, 'Empty'), (24, 'Empty'), (25, '?')]

DQI #5 (Extraneous Data - Consistency, Uniqueness):
 3 Extraneous data value(s) at index(es): [(0, 'CA2'), (26, 'California!'), (27, '11')]

DQI #15 (Domain Violation - Accuracy):
 4 Capitalization/Format issue(s) at index(es): [(10, 'New york'), (21, 'new south wales'), (29, 'new Jersey'), (30, 'n york')]


Frequency Distribution (showing top and bottom 10 of 32 states/provinces):
            State Frequency
                          2
             NULL         2
            Texas         2
               11         1
                ?         1
           Alaska         1
          Arizona         1
          Bavaria         1
              CA2         1
      California!         1
              ...       ...
          Ontario         1
           Punjab         1
       Que

22 Check Country

In [1994]:
def check_country(df, column):
    """
    Check if country names in the specified column conform to expected standards of capitalization 
    and provide a frequency distribution.
    """
    incorrect_indices_and_values = []
    country_counts = {}
    linking_words = {'the', 'and', 'of', 'do', 'da', 'de', 'del', 'e', 'dos','etc'}  # Set of lowercase linking words

    # Calculate the total number of values in the column
    total_values_count = df[column].size

    # First, handle Blank/Empty/Null/Missing values
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)

    # Filtering df based on blank checks
    blank_indices = list(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)]

    # Handling special characters and extraneous data
    result_extraneous = DataQualityIssues.handle_extraneous_data(df_filtered, column)

    if result_extraneous['issue']:
        indices_to_exclude_extraneous = result_extraneous['indices']  # Directly use the indices
        df_filtered = df_filtered.drop(index=indices_to_exclude_extraneous)

    result_capitalization_format = DataQualityIssues.handle_capitalization_format_country(df_filtered, column, linking_words)

    error_summary_parts = []

    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message']+ '\n')

    if result_extraneous['issue']:
        error_summary_parts.append(result_extraneous['dq_issue'] + ':\n ' + result_extraneous['error_message']+ '\n')

    if result_capitalization_format['issue']:
        error_summary_parts.append(result_capitalization_format['dq_issue'] + ':\n ' + result_capitalization_format['error_message']+ '\n')

    # Frequency distribution calculation
    for idx, country in df.iterrows():
        country_value = country[column]
        if pd.isna(country_value):
            country_str = 'NULL'
        else:
            country_str = str(country_value).strip()
        
        # Skip Blank/Empty/Null/Missing values and incorrect values for frequency calculation
        if country_str and country_str not in [item[1] for item in incorrect_indices_and_values]:
            # Counting occurrences of each country
            country_counts[country_str] = country_counts.get(country_str, 0) + 1

    # Creating a frequency table sorted first by frequency and then alphabetically
    if country_counts:
        frequency_table = pd.DataFrame(country_counts.items(), columns=['Country', 'Frequency'])
        frequency_table = frequency_table.sort_values(by=['Frequency', 'Country'], ascending=[False, True]).reset_index(drop=True)

        # Select the first 10 and last 10 rows if more than 20 distinct items
        top_rows = frequency_table.head(10)
        bottom_rows = frequency_table.tail(10)
        if len(frequency_table) > 20:
            ellipsis_row = pd.DataFrame([("...", "...")], columns=['Country', 'Frequency'])
            display_table = pd.concat([top_rows, ellipsis_row, bottom_rows], ignore_index=True)
            result_str = f"Frequency Distribution (showing top and bottom 10 of {len(frequency_table)} countries):\n{display_table.to_string(index=False)}\n"
        else:
            result_str = f"Frequency Distribution:\n{frequency_table.to_string(index=False)}\n"
    else:
        result_str = "All country values are valid"

    # Compile the final result message
    error_summary = ''
    if error_summary_parts:
        error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts) + "\n"

    final_message = error_summary + result_str

    # If no errors are found, return a message stating that all values are valid along with frequency distribution
    if not error_summary_parts:
        final_message = f"All {total_values_count} country values are valid.\n{result_str}"

    return final_message


# Test function
countries = [
    'USA', 'United Kingdom', 'France', 'Japan', 'Australia', 
    'India', 'Australia', 'China', 'Egypt', 'Canada', 
    'Germany', 'Brazil', 'South Africa', 'Russia', 'puerto rico', 
    'ITA', 'SPA', 'BR', 'SWE', 'Papua New Guinea',
    None, '', '  ', '?', 'Canada!', '11', 'The United States of America', 'Outlying-US (Guam-USVI-etc)',
    'guatemala', 'papua New Guinea', 'Saint Vincent and the Grenadines', 'Null', 'US-Virgin-Isles', None
]

'''"AnalysedColumns 2202.xlsx"countries = [
    'USA', 'United Kingdom', 'France', 'Japan', 'Australia', 
    'India', 'Australia', 'China', 'Egypt', 'Canada', 
    'Germany', 'Brazil', 'South Africa', 'Russia', 'Puerto Rico', 
    'ITA', 'SPA', 'BR', 'SWE', 'Papua New Guinea',
    'Canada', 'The United States of America',
    'Guatemala', 'Papua New Guinea', 'Saint Vincent and the Grenadines', 'US-Virgin-Isles','?'
]'''

df_countries = pd.DataFrame({'country': countries})
result = check_country(df_countries, 'country')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Error(s) found: 
DQI #1 (Missing Data - Completeness):
 6 Blank/Empty/Null/Missing value(s) at index(es): 
3 NULL values: [(20, 'NULL'), (31, 'NULL'), (33, 'NULL')]
3 Non-NULL values: [(21, 'Empty'), (22, 'Empty'), (23, '?')]

DQI #5 (Extraneous Data - Consistency, Uniqueness):
 2 Extraneous data value(s) at index(es): [(24, 'Canada!'), (25, '11')]

DQI #15 (Domain Violation - Accuracy):
 3 Capitalization/Format issue(s) at index(es): [(14, 'puerto rico'), (28, 'guatemala'), (29, 'papua New Guinea')]

Frequency Distribution (showing top and bottom 10 of 30 countries):
                         Country Frequency
                       Australia         2
                            NULL         2
                              11         1
                               ?         1
                              BR         1
                          Brazil         1
                          Canada         1
                         Canada!         1
                           China      

23 Check Postal Code

In [1995]:
def check_postal_code(df: pd.DataFrame, column: str) -> dict:
    """
    Check if postal code entries in the specified column are valid.
    """
    # Calculate the total number of values in the column
    total_values_count = df[column].size

    # First, handle Blank/Empty/Null/Missing values
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)

    # Filtering df based on blank checks
    blank_indices = list(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)]
    
    result_non_alphanumeric = DataQualityIssues.handle_non_alphanumeric_values(df_filtered, column)
    result_short_length = DataQualityIssues.handle_short_length_values(df_filtered, column, 4)

    error_summary_parts = []

    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message']+ '\n')

    if result_short_length['issue']:
        error_summary_parts.append(result_short_length['dq_issue'] + ':\n ' + result_short_length['error_message']+ '\n')
        
    if result_non_alphanumeric['issue']:
        error_summary_parts.append(result_non_alphanumeric['dq_issue'] + ':\n ' + result_non_alphanumeric['error_message']+ '\n')
 
    error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts)  
    # Sort the DataFrame by the name column
    sorted_df = df.sort_values(by=column)

    # Get the first and last name after sorting
    smallest_name = sorted_df.iloc[0][column]
    biggest_name = sorted_df.iloc[-1][column]
 
    return error_summary if error_summary_parts else f"All {total_values_count} postal codes values are valid in the range ({smallest_name} to {biggest_name}).\n"

# Sample data for postal code checks, including edge cases
postal_codes = [
    '10001', 'SW1A 1AA', '75008', '100-0001', 
    '110001', '2000', '100000', '11511', 'M4W 1A8', 
    '10115', '01311', '2001', '101000', '06500', 
    '00184', '28013', '1012 WX', '111 20', '0101', 
    '71676-110', '6000', None, '', '  ', '?', '1000!', '11'
]

'''postal_codes = [
    '10001', 'SW1A 1AA', '75008', '100-0001', 
    '110001', '2000', '100000', '11511', 'M4W 1A8', 
    '10115', '01311', '2001', '101000', '06500', 
    '00184', '28013', '1012 WX', '111 20', '0101', 
    '71676-110', '6000'
]'''

df_postal_codes = pd.DataFrame({'postal_code': postal_codes})

# Test the function
result = check_postal_code(df_postal_codes, 'postal_code')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Error(s) found: 
DQI #1 (Missing Data - Completeness):
 4 Blank/Empty/Null/Missing value(s) at index(es): 
1 NULL values: [(21, 'NULL')]
3 Non-NULL values: [(22, 'Empty'), (23, 'Empty'), (24, '?')]

DQI #15 (Domain Violation - Accuracy):
 1 Short length alphanumeric value(s) at index(es): [(26, '11')]

DQI #17 (Wrong Data Type - Consistency):
 1 Non-alphanumeric value(s) at index(es): [(25, '1000!')]

Last run on: 2025-04-04 19:28:37


24 Check Phone Numbers

In [1996]:
def check_phone_numbers(df, column):
    """
    Check if phone number entries in the specified column are valid.
    """
    # Calculate the total number of values in the column
    total_values_count = df[column].size

    # First, handle Blank/Empty/Null/Missing values
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)

    # Filtering df based on blank checks
    blank_indices = list(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)]

    result_format = DataQualityIssues.handle_phone_number_format(df_filtered, column)

    error_summary_parts = []

    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message']+ '\n')

    if result_format['issue']:
        error_summary_parts.append(result_format['dq_issue'] + ':\n ' + result_format['error_message'])
      
    error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts)
    
    # Convert all entries to strings for sorting
    df[column] = df[column].astype(str)
    sorted_df = df.sort_values(by=column)

    # Get the first and last name after sorting
    smallest_name = sorted_df.iloc[0][column]
    biggest_name = sorted_df.iloc[-1][column]
 
    return error_summary if error_summary_parts else f"All {total_values_count} telephone numbers are valid in the range ({smallest_name} to {biggest_name}).\n"
    
# Test data
df_test = pd.DataFrame({
    'phone_numbers': [
        '123-456-7890', '(123) 456-7890', '+1 123 456 7890', 'InvalidNumber', 
        '+55 21 11 3415 1515', '04148991268624', '+55 48 3224-4209', '+55 48 91268-624', 
        '000', '+61137425', '+1 414-690-7935', '04121993720444', '01188335944', 
        '4144494331', '+55 31 3414-2179', '+61 405 833 952', '0405 833 952',
        None, '', '  ', '?', 'John Doe', '0405 833 952!', 11, 'A191'
    ]
})

'''df_test = pd.DataFrame({
    'phone_numbers': [
        '123-456-7890', '(123) 456-7890', '+1 123 456 7890', 
        '+55 21 11 3415 1515', '04148991268624', '+55 48 3224-4209', '+55 48 91268-624', 
        '000', '+61137425', '+1 414-690-7935', '04121993720444', '01188335944', 
        '4144494331', '+55 31 3414-2179', '+61 405 833 952', '0405 833 952',
        '0405 833 952'
    ]
})'''

result = check_phone_numbers(df_test, 'phone_numbers')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Error(s) found: 
DQI #1 (Missing Data - Completeness):
 4 Blank/Empty/Null/Missing value(s) at index(es): 
1 NULL values: [(17, 'NULL')]
3 Non-NULL values: [(18, 'Empty'), (19, 'Empty'), (20, '?')]

DQI #15 (Domain Violation - Accuracy):
 5 Incorrect telephone number format issue(s) at index(es): [(3, 'InvalidNumber'), (21, 'John Doe'), (22, '0405 833 952!'), (23, 11), (24, 'A191')]
Last run on: 2025-04-04 19:28:37


25 Check IP format

In [1997]:
def check_ip_format(df, column):

    # Calculate the total number of values in the column
    total_values_count = df[column].size

    # Handle Blank/Empty/Null/Missing values first
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)
    # Filtering df based on blank checks
    blank_indices = list(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)]

    # Then check IP format
    result_ip_format = DataQualityIssues.handle_ip_format(df_filtered, column)

    error_summary_parts = []

    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message'] + '\n')

    if result_ip_format['issue']:
        error_summary_parts.append(result_ip_format['dq_issue'] + ':\n ' + result_ip_format['error_message'] + '\n')

    return "Error(s) found: \n" + "\n".join(error_summary_parts) if error_summary_parts else f"All {total_values_count} IP values are valid."

# Example usage
df_test = pd.DataFrame({
    'ip_column': [
        '192.168.1.1', '256.256.256.256', '127.0.0.1', '1.1', 
        '2001:0db8:85a3:0000:0000:8a2e:0370:7334', '::1', 
        '2001:db8::1234:5678', 'fe80::1ff:fe23:4567:890a', 
        None, '', '  ', '?', '0.0.0.0.0!', '11', 'incorrect:ipv6:address'
    ]
})

'''df_test = pd.DataFrame({
    'ip_column': [
        '192.168.1.1', '256.256.256.256', '127.0.0.1'
    ]
})'''

result = check_ip_format(df_test, 'ip_column')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Error(s) found: 
DQI #1 (Missing Data - Completeness):
 4 Blank/Empty/Null/Missing value(s) at index(es): 
1 NULL values: [(8, 'NULL')]
3 Non-NULL values: [(9, 'Empty'), (10, 'Empty'), (11, '?')]

DQI #15 (Domain Violation - Accuracy):
 7 Invalid IP format issue(s) at index(es): [(3, '1.1'), (5, '::1'), (6, '2001:db8::1234:5678'), (7, 'fe80::1ff:fe23:4567:890a'), (12, '0.0.0.0.0!'), (13, '11'), (14, 'incorrect:ipv6:address')]

Last run on: 2025-04-04 19:28:37


26 Check URL format

In [1998]:
def check_url_format(df, column):
    # Calculate the total number of values in the column
    total_values_count = df[column].size
    
    # Handle Blank/Empty/Null/Missing values first
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)
    # Filtering df based on blank checks
    blank_indices = list(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)]

    # Then check URL format
    result_url_format = DataQualityIssues.handle_url_format(df_filtered, column)

    error_summary_parts = []

    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message'] + '\n')

    if result_url_format['issue']:
        error_summary_parts.append(result_url_format['dq_issue'] + ':\n ' + result_url_format['error_message'] + '\n')

    # Frequency distribution calculation
    url_counts = {}
    for url in df[column]:
        category = DataQualityIssues.represent_value(url)
        if category == url:  # It's a valid URL
            url_counts[url] = url_counts.get(url, 0) + 1
        else:  # It's a special category
            url_counts[category] = url_counts.get(category, 0) + 1

    # Creating a frequency table
    if url_counts:
        frequency_table = pd.DataFrame(url_counts.items(), columns=['URL', 'Frequency'])
        frequency_table = frequency_table.sort_values(by=['Frequency', 'URL'], ascending=[False, True]).reset_index(drop=True)

        if len(frequency_table) > 20:
            top_rows = frequency_table.head(10)
            bottom_rows = frequency_table.tail(10)
            ellipsis_row = pd.DataFrame([("...", "...")], columns=['URL', 'Frequency'])
            display_table = pd.concat([top_rows, ellipsis_row, bottom_rows], ignore_index=True)
            freq_dist_str = f"\nFrequency Distribution (showing top and bottom 10 of {len(frequency_table)} entries):\n{display_table.to_string(index=False)}\n"
        else:
            freq_dist_str = f"\nFrequency Distribution:\n{frequency_table.to_string(index=False)}\n"
    else:
        freq_dist_str = "\nNo URL values found for frequency distribution."

    # Compile the final result message
    error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts) if error_summary_parts else ""

    final_message = (error_summary + freq_dist_str) if error_summary_parts else \
                    f"All {total_values_count} URL values are valid.\n" + freq_dist_str

    return final_message
test_urls = [
    'https://www.example.com', 'http://example.org', 'http://192.168.1.1', 'http://localhost/test',
    'https://www.example.com:8080/path/to/resource', 'ftp://example.com', 'http://exa_mple.com',
    'http://999.999.999.999', 'https://', 'http://', 'http://exam!ple.com', 'https://www.example..com',
    'http:// example .com', 'justsometext', '12345', '', '  ', None, 'null',
    'https://chat.openai.com/c/9c317ba2-cefe-44b9-b9f4-7ef818744434',
    'https:--www.uol.com.br', 'https://www.uol.com.br', 'https:///www.uol.com.br'
]

'''test_urls = [
    'https://www.example.com', 'http://example.org', 'http://192.168.1.1', 'http://localhost/test',
    'https://www.example.com:8080/path/to/resource', 'https://www.uol.com.br'
]'''

# Example usage
df_test_urls = pd.DataFrame({'url_column': test_urls})
result = check_url_format(df_test_urls, 'url_column')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Error(s) found: 
DQI #1 (Missing Data - Completeness):
 4 Blank/Empty/Null/Missing value(s) at index(es): 
2 NULL values: [(17, 'NULL'), (18, 'NULL')]
2 Non-NULL values: [(15, 'Empty'), (16, 'Empty')]

DQI #15 (Domain Violation - Accuracy):
 11 Invalid URL format issue(s) at index(es): [(5, 'ftp://example.com'), (6, 'http://exa_mple.com'), (8, 'https://'), (9, 'http://'), (10, 'http://exam!ple.com'), (11, 'https://www.example..com'), (12, 'http:// example .com'), (13, 'justsometext'), (14, '12345'), (20, 'https:--www.uol.com.br'), (22, 'https:///www.uol.com.br')]

Frequency Distribution (showing top and bottom 10 of 21 entries):
                                                           URL Frequency
                                                         Empty         2
                                                          NULL         2
                                                         12345         1
                                             ftp://example.com         

27 Check Email Format

In [1999]:

def check_email_format(df, column):
    # Calculate the total number of values in the column
    total_values_count = df[column].size

    # Handle Blank/Empty/Null/Missing values first
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)

    # Filtering df based on blank checks
    blank_indices = list(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)]

    # Then check email format
    result_email_format = DataQualityIssues.handle_email_format(df_filtered, column)

    error_summary_parts = []

    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message'] + '\n')

    if result_email_format['issue']:
        error_summary_parts.append(result_email_format['dq_issue'] + ':\n ' + result_email_format['error_message'] + '\n')

    return "Error(s) found: \n" + "\n".join(error_summary_parts) if error_summary_parts else f"All {total_values_count} email values are valid."

# Example usage

df_test = pd.DataFrame({
    'email_column': [
        'example.com', 'userexample.com', 'name.domain.com',  # Missing @ Symbol
        'user@.com', 'name@',  # Missing Domain
        'user name@example.com', 'user@ exam ple.com', 'user @example.com',  # Spaces in Email Address
        'user!@example.com', 'name#domain.com', 'user*name@example.com',  # Special Characters
        'user@@example.com', 'name@domain@domain.com',  # Multiple @ Symbols
        '@example.com', '@domain.com',  # Missing Username
        'user@example.c', 'name@domain.',  # Domain Extension Too Short or Missing
        'user..name@example.com', 'user@domain..com',  # Consecutive Dots
        'user@-example.com', 'user@domain--name.com',  # Dashes in Domain
        'user@[192.168.0.1]', 'name@[123.123.123.123]',  # IP Address in Domain
        'user[name]@example.com', 'name[123]@domain.com',  # Brackets in Local Part
        'a'*255 + '@example.com',  # Too Long Email Address
        '', '  ', None, 'null', '%'
    ]
})

'''df_test = pd.DataFrame({
    'email_column': [
        'marcelo.valentimsilva@postgrad.curtin.edu.au', 'marcelovalentimsilva@gmail.com', 'marcelo_valentim@uol.com.br'
    ]
})'''

result = check_email_format(df_test, 'email_column')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Error(s) found: 
DQI #1 (Missing Data - Completeness):
 5 Blank/Empty/Null/Missing value(s) at index(es): 
2 NULL values: [(28, 'NULL'), (29, 'NULL')]
3 Non-NULL values: [(26, 'Empty'), (27, 'Empty'), (30, '%')]

DQI #15 (Domain Violation - Accuracy):
 24 Invalid email format issue(s) at index(es): [(0, 'example.com'), (1, 'userexample.com'), (2, 'name.domain.com'), (3, 'user@.com'), (4, 'name@'), (5, 'user name@example.com'), (6, 'user@ exam ple.com'), (7, 'user @example.com'), (9, 'name#domain.com'), (11, 'user@@example.com'), ('...', '...'), (16, 'name@domain.'), (17, 'user..name@example.com'), (18, 'user@domain..com'), (19, 'user@-example.com'), (20, 'user@domain--name.com'), (21, 'user@[192.168.0.1]'), (22, 'name@[123.123.123.123]'), (23, 'user[name]@example.com'), (24, 'name[123]@domain.com'), (25, 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

28 Check Binary Values

In [2000]:
import pandas as pd
import numpy as np
from collections import OrderedDict

def check_binary_values(df, column):
    """
    Check if the values in the specified column conform to binary values and provide a comprehensive frequency distribution.
    """
    total_values_count = df[column].size
    binary_counts = OrderedDict()
    error_summary_parts = []

    # Frequency distribution calculation (including all values)
    for value in df[column]:
        if pd.isna(value):
            if value is None:
                value_str = 'NULL'
            elif isinstance(value, float) and np.isnan(value):
                value_str = '[NaN]'
            else:
                value_str = '[OTHER_NA]'
        elif value == '':
            value_str = '[EMPTY]'
        elif isinstance(value, str) and value.isspace():
            value_str = '[SPACE]'
        elif isinstance(value, (int, float)):
            value_str = str(value)  # Preserve numeric values as strings
        else:
            value_str = str(value)  # Preserve original case for strings
        binary_counts[value_str] = binary_counts.get(value_str, 0) + 1

    # Error checking
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)
    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message'] + '\n')

    df_filtered = df[~df.index.isin(result_blank["blank_indices"])]

    result_binary_format = DataQualityIssues.handle_binary_values(df_filtered, column)
    if result_binary_format['issue']:
        error_summary_parts.append(result_binary_format['dq_issue'] + ':\n ' + result_binary_format['error_message'] + '\n')

    # Creating a frequency table sorted by frequency (descending) and then alphabetically
    frequency_table = pd.DataFrame({'Value': list(binary_counts.keys()), 'Frequency': list(binary_counts.values())})
    frequency_table = frequency_table.sort_values(by=['Frequency', 'Value'], ascending=[False, True]).reset_index(drop=True)

    # Prepare the display table
    total_categories = len(frequency_table)
    if total_categories > 20:
        top_rows = frequency_table.head(10)
        bottom_rows = frequency_table.tail(10)
        ellipsis_row = pd.DataFrame([("...", "...")], columns=['Value', 'Frequency'])
        display_table = pd.concat([top_rows, ellipsis_row, bottom_rows], ignore_index=True)
    else:
        display_table = frequency_table

    frequency_distribution = f"\nFrequency Distribution (showing top and bottom 10 of {total_categories} binary values):\n{display_table.to_string(index=False)}\n"

    if not error_summary_parts:
        return f"All {total_values_count} binary values are valid.\n{frequency_distribution}"

    error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts)
    return error_summary + frequency_distribution

# Example usage
df_test = pd.DataFrame({
    'binary_column': ['1', '0', 'Yes', 'YES', 'y', 'no', 'true', 'False', 'Invalid', '2', 'Y', 0, 3, None, '', '?', '   ', "T", 0.1, '-2', 'no', '','  ',]
})

'''df_test = pd.DataFrame({
    'binary_column': ['1', '0', 'Yes', 'YES', 'y', 'no', 'true', 'False', 'Y', 0, "T", 'no']
})'''

result = check_binary_values(df_test, 'binary_column')
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Error(s) found: 
DQI #1 (Missing Data - Completeness):
 6 Blank/Empty/Null/Missing value(s) at index(es): 
1 NULL values: [(13, 'NULL')]
5 Non-NULL values: [(14, 'Empty'), (15, '?'), (16, 'Empty'), (21, 'Empty'), (22, 'Empty')]

DQI #15 (Domain Violation - Accuracy):
 5 Non-binary value(s) at index(es): [(8, 'Invalid'), (9, '2'), (12, 3), (18, 0.1), (19, '-2')]

Frequency Distribution (showing top and bottom 10 of 19 binary values):
  Value  Frequency
      0          2
[EMPTY]          2
[SPACE]          2
     no          2
     -2          1
    0.1          1
      1          1
      2          1
      3          1
      ?          1
  False          1
Invalid          1
   NULL          1
      T          1
      Y          1
    YES          1
    Yes          1
   true          1
      y          1

Last run on: 2025-04-04 19:28:37


29 Parse Analysis Result and Generate Summary

In [2001]:
from collections import Counter
import ast

# Define a dictionary to map function names to check numbers
check_numbers = {
    "check_numerical_ge_zero": 1,
    "check_numerical": 2,
    "check_numerical_between": 3,
    "check_id_attributes": 4,
    "check_string": 5,
    "check_categorical": 6,
    "check_month": 7,
    "check_weekday": 8,
    "check_date": 9,
    "check_datetime": 10,
    "check_time": 11,
    "check_model_name": 11.5,
    "check_name": 12,
    "check_street": 13,
    "check_city": 14,
    "check_state": 15,
    "check_country": 16,
    "check_postal_code": 17,
    "check_phone_numbers": 18,
    "check_ip_format": 19,
    "check_url_format": 20,
    "check_email_format": 21,
    "check_binary_values": 22
}

def parse_analysis_result(column, result, format_type, dataset_name, total_rows):
    summary_lines = []
    
    numerical_range_checks = ('year','age','percentage', 'ph', 'latitude', 'longitude', 'normalized','week','day',
                              'hour', 'tannins', 'saltness','heartrate','acidity','alkalinity','bloodpressure')

    if isinstance(result, dict):
        for key, value in result.items():
            dqi_matches = re.findall(r'DQI #(\d+) \((.*?)\):', value)

            if dqi_matches:  # Only process if DQIs are found
                for dqi_num, dq_issue_desc in dqi_matches:
                    # Separate DQ Issue Description from Data Quality Dimension
                    dq_parts = dq_issue_desc.split(' - ')
                    dq_issue_desc = dq_parts[0].strip()
                    data_quality_dim = dq_parts[1].strip() 

                    # Use regex with DOTALL flag to capture multiline content                 
                    issue_count_match = re.search(rf'DQI #{dqi_num}.*?(\d+)\s+(.*?)\s+(?:at index|value\(s\))', value, re.DOTALL)
                    if issue_count_match:
                        issue_count = int(issue_count_match.group(1))
                        error_desc = issue_count_match.group(2)
                    else:
                        issue_count = 0
                        error_desc = "See full analysis for details"

                    # Calculate error percentage
                    error_percentage = (issue_count / total_rows) * 100 if total_rows > 0 else "N/A"

                    if dqi_num == '1':
                        # Extract NULL values explicitly (handles optional "All" prefix)
                        null_values_match = re.search(r'(?:All )?NULL values:\s*(\[[^\]]*\])', value, re.DOTALL)

                        # Extract Non-NULL values explicitly
                        non_null_values_match = re.search(r'Non-NULL values:\s*(\[[^\]]*\])', value, re.DOTALL)

                        unique_values = set()

                        try:
                            if null_values_match:
                                null_data = ast.literal_eval(null_values_match.group(1))
                                for _, val in null_data:
                                    val_cleaned = val.strip("'\"") if isinstance(val, str) else val
                                    if val_cleaned and val_cleaned != '...':
                                        unique_values.add(f'"{val_cleaned}"')

                            if non_null_values_match:
                                non_null_data = ast.literal_eval(non_null_values_match.group(1))
                                for _, val in non_null_data:
                                    val_cleaned = val.strip("'\"") if isinstance(val, str) else val
                                    if val_cleaned and val_cleaned != '...':
                                        unique_values.add(f'"{val_cleaned}"')

                            # If still no values, explicitly handle the scenario
                            if not unique_values:
                                data_issues = "No specific values extracted"
                            else:
                                unique_values_sorted = sorted(unique_values)
                                data_issues = ', '.join(unique_values_sorted[:3])
                                if len(unique_values_sorted) > 3:
                                    data_issues += ", ..."
                        except Exception as e:
                            data_issues = f"Error parsing issue data: {str(e)}"

                    # Special handling for categorical data
                    elif dqi_num == '10':
                        categorical_match = re.search(r'Categorical format with (\d+) unique value\(s\):', value)
                        if categorical_match:
                            issue_count = int(categorical_match.group(1))
                            error_desc = "Too many categories"
                        data_issues = f"{issue_count} categories"
                        error_percentage = "N/A"
                        issue_count = 'N/A'

                    elif dqi_num == '9':  # Duplicates case
                     
                        # Try different patterns to extract duplicate values
                        patterns = [
                            r'\((\d+),\s*\'([a-fA-F0-9]{32})\'\)',  # Hash pattern
                            r'\[(\d+),\s*(\d+(?:\.\d+)?)\]',  # Original pattern
                            r'(\d+),\s*(\d+(?:\.\d+)?)',      # Without square brackets
                            r"'(\d+(?:\.\d+)?)'",             # Values in single quotes
                            r'"(\d+(?:\.\d+)?)"',             # Values in double quotes
                            r'\b(\d+(?:\.\d+)?)\b'            # Any numeric value
                        ]
                        
                        duplicate_values = []
                        for pattern in patterns:
                            duplicate_values = re.findall(pattern, value)
                        
                        if duplicate_values:
                            # If we found values with index, use the second item (the actual value)
                            if isinstance(duplicate_values[0], tuple) and len(duplicate_values[0]) > 1:
                                value_counts = Counter(v[1] for v in duplicate_values)
                            else:
                                value_counts = Counter(v for v in duplicate_values)
                            
                            # Convert to int if possible, otherwise keep as float                          
                            def convert_to_int_if_possible(v):
                                try:
                                    return int(float(v))
                                except ValueError:
                                    try:
                                        return float(v)
                                    except ValueError:
                                        return v  # Return the original string if it can't be converted
                            
                            
                            # Get all unique values, convert to int if possible, and sort
                            unique_values = sorted(set(convert_to_int_if_possible(v) for v in value_counts.keys()))
                            
                            # Format the output
                            if len(unique_values) <= 3:
                                data_issues = ', '.join(str(v) for v in unique_values)
                            else:
                                data_issues = ', '.join(str(v) for v in unique_values[:3]) + '...'
                        else:
                            data_issues = "Unable to extract duplicate values"

                    elif dqi_num == '19':  # Uniqueness violation case
                     
                        # Try different patterns to extract violation values
                        patterns = [
                            r'\((\d+),\s*\'([a-fA-F0-9]{32})\'\)',  # Hash pattern
                            r'\[(\d+),\s*(\d+(?:\.\d+)?)\]',  # Original pattern
                            r'(\d+),\s*(\d+(?:\.\d+)?)',      # Without square brackets
                            r"'(\d+(?:\.\d+)?)'",             # Values in single quotes
                            r'"(\d+(?:\.\d+)?)"',             # Values in double quotes
                            r'\b(\d+(?:\.\d+)?)\b'            # Any numeric value
                        ]
                        
                        violation_values = []
                        for pattern in patterns:
                            violation_values = re.findall(pattern, value)

                        
                        if violation_values:
                            # If we found values with index, use the second item (the actual value)
                            if isinstance(violation_values[0], tuple) and len(violation_values[0]) > 1:
                                values = [v[1] for v in violation_values]
                            else:
                                values = violation_values
                                                      
                            # Convert to int if possible, otherwise keep as float
                            def convert_to_int_if_possible(v):
                                try:
                                    return int(float(v))
                                except ValueError:
                                    try:
                                        return float(v)
                                    except ValueError:
                                        return v  # Return the original string if it can't be converted
                            
                            # Get all unique values, convert to int if possible, and sort
                            unique_values = sorted(set(convert_to_int_if_possible(v) for v in values))
                            
                            # Format the output
                            if len(unique_values) <= 3:
                                data_issues = ', '.join(str(v) for v in unique_values)
                            else:
                                data_issues = ', '.join(str(v) for v in unique_values[:3]) + '...'

                    elif dqi_num == '15' and 'name' in format_type.lower():
                        # Special handling for DQI #15 related to names
                        data_issues_match = re.search(rf'DQI #{dqi_num}.*?at index\(es\): \[(.*?)\]', value, re.DOTALL)
                        if data_issues_match:
                            data_issues = data_issues_match.group(1)
                            # Remove indices and other unnecessary characters, but keep apostrophes
                            data_issues = re.sub(r'\(\d+,\s*|\)', '', data_issues)
                            # Split by comma, strip whitespace, and remove outer quotes, but keep apostrophes within words
                            unique_issues = [re.sub(r'^["\']|["\']$', '', issue.strip()) for issue in data_issues.split(',') if issue.strip()]
                            # Remove duplicates while preserving order
                            unique_issues = list(dict.fromkeys(unique_issues))
                            
                            # Format the output
                            if len(unique_issues) <= 3:
                                data_issues = ', '.join(unique_issues)
                            else:
                                data_issues = ', '.join(unique_issues[:3]) + '...'
                        else:
                            data_issues = "N/A"

                    elif dqi_num == '6':  # Outdated Temporal Data
                       
                        issue_data = re.findall(r'(\d{4})-(\d{2})-(\d{2})', value)
                        unique_values = set()

                        for year, month, day in issue_data:
                            date_value = f'{int(day):02d}-{int(month):02d}-{int(year):04d}'
                            unique_values.add(date_value)

                        if len(unique_values) > 3:
                            data_issues = ', '.join(list(unique_values)[:3]) + ", ..."
                        else:
                            data_issues = ', '.join(unique_values)

                        error_desc = "Date value(s) not in [1800-2100] period"
                        # Ensure the data is passed correctly to the next step
                        add_quotes_to_items(data_issues)
                    else:
                        data_issues_match = re.search(rf'DQI #{dqi_num}.*?(?:at index\(es\): \[(.*?)\]|Sample values: \[(.*?)\])', value, re.DOTALL)
                        if data_issues_match:
                            data_issues = data_issues_match.group(1) or data_issues_match.group(2)
                            # Remove indices, quotes, parentheses, ellipsis, and leading/trailing commas
                            data_issues = re.sub(r'\(\d+,\s*|\)|\(|\'|\"|\.\.\.|^,\s*|,\s*$', '', data_issues)
                            
                            # Split, clean, and get unique values
                            unique_issues = list(set(issue.strip() for issue in data_issues.split(',') if issue.strip()))
                            
                            # Sort the unique issues to ensure consistent output
                            unique_issues.sort()
                            
                            # Format the output
                            if len(unique_issues) <= 3:
                                data_issues = ', '.join(unique_issues)
                            else:
                                data_issues = ', '.join(unique_issues[:3]) + '...'
                        else:
                            data_issues = "N/A"

                    # Determine the correct function name based on format_type
                    if format_type.lower() == 'categorical':
                        function_name = "check_categorical"
                    elif format_type.lower() == 'idcolumn':
                        function_name = "check_id_attributes"
                    elif format_type.lower() == 'numerical>=0':
                        function_name = "check_numerical_ge_zero"
                    elif format_type.lower() == 'modelname':
                        function_name = "check_model_name"
                    elif format_type.lower() == 'postalcode':
                        function_name = "check_postal_code"
                    elif format_type.lower() == 'phone':
                        function_name = "check_phone_numbers"
                    elif format_type.lower() == 'ipformat':
                        function_name = "check_ip_format"
                    elif format_type.lower() == 'urlformat':
                        function_name = "check_url_format"
                    elif format_type.lower() == 'e-mailformat':
                        function_name = "check_email_format"
                    elif format_type.lower() == 'binary':
                        function_name = "check_binary_values"
                    elif format_type.lower() in numerical_range_checks:
                        function_name = "check_numerical_between"
                    else:
                        function_name = f"check_{format_type.lower().replace(' ', '_')}"
                    
                    check_number = check_numbers.get(function_name, "N/A")
                     
                    if dqi_num == '10': 
                        data_issues_final = data_issues
                    else: 
                        data_issues_final = add_quotes_to_items(data_issues)

                    summary_line = {
                        "DQI#": dqi_num,
                        "DQ Issue Description": dq_issue_desc,
                        "Data Quality Dimension": data_quality_dim,
                        "Function Name": function_name,
                        "Check#": int(check_number) if str(check_number).isdigit() else check_number,
                        "Format Being Analyzed": format_type,
                        "Error Count": issue_count,
                        "Error Percentage": f"{error_percentage:.2f}%" if isinstance(error_percentage, float) else error_percentage,
                        "Error Explanation": error_desc,
                        "Data Issues": data_issues_final,
                        "Dataset": dataset_name,
                        "Columns - Attributes": column
                    }
                  
                    summary_lines.append(summary_line)       
    return summary_lines 

def generate_summary(all_results_ordered, analysed_columns_df, desired_dataset_index, dataset_name, total_rows):
    all_summary_lines = []
    
    for column, analysis_result in all_results_ordered.items():
       
        matching_rows = analysed_columns_df[
            (analysed_columns_df['index'] == desired_dataset_index) & 
            (analysed_columns_df['Column'] == column)
        ]
               
        format_type = matching_rows['FinalFormat'].iloc[0]

        if isinstance(analysis_result, dict):
            for key, value in analysis_result.items():
                    summary_lines = parse_analysis_result(column, {key: value}, format_type, dataset_name, total_rows)
                    all_summary_lines.extend(summary_lines)
        elif isinstance(analysis_result, str):
            summary_lines = parse_analysis_result(column, {"General": analysis_result}, format_type, dataset_name, total_rows)
            all_summary_lines.extend(summary_lines)
        else:
            summary_lines = parse_analysis_result(column, {"Unexpected": str(analysis_result)}, format_type, dataset_name, total_rows)
            all_summary_lines.extend(summary_lines)
    
    return all_summary_lines

def add_quotes_to_items(text):
    # Convert strings like "datetime.date(1001, 1, 1)" to "01-01-1001"
    date_pattern = re.compile(r"datetime\.date\((\d+),\s*(\d+),\s*(\d+)\)")
    
    def replace_date(match):
        year, month, day = match.groups()
        return f'{int(day):02d}-{int(month):02d}-{int(year):04d}'
    
    formatted_text = date_pattern.sub(replace_date, text)
    # Split the text by comma, preserving empty items and whitespace
    items = re.split(r'(,\s*)', formatted_text)
    
    quoted_items = []
    for i, item in enumerate(items):
        if i % 2 == 0:  # This is an actual item, not a separator
            stripped_item = item.strip()
            if stripped_item == '':
                quoted_items.append('""')  # Empty string becomes two quotes
            elif stripped_item.lower() == 'null':
                quoted_items.append('"NULL"')
            elif stripped_item.isspace():
                quoted_items.append(f'"{item}"')  # Preserve original whitespace
            elif not (stripped_item.startswith('"') and stripped_item.endswith('"')):
                quoted_items.append(f'"{stripped_item}"')
            else:
                quoted_items.append(stripped_item)  # Already quoted item
        else:  # This is a separator (comma and possible whitespace)
            quoted_items.append(item)

    result = ''.join(quoted_items)
    return result

def extract_and_categorize_dqi1_values(text: str) -> str:
    # Extract all values
    values = re.findall(r"'([^']*)'", text)
    print(values)
    # Categorize and represent values
    categorized_values = [DataQualityIssues.represent_value(v) for v in values]
    print('categorized_values', categorized_values)
    # Remove duplicates and 'Valid' values
    unique_issues = [v for v in set(categorized_values) if v != 'Valid' and v != '...']
    print('UNIQUE ISSUES ', unique_issues)
    if unique_issues:
        # Sort the unique issues
        sorted_issues = sorted(unique_issues)
        
        # Determine if we need to add '...'
        if len(sorted_issues) > 5  in categorized_values:
            display_issues = sorted_issues[:5] + ['...']
        else:
            display_issues = sorted_issues
        
        data_issues = ', '.join(f'"{v}"' for v in display_issues)
    
    return data_issues

def extract_duplicate_values(text: str) -> str:
    value_match = re.findall(r'\[(\d+),\s*\'([^\']+)\'\]', text)
    if value_match:
        duplicate_values = [v[1] for v in value_match]
        data_issues = ', '.join(f'"{v}"' for v in set(duplicate_values)[:5])
        if len(set(duplicate_values)) > 5:
            data_issues += ', ...'
    else:
        data_issues = "Duplicate or uniqueness violation found"
    return data_issues

def append_to_excel(df, filename, headers_iq_score, total_rows, total_columns, total_dqi_count, dataset_name):
    from openpyxl import load_workbook, Workbook
    from openpyxl.styles import PatternFill, Font
    from openpyxl.utils.dataframe import dataframe_to_rows
    from datetime import datetime
    import os

    # Prepare Excel workbook and sheet
    if os.path.exists(filename):
        book = load_workbook(filename)
        sheet = book.active
    else:
        book = Workbook()
        sheet = book.active
        sheet.title = "Summary"

    # Dataset summary values
    timestamp = datetime.now().strftime("%d-%m-%Y %H:%M")
    data_items = total_rows * total_columns

    # Ensure headers_iq_score is a float
    try:
        headers_iq_score = float(headers_iq_score)
    except (ValueError, TypeError):
        headers_iq_score = 0.0

    # Remove the HeadersIQ summary row from the DataFrame if present
    if not df.empty and "Columns - Attributes" in df.columns:
        df = df[df["Columns - Attributes"] != "HeadersIQ"]

    # Calculate total error count from cleaned df
    if not df.empty and "Error Count" in df.columns:
        try:
            df.loc[:, "Error Count"] = pd.to_numeric(df["Error Count"], errors='coerce').fillna(0)
            total_error_count = int(df["Error Count"].sum())
        except Exception:
            total_error_count = 0
    else:
        total_error_count = 0

    yellow_fill = PatternFill(start_color='FFFACD', end_color='FFFACD', fill_type='solid')
    bold_font = Font(bold=True)

    # Ensure the insertion happens after the last row with content
    last_row = sheet.max_row + 2

    # HeadersIQ summary block (two lines: titles and values), with 6 blank columns before
    summary_titles = ["", "", "", "", "", "", "Total Rows", "Columns", "Total Data Items", "Total Errors", "Dataset", "HeadersIQ", "Timestamp"]
    summary_values = ["", "", "", "", "", "", total_rows, total_columns, data_items,
                      total_error_count, dataset_name, headers_iq_score, timestamp]

    for col_idx, title in enumerate(summary_titles, start=1):
        cell = sheet.cell(row=last_row, column=col_idx, value=title)
        cell.font = bold_font

    for col_idx, value in enumerate(summary_values, start=1):
        cell = sheet.cell(row=last_row + 1, column=col_idx, value=value)
        cell.font = bold_font
        if col_idx == 12 and headers_iq_score < 100:
            cell.fill = yellow_fill

    # Append DQI rows only if HeadersIQ < 100 and df has data
    if headers_iq_score < 100 and not df.empty:
        sheet.append([])  # Blank row before DQI table

        # Append DQI header
        dqi_header = list(df.columns)
        sheet.append(dqi_header)
        for cell in sheet[sheet.max_row]:
            cell.font = bold_font

        for row in dataframe_to_rows(df, index=False, header=False):
            sheet.append(row)

    # Save workbook
    book.save(filename)
    print(f"Data appended to {filename}")


print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Last run on: 2025-04-04 19:28:37


30 Analyse Data Quality

In [2007]:
import re

def calculate_headers_iq_from_summary(summary_lines, total_columns):
    dq_issues_per_column = {}

    for line in summary_lines:
        col = line['Columns - Attributes']
        dqi_number = line['DQI#']
        percentage_str = str(line['Error Percentage']).replace('%','').strip()

        if percentage_str.upper() == "N/A":
            continue

        try:
            percentage = float(percentage_str)
        except ValueError:
            continue

        if col not in dq_issues_per_column:
            dq_issues_per_column[col] = {}

        dq_issues_per_column[col][dqi_number] = percentage

    # Reuse existing HeadersIQ logic
    return calculate_headers_iq(dq_issues_per_column, total_columns)

def calculate_headers_iq(dq_issues, total_columns):
    if total_columns == 0:
        return 100.0

    total_iq = 0
    for col in range(total_columns):
        column_name = list(dq_issues.keys())[col] if col < len(dq_issues) else None
        issues = dq_issues.get(column_name, {})
        
        column_iq = 100
        unique_issues = set(issues.keys())
        if '9' in unique_issues and '10' in unique_issues:
            unique_issues.remove('10')
        penalty = sum(min(issues[dqi], 100) for dqi in unique_issues)
        column_iq -= min(penalty, 100)
        total_iq += column_iq

    headers_iq = total_iq / total_columns
    return round(headers_iq, 2)


def analyse_data_quality(df, analysed_columns_df, desired_dataset_index):

    print(f"Start of Analysis on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    all_results_ordered = {}

    # Define a range for valid years and weeks
    min_valid_year = 1800
    max_valid_year = 2100
        
    categorical_threshold = 100

    # Initialize primary_key and foreign_key_columns
    primary_key = None
    foreign_key_columns = []
    primary_key_columns = []

    # Get PK and FK information for this dataset if available
    if all_datasets_info is not None:
        dataset_info = all_datasets_info[all_datasets_info['index'] == desired_dataset_index]
        if not dataset_info.empty:
            primary_key = dataset_info.iloc[0]['primary_key']
            if isinstance(primary_key, str):
                primary_key_columns = [col.strip() for col in primary_key.split(',')]
            else:
                primary_key_columns = []
            foreign_keys = dataset_info.iloc[0]['foreign_keys']
            print(f"Primary key columns: {primary_key_columns}")
            print(f"foreign_keys: {foreign_keys}")
            if pd.notna(foreign_keys) and isinstance(foreign_keys, str):
                foreign_key_columns = [fk.split(' -> ')[0] for fk in foreign_keys.split(', ')]
                print(f"foreign_key_columns: {foreign_key_columns}")
            else:
                print("No valid foreign keys found.")

    if (DB_or_dataset == "Dataset"):
        desired_dataset_index = int(desired_dataset_index)
    
    column_order = analysed_columns_df[analysed_columns_df['index'] == desired_dataset_index].sort_values('ID')['Column'].tolist()

    # Check if there's already a composite primary key column
    composite_pk_name = next((col for col in df.columns if '|' in col), None)

    if composite_pk_name:
        print(f"Found existing composite primary key: {composite_pk_name}")
        all_results_ordered[composite_pk_name] = {
            "Composite Primary Key format": check_id_attributes(df, composite_pk_name, is_foreign_key=False)
        }
        print(f"Analyzing composite primary key: {composite_pk_name}")
        
        # Remove individual primary key columns from the analysis but not from the DataFrame
        individual_pk_columns = composite_pk_name.split('|')
        column_order = [col for col in column_order if col not in individual_pk_columns]
        print(f"Removed individual primary key columns from analysis: {individual_pk_columns}")
    elif len(primary_key_columns) > 1:
        composite_pk_name = '|'.join(primary_key_columns)
        df[composite_pk_name] = df[primary_key_columns].astype(str).agg('|'.join, axis=1)
        all_results_ordered[composite_pk_name] = {
            "Composite Primary Key format": check_id_attributes(df, composite_pk_name, is_foreign_key=False)
        }
        print(f"Created and analyzing composite primary key: {composite_pk_name}")
        
        # Remove individual primary key columns from the analysis but not from the DataFrame
        column_order = [col for col in column_order if col not in primary_key_columns]
        print(f"Removed individual primary key columns from analysis: {primary_key_columns}")
    keyword = "categorical"  # Keyword to search for in the "FinalFormat"

    # Find columns marked as "categorical" for the specified dataset index
    categorical_columns = analysed_columns_df[
        (analysed_columns_df['index'] == desired_dataset_index) & 
        (analysed_columns_df['FinalFormat'].str.contains(keyword, case=False, na=False))
    ]['Column'].tolist()

    def handle_blob_column(df, column_name):
        def process_blob(value):
            if pd.isna(value):
                return 'NULL'
            elif isinstance(value, bytes):
                return 'BLOB'
            else:
                return str(value)

        df[column_name] = df[column_name].apply(process_blob)
        return df

    # Identify and process BLOB columns
    for column in df.columns:
        if df[column].dtype == 'object':
            # Check if any non-null value in the column is bytes
            if df[column].dropna().apply(lambda x: isinstance(x, bytes)).any():
                print(f"Identified BLOB column: {column}")
                df = handle_blob_column(df, column)
                
    # Iterate through the columns in the custom order and analyze them
    for column in column_order:
        # Ensure that the column exists in the DataFrame
        if column in dataset_df.columns:
            if column in categorical_columns:
                all_results_ordered[column] = {
                            "Categorical format": check_categorical(dataset_df, column, categorical_threshold)
                        }
            else:
                # Check the format specified in the "AnalysedColumns" sheet
                format_in_sheet = analysed_columns_df[
                    (analysed_columns_df['index'] == desired_dataset_index) &
                    (analysed_columns_df['Column'] == column)
                ]['FinalFormat'].iloc[0]
                
                if pd.isna(format_in_sheet):
                    all_results_ordered[column] = "Target word not found, and Format not determined"
                else:
                    if "IDcolumn" in format_in_sheet:
                        is_foreign_key = column in foreign_key_columns
                        print(f"Column: {column}, is_foreign_key: {is_foreign_key}")
                        id_check_result = check_id_attributes(dataset_df, column, is_foreign_key)
                        if column == primary_key:
                            all_results_ordered[column] = {
                                "Primary Key format": id_check_result,
                            }
                        elif is_foreign_key:
                            referenced_table = next((fk.split(' -> ')[1] for fk in foreign_keys.split(', ') if fk.startswith(column)), None)
                            all_results_ordered[column] = {
                                "Foreign Key format": id_check_result,
                                "References": referenced_table
                            }
                        else:
                            all_results_ordered[column] = {
                                "ID column format": id_check_result,
                            }
                    elif "numerical>=0" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Numerical >=0 format": check_numerical_ge_zero(dataset_df, column),
                        }
                    elif "percentage" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Percentage format": check_numerical_between(dataset_df, column, 0, 100),
                        }
                    elif "age" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Age format": check_numerical_between(dataset_df, column, 0, 130),
                        }
                    elif "numerical between 0 and 360" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Numerical (between 0 and 360) format": check_numerical_between(dataset_df, column, 0, 360),
                        }
                    elif "numerical between 0 and 60" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Numerical (between 0 and 60) format": check_numerical_between(dataset_df, column, 0, 60),
                        }
                    elif "numerical" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Numerical format": check_numerical(dataset_df, column),
                        }
                    elif "string" in format_in_sheet:
                        all_results_ordered[column] = {
                            "String format": check_string(dataset_df, column),
                        }
                    elif "datetime" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Datetime format": check_datetime(dataset_df, column),
                        }
                    elif "date" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Date format": check_date(dataset_df, column),
                        }
                    elif "time" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Time format": check_time(dataset_df, column),
                        }
                    elif "month" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Month format": check_month(dataset_df, column),
                        }
                    elif "year" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Year format": check_numerical_between(dataset_df, column, min_valid_year, max_valid_year),
                        }
                    elif "weekday" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Weekday format": check_weekday(dataset_df, column),
                        } 
                    elif "week" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Week format": check_numerical_between(dataset_df, column, 1, 53),
                        }
                    elif "day" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Day format" : check_numerical_between(dataset_df, column, 1, 366),
                        }
                    elif "hour" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Hour format" : check_numerical_between(dataset_df, column, 0, 24),
                        }
                    elif "modelname" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Model Name format": check_model_name(dataset_df, column),
                        }
                    elif "name" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Name format": check_name(dataset_df, column),
                        }
                    elif "street" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Street format": check_street(dataset_df, column),
                        }
                    elif "city" in format_in_sheet:
                        all_results_ordered[column] = {
                            "City format": check_city(dataset_df, column),
                        }
                    elif "state" in format_in_sheet:
                        all_results_ordered[column] = {
                            "State format": check_state(dataset_df, column),
                        }
                    elif "country" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Country format": check_country(dataset_df, column),
                        }
                    elif "postalcode" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Postal Code format": check_postal_code(dataset_df, column),
                        }  
                    elif "phone" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Phone format": check_phone_numbers(dataset_df, column),
                        }
                    elif "ph" in format_in_sheet:
                        all_results_ordered[column] = {
                            "pH format": check_numerical_between(dataset_df, column, 0, 14),
                        }    
                    elif "acidity" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Acidity format": check_numerical_between(dataset_df, column, 0, 7),
                        }                                                 
                    elif "alkalinity" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Alkalinity format": check_numerical_between(dataset_df, column, 7, 14),
                        }  
                    elif "tannins" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Tannins format": check_numerical_between(dataset_df, column, 0, 100),
                        }                         
                    elif "saltness" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Saltiness format": check_numerical_between(dataset_df, column, 0, 40),
                        }
                    elif "heartrate" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Heart Rate format": check_numerical_between(dataset_df, column, 40, 200),
                        } 
                    elif "bloodpressure" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Blood Pressure format": check_numerical_between(dataset_df, column, 0, 250),
                        }                        
                    elif "latitude" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Latitude format": check_numerical_between(dataset_df, column, -90, 90),
                        }
                    elif "longitude" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Longitude format": check_numerical_between(dataset_df, column, -180, 180),
                        }
                    elif "normalized" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Normalized format": check_numerical_between(dataset_df, column, 0, 1),
                        }
                    elif "IPformat" in format_in_sheet:
                        all_results_ordered[column] = {
                            "IP Address format": check_ip_format(df, column),
                        }
                    elif "URLformat" in format_in_sheet:
                        url_result = check_url_format(df, column)
                        all_results_ordered[column] = {
                            "URL format": url_result
                        }
                    elif "E-mailformat" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Email format": check_email_format(df, column),
                        }
                    elif "binary" in format_in_sheet:
                        all_results_ordered[column] = {
                            "Binary format": check_binary_values(df, column),
                        }
                        # After each analysis, print the result
                        print(f"Analysis result for {column}:")
                        print(all_results_ordered[column])
                    else:
                        print(f"Column '{column}' not found in the dataset.")
                        all_results_ordered[column] = {
                            "Error": f"Column '{column}' not found in the dataset."
                        }

    # Loop to print the results in the desired format
    for column, analysis_result in all_results_ordered.items():
        # Fetch the 'SourceKeyword' for this column from 'analysed_columns_df'
        if '|' in column:  # This is a composite primary key
            print(f"{column} (Composite Primary Key):")
        else:
            try:
                SourceKeyword_value = analysed_columns_df.loc[(analysed_columns_df['index'] == desired_dataset_index) & (analysed_columns_df['Column'] == column), 'SourceKeyword'].iloc[0]
                # Check if 'SourceKeyword' is not NaN before converting to lower case
                if pd.notna(SourceKeyword_value) and SourceKeyword_value.lower() != column.lower():
                    print(f"{column} ({SourceKeyword_value}):")
                else:
                    print(f"{column}:")
            except IndexError:
                print(f"{column}:")
        
        if isinstance(analysis_result, pd.DataFrame):
            print(analysis_result.to_string(index=False))
        elif isinstance(analysis_result, dict):
            for key, value in analysis_result.items():
                print(f"  {key}: {value}")
        elif isinstance(analysis_result, str):  # Added condition to handle string results
            print(f"  {analysis_result}")
        else:
            print("No results available.")
        
        print()  # Add an empty line for separation
    
    # Return the results as well
    return all_results_ordered

def generate_pk_fk_summary(all_results_ordered, primary_key, foreign_key_columns):
    summary = "Primary and Foreign Key Analysis:\n\n"
    
    if primary_key in all_results_ordered:
        pk_result = all_results_ordered[primary_key].get("Primary Key format")
        if isinstance(pk_result, dict) and pk_result.get("issue"):
            summary += f"Primary Key '{primary_key}' has issues: {pk_result['error_message']}\n"
        else:
            summary += f"Primary Key '{primary_key}' is valid.\n"
    
    for column in foreign_key_columns:
        if column in all_results_ordered:
            fk_result = all_results_ordered[column].get("Foreign Key format")
            referenced_table = all_results_ordered[column].get("References")
            if isinstance(fk_result, dict) and fk_result.get("issue"):
                summary += f"Foreign Key '{column}' (referencing {referenced_table}) has issues: {fk_result['error_message']}\n"
            else:
                summary += f"Foreign Key '{column}' (referencing {referenced_table}) is valid.\n"
    
    return summary

# Main execution

# Load AllDatasetsInfo if available
try:
    all_datasets_info = pd.read_excel("AllDatasetsInfo.xlsx")      
       
except FileNotFoundError:
    all_datasets_info = None

if all_datasets_info is not None:
    dataset_info = all_datasets_info[all_datasets_info['index'] == desired_dataset_index]
    if not dataset_info.empty:
        primary_key = dataset_info.iloc[0]['primary_key']
        foreign_keys = dataset_info.iloc[0]['foreign_keys']
        print(f"Dataset info for index {desired_dataset_index}:")
        print(dataset_info)
        print(f"Primary key: {primary_key}")
        print(f"Foreign keys: {foreign_keys}")
        if pd.notna(foreign_keys) and isinstance(foreign_keys, str):
            foreign_key_columns = [fk.split(' -> ')[0] for fk in foreign_keys.split(', ')]
            print("Foreign key columns:", foreign_key_columns)
        else:
            print(f"Invalid foreign keys value: {foreign_keys}")
            foreign_key_columns = []
    
dataset_name = desired_dataset_index  
results = analyse_data_quality(dataset_df, analysed_columns_df, desired_dataset_index)

# Get the total number of rows in the dataset
total_rows = len(dataset_df)

# get all parsed results
summary_lines = generate_summary(results, analysed_columns_df, desired_dataset_index, dataset_name, total_rows)

# compute and inject HeadersIQ
total_columns = analysed_columns_df[analysed_columns_df['index'] == desired_dataset_index].shape[0]
headers_iq_score = calculate_headers_iq_from_summary(summary_lines, total_columns)

# Count DQIs (one per summary line that has a DQI#)
#total_dqi_count = sum(1 for line in summary_lines if str(line.get("DQI#", "")).isdigit())
total_dqi_count = len([line for line in summary_lines if line['DQI#']])
# Obter timestamp
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Linha de títulos personalizados (apenas para o resumo HeadersIQ)
header_summary_titles = {
    'DQI#': '',
    'DQ Issue Description': '',
    'Data Quality Dimension': '',
    'Function Name': '',
    'Check#': '',
    'Format Being Analyzed': '',
    'Error Count': 'Total Rows',
    'Error Percentage': 'Columns',
    'Error Explanation': 'Total Data Items',
    'Data Issues': 'DQIs Found',
    'Dataset': '',
    'Columns - Attributes': 'HeadersIQ',
    'Timestamp': ''
}

headers_iq_summary = {
    'DQI#': '',
    'DQ Issue Description': '',
    'Data Quality Dimension': '',
    'Function Name': '',
    'Check#': '',
    'Format Being Analyzed': '',
    'Error Count': total_rows,
    'Error Percentage': total_columns,
    'Error Explanation': total_rows * total_columns,
    'Data Issues': total_dqi_count,
    'Dataset': dataset_name,
    'Columns - Attributes': 'HeadersIQ',
    'Timestamp': timestamp
}

summary_lines.insert(0, headers_iq_summary)
summary_lines.insert(0, header_summary_titles)


# build your DataFrame and write to Excel
if summary_lines:
    summary_df = pd.DataFrame(summary_lines)
    # Add the Timestamp column here
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    summary_df['Timestamp'] = timestamp
    
    # Ensure Timestamp is the last column (this is now safe to do)
    columns = summary_df.columns.tolist()
    columns.remove('Timestamp')
    columns.append('Timestamp')
    summary_df = summary_df[columns]

    excel_filename = "SummaryofDQIssuesDiscovered.xlsx"
    append_to_excel(summary_df, excel_filename, headers_iq_score, total_rows, total_columns, total_dqi_count, dataset_name)
    print(f"Analysis completed and summary appended. Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
else:
    print("No summary lines generated. Nothing to append to Excel.")

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Start of Analysis on: 2025-04-04 19:34:48
Analysis result for y - has the client subscribed a term deposit?:
{'Binary format': 'All 41188 binary values are valid.\n\nFrequency Distribution (showing top and bottom 10 of 2 binary values):\nValue  Frequency\n   no      36548\n  yes       4640\n'}
age:
  Age format: All 41188 values are numerical and valid in the range [0, 130].
Actual range of values: (17 : 98)

job:
  Categorical format: Error(s) found: 
DQI #4 (Ambiguous Data - Accuracy, Consistency):
 330 Unacceptable value(s) at index(es): [(29, 'unknown'), (35, 'unknown'), (73, 'unknown'), (91, 'unknown'), (144, 'unknown'), (299, 'unknown'), (303, 'unknown'), (343, 'unknown'), (388, 'unknown'), (428, 'unknown'), ('...', '...'), (40058, 'unknown'), (40208, 'unknown'), (40240, 'unknown'), (40244, 'unknown'), (40289, 'unknown'), (40370, 'unknown'), (40428, 'unknown'), (40656, 'unknown'), (41005, 'unknown'), (41108, 'unknown')] (displaying only the first and last 10 items)

Categorical f

EXIT


In [2003]:
raise SystemExit

SystemExit: 

c:\Users\290099c\AppData\Local\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
